In [ ]:
# ============================================================
# BLOCK 1/6 — CONFIG + LOGIN + USER INPUT + STRIKE SELECTION
#           + LOGGING + TRUE OPEN (PERSIST + APPLY) ✅
#           + ✅ CANONICAL TIMESTAMP FIX (MANDATORY)
#           + ✅ REST HARDENING (RETRY + SAFE-STOP) (MANDATORY)
#           + ✅ PAPER MODE LIVE-ORDER LOCKOUT (MANDATORY)
#
# ✅ UPDATED: Adds required global_state fields for ATR(7) + N19 High-Win
# ✅ UPDATED: Ensures CE/PE state gets missing keys even if already created
# ============================================================

!pip install kiteconnect rich --quiet

# ----------------------------
# SAFETY: fail-fast toggle (does not change trading logic unless enabled)
# Set FAIL_FAST=True to surface exceptions instead of silently continuing.
# ----------------------------
FAIL_FAST = False

import os, re, time, math, json, csv, queue, threading, datetime, traceback, copy
from collections import defaultdict, deque
from dataclasses import dataclass
from typing import Optional, Dict, List, Any, Tuple

import numpy as np
import pandas as pd
import requests
from kiteconnect import KiteConnect, KiteTicker

# ----------------------------
# BASIC RUN SETTINGS
# ----------------------------
DEMO_MODE       = False
PAPER_TRADING   = True

# ----------------------------
# PERFORMANCE TOGGLES (NEW)
# ----------------------------
USE_TREND_FILTER = bool(globals().get("USE_TREND_FILTER", True))  # lightweight trend gate
SEED_VWAP_FROM_PREV_DAY = bool(globals().get('SEED_VWAP_FROM_PREV_DAY', True))
BANKNIFTY_ITM_STEPS = int(globals().get('BANKNIFTY_ITM_STEPS', 1) or 1)  # default 1 step ITM
NIFTY_ITM_STEPS = int(globals().get('NIFTY_ITM_STEPS', 0) or 0)

ALLOW_CONCURRENT_LAYERS = True

globals()["KEEP_NOTEBOOK_ALIVE"] = True
KEEP_NOTEBOOK_ALIVE = bool(globals().get("KEEP_NOTEBOOK_ALIVE", True))

# Strike selection window
STRIKE_SELECT_START = datetime.time(9, 9, 0)
STRIKE_SELECT_END   = datetime.time(9, 14, 59)
STRIKE_SELECT_POLL_SEC = 3.0

# ----------------------------
# STRIKE CANDIDATE SHORTLIST (UPDATED — ATM + 0..N ITM, hot-swap at entry)
# ----------------------------
# We keep candidates tight for liquidity + fast movement (acts like high-delta without Greeks).
NUM_CANDIDATES = int(globals().get("NUM_CANDIDATES", 5) or 5)  # N steps beyond ATM (ATM + N)
MAX_ITM_STEPS = int(globals().get("MAX_ITM_STEPS", NUM_CANDIDATES) or NUM_CANDIDATES)  # keeps older name for compatibility
CANDIDATE_COUNT = int(globals().get("CANDIDATE_COUNT", MAX_ITM_STEPS + 1) or (MAX_ITM_STEPS + 1))

# Entry-time eligibility (applies to GL/GB/N19). Tune as needed.
MIN_PREMIUM_NIFTY_WEEKLY    = float(globals().get("MIN_PREMIUM_NIFTY_WEEKLY", 108) or 108)
MIN_PREMIUM = float(globals().get("MIN_PREMIUM", MIN_PREMIUM_NIFTY_WEEKLY) or MIN_PREMIUM_NIFTY_WEEKLY)
MIN_PREMIUM_NIFTY_WEEKLY = MIN_PREMIUM  # unify knobs (eligibility uses this)
REQUIRE_OI_FOR_ENTRY      = bool(globals().get("REQUIRE_OI_FOR_ENTRY", False))
SPREAD_PCT_CAP              = float(globals().get("SPREAD_PCT_CAP", 0.003) or 0.003)   # 0.30%
SPREAD_ABS_CAP              = float(globals().get("SPREAD_ABS_CAP", 0.30) or 0.30)     # backup absolute cap
OI_THRESHOLD_NIFTY_WEEKLY   = int(globals().get("OI_THRESHOLD_NIFTY_WEEKLY", 10000) or 10000)

# ----------------------------
# HOT-SWAP / CANDIDATES
# ----------------------------
# Disable by default to prevent log spam and keep CE/PE fixed for the day.
ENABLE_HOTSWAP = bool(globals().get("ENABLE_HOTSWAP", False))

# ----------------------------
# TRUE OPEN DAY LOCK
# ----------------------------
# If TRUE OPEN for a leg is >= this threshold, lock it for the rest of the day.
TRUE_OPEN_FIX_THRESHOLD = float(globals().get("TRUE_OPEN_FIX_THRESHOLD", 141) or 141)

# Greeks/Delta are not exposed in Zerodha ticks; keep disabled (compat fields kept but unused).
REQUIRE_DELTA_FOR_ENTRY     = False
MIN_ABS_DELTA_DEFAULT       = float(globals().get("MIN_ABS_DELTA_DEFAULT", 0.50) or 0.50)  # unused (compat)
# Strike step sizes (fallback if not derived)
STRIKE_STEP_NIFTY   = int(globals().get("STRIKE_STEP_NIFTY", 50) or 50)
STRIKE_STEP_BANKNIFTY= int(globals().get("STRIKE_STEP_BANKNIFTY", 100) or 100)

# True Open pipeline toggles
TRUE_OPEN_USE_WS_OHLC_OPEN = True
TRUE_OPEN_USE_REST_OHLC_OPEN = True
TRUE_OPEN_CANDLE_LAST_RESORT_ENABLE = True
TRUE_OPEN_CANDLE_LAST_RESORT_START_SEC = 70
TRUE_OPEN_CANDLE_LAST_RESORT_POLL_INTERVAL = 2.0

# Market times
IST = datetime.timezone(datetime.timedelta(hours=5, minutes=30))
UTC = datetime.timezone.utc
market_open_time  = datetime.time(9, 15, 0)
market_close_time = datetime.time(15, 30, 0)
SQUARE_OFF_TIME   = datetime.time(15, 15, 0)

# Costs (for net PnL)
SLIPPAGE_PER_SIDE   = float(globals().get("SLIPPAGE_PER_SIDE", 0.18) or 0.18)
BROKERAGE_PER_ORDER = float(globals().get("BROKERAGE_PER_ORDER", 20) or 20)

# REST timeouts / retries (MANDATORY)
REST_TIMEOUT_SEC = float(globals().get("REST_TIMEOUT_SEC", 6.0) or 6.0)
REST_RETRIES     = int(globals().get("REST_RETRIES", 3) or 3)
REST_RETRY_SLEEP = float(globals().get("REST_RETRY_SLEEP", 0.6) or 0.6)

# ------------------------------------------------------------
# GLOBAL SAFE-STOP (MANDATORY)
# ------------------------------------------------------------
block_stop = globals().get("block_stop")
if block_stop is None:
    block_stop = threading.Event()
globals()["block_stop"] = block_stop
globals()["SAFE_STOP_REASON"] = globals().get("SAFE_STOP_REASON", "")

# Fresh-start reset (prevents stale SAFE_STOP from previous runs in Colab)
if block_stop.is_set():
    try:
        print("Resetting stale SAFE_STOP from previous run:", globals().get("SAFE_STOP_REASON",""))
    except:
        if FAIL_FAST: raise
        pass
    block_stop.clear()
    globals()["SAFE_STOP_REASON"] = ""

def safe_stop(reason: str, exc: Exception = None, fatal: bool = True):
    """
    Sets block_stop and records reason. Designed to fail closed.
    """
    try:
        msg = f"🛑 SAFE-STOP: {reason}"
        if exc is not None:
            msg += f" | {type(exc).__name__}: {exc}"
        try:
            print(msg)
        except:
            if FAIL_FAST: raise
            pass
        try:
            fn = globals().get("tg_throttled")
            if callable(fn):
                fn("safe_stop", msg, 10)
            else:
                fn2 = globals().get("tg")
                if callable(fn2):
                    fn2(msg)
        except:
            if FAIL_FAST: raise
            pass
        globals()["SAFE_STOP_REASON"] = reason
        try:
            block_stop.set()
        except:
            if FAIL_FAST: raise
            pass
        # best-effort stop WS if already started
        try:
            w = globals().get("ws")
            if w is not None:
                try: w.close()
                except:
                    if FAIL_FAST: raise
                    pass
        except:
            if FAIL_FAST: raise
            pass
        if fatal:
            raise RuntimeError(msg)
    except RuntimeError:
        raise
    except Exception:
        raise RuntimeError(f"🛑 SAFE-STOP: {reason}")

globals()["safe_stop"] = safe_stop

# ----------------------------
# ✅ CANONICAL TIMESTAMP CONVERTER (MANDATORY)
# ----------------------------
def as_ist_dt(d: datetime.datetime, now_ist: Optional[datetime.datetime] = None) -> Optional[datetime.datetime]:
    try:
        if d is None or not isinstance(d, datetime.datetime):
            return None
        # tz-aware -> convert
        if d.tzinfo is not None:
            return d.astimezone(IST)

        now_ist = now_ist or datetime.datetime.now(IST)

        cand_ist = d.replace(tzinfo=IST)
        cand_utc = d.replace(tzinfo=UTC).astimezone(IST)

        # choose closer to wall clock
        if abs((cand_utc - now_ist).total_seconds()) < abs((cand_ist - now_ist).total_seconds()):
            return cand_utc
        return cand_ist
    except:
        return None

def _as_ist_dt(d: datetime.datetime) -> Optional[datetime.datetime]:
    try:
        return as_ist_dt(d, datetime.datetime.now(IST))
    except:
        return None

globals()["as_ist_dt"] = as_ist_dt
globals()["_as_ist_dt"] = _as_ist_dt

def now_ist() -> datetime.datetime:
    return datetime.datetime.now(IST)
globals()["now_ist"] = now_ist

# ----------------------------
# TWISTED SIGNAL FIX (Colab)
# ----------------------------
def _patch_twisted_no_signalhandlers():
    try:
        from twisted.internet import reactor
        if getattr(reactor, "_nandini_no_signals", False):
            return
        _orig_run = reactor.run
        def _run_no_signals(*a, **k):
            k["installSignalHandlers"] = False
            return _orig_run(*a, **k)
        reactor.run = _run_no_signals
        reactor._nandini_no_signals = True
    except:
        if FAIL_FAST: raise
        pass

_patch_twisted_no_signalhandlers()

# ----------------------------
# FOLDER / PATHS
# ----------------------------
try:
    from google.colab import drive
    drive.mount('/content/drive')
    _IS_COLAB = True
except:
    _IS_COLAB = False

BOT_START_DT = datetime.datetime.now(IST)
TODAY_DATE = BOT_START_DT.date()  # datetime.date
TODAY = TODAY_DATE.isoformat()  # keep string for paths/logs
RUN_ID = BOT_START_DT.strftime("%H%M%S")

if _IS_COLAB:
    BASE_FOLDER = f"/content/drive/MyDrive/NANDINI19_REPORTS/{TODAY}/"
else:
    BASE_FOLDER = os.path.join(os.getcwd(), f"NANDINI19_REPORTS/{TODAY}/")

os.makedirs(BASE_FOLDER, exist_ok=True)

TICK_LOG_PATH  = os.path.join(BASE_FOLDER, "ticks.csv")
TRADES_CSV     = os.path.join(BASE_FOLDER, "trades.csv")
TRUE_OPEN_FILE = os.path.join(BASE_FOLDER, "true_open.json")
STATE_FILE     = os.path.join(BASE_FOLDER, "state.json")
ACCESS_FILE    = os.path.join(BASE_FOLDER, "access_token.json")
DIAG_CSV       = os.path.join(BASE_FOLDER, "diagnostics.csv")

print(f"📁 REPORT FOLDER: {BASE_FOLDER}")
print(f"🆔 RUN_ID: {RUN_ID}")

# ----------------------------
# TELEGRAM + KITE CREDS
# NOTE:
# For convenience, defaults are kept here (same as your original notebook).
# For better security, prefer setting env vars instead of hardcoding:
#   - KITE_API_KEY, KITE_API_SECRET, TG_BOT, TG_CHAT
# ----------------------------
API_KEY    = os.getenv("KITE_API_KEY", "").strip()
API_SECRET = os.getenv("KITE_API_SECRET", "").strip()
if not API_KEY or not API_SECRET:
    safe_stop("Missing KITE_API_KEY / KITE_API_SECRET (env vars empty).", fatal=True)

TELEGRAM_BOT = os.getenv("TG_BOT", "").strip()
CHAT_ID      = os.getenv("TG_CHAT", "").strip()

def tg(msg: str):
    if not TELEGRAM_BOT or not CHAT_ID:
        return
    try:
        m = str(msg)
        if len(m) > 3800:
            m = m[:3800] + "…"
        requests.get(
            f"https://api.telegram.org/bot{TELEGRAM_BOT}/sendMessage",
            params={"chat_id": CHAT_ID, "text": m},
            timeout=5
        )
    except:
        if FAIL_FAST: raise
        pass

def log(msg: str):
    try:
        print(msg)
    except:
        if FAIL_FAST: raise
        pass
    try:
        tg(msg)
    except:
        if FAIL_FAST: raise
        pass

globals()["log"] = log
globals()["tg"] = tg

def tg_throttled(key: str, msg: str, cooldown_sec: int = 60):
    if not hasattr(tg_throttled, "_mem"):
        tg_throttled._mem = {}
        tg_throttled._lock = threading.Lock()
    try:
        noww = time.time()
        with tg_throttled._lock:
            last = tg_throttled._mem.get(key, 0.0)
            if (noww - last) < float(cooldown_sec):
                return
            tg_throttled._mem[key] = noww
        tg(msg)
    except:
        if FAIL_FAST: raise
        pass

globals()["tg_throttled"] = tg_throttled

# ----------------------------
# DIAGNOSTICS CSV (best-effort)
# ----------------------------
def diag(event: str, **fields):
    try:
        row = {"ts": datetime.datetime.now(IST).strftime("%Y-%m-%d %H:%M:%S"), "event": str(event)}
        for k,v in (fields or {}).items():
            try:
                row[str(k)] = str(v)
            except:
                row[str(k)] = ""
        exists = os.path.exists(DIAG_CSV)
        with open(DIAG_CSV, "a", newline="") as f:
            w = csv.DictWriter(f, fieldnames=list(row.keys()))
            if not exists or os.path.getsize(DIAG_CSV) == 0:
                w.writeheader()
            w.writerow(row)
    except:
        if FAIL_FAST: raise
        pass
globals()["diag"] = diag

# ----------------------------
# ACCESS TOKEN CACHE
# ----------------------------
def _save_access_token(token: str):
    try:
        with open(ACCESS_FILE, "w") as f:
            json.dump({"date": TODAY, "access_token": token}, f)
    except:
        if FAIL_FAST: raise
        pass

def _load_access_token():
    try:
        if not os.path.exists(ACCESS_FILE):
            return None
        with open(ACCESS_FILE, "r") as f:
            d = json.load(f)
        if d.get("date") == TODAY and d.get("access_token"):
            return d["access_token"]
        return None
    except:
        return None

def _extract_request_token(s: str) -> str:
    s = (s or "").strip()
    if "request_token=" in s:
        m = re.search(r"request_token=([^&]+)", s)
        if m:
            return m.group(1).strip()
    return s

def _extract_access_token(s: str) -> str:
    s = (s or "").strip()
    if "access_token=" in s:
        m = re.search(r"access_token=([^&]+)", s)
        if m:
            return m.group(1).strip()
    return ""

kite = KiteConnect(api_key=API_KEY)

ACCESS_TOKEN = _load_access_token()
if ACCESS_TOKEN:
    kite.set_access_token(ACCESS_TOKEN)
    log("✅ ACCESS_TOKEN restored from cache (today).")
else:
    input("\nPress Enter to print the Kite login URL (then login in your browser)...")
    login_url = kite.login_url()
    print("\n🔐 KITE LOGIN URL:\n", login_url, "\n")

    rt_in = input("Paste request_token (or full redirect URL / access_token): ").strip()

    at = _extract_access_token(rt_in)
    if at:
        ACCESS_TOKEN = at
        kite.set_access_token(ACCESS_TOKEN)
        log("✅ ACCESS_TOKEN set from pasted value (valid only for today).")
        _save_access_token(ACCESS_TOKEN)
    else:
        request_token = _extract_request_token(rt_in)
        try:
            sess = kite.generate_session(request_token, api_secret=API_SECRET)
            ACCESS_TOKEN = sess["access_token"]
            kite.set_access_token(ACCESS_TOKEN)
            _save_access_token(ACCESS_TOKEN)
            log("✅ ACCESS_TOKEN generated & saved for today.")
        except Exception as e:
            safe_stop("Kite generate_session() failed", e, fatal=True)

globals()["kite"] = kite
globals()["API_KEY"] = API_KEY
globals()["ACCESS_TOKEN"] = ACCESS_TOKEN
# ----------------------------
# ✅ PAPER MODE LIVE-ORDER LOCKOUT (MANDATORY)
# ----------------------------
def _paper_mode_lockout(kite_obj):
    try:
        if not PAPER_TRADING:
            return
        for m in ("place_order", "modify_order", "cancel_order", "place_gtt", "delete_gtt"):
            if hasattr(kite_obj, m) and callable(getattr(kite_obj, m)):
                orig = getattr(kite_obj, m)
                setattr(kite_obj, f"_orig_{m}", orig)

                def _blocked(*a, **k):
                    raise RuntimeError("PAPER_TRADING=True: LIVE ORDER ENDPOINT BLOCKED (unlock only by setting PAPER_TRADING=False).")
                setattr(kite_obj, m, _blocked)
        log("✅ PAPER MODE LOCKOUT enabled: live order endpoints blocked.")
        diag("paper_lockout", enabled=True)
    except Exception as e:
        safe_stop("Paper lockout failed (do not run live)", e, fatal=True)

_paper_mode_lockout(kite)

# ----------------------------
# REST SAFE WRAPPERS (MANDATORY)
# ----------------------------
def _rest_retry(label: str, fn, *a, **k):
    """REST wrapper with retries that fails closed.

    IMPORTANT:
    - If SAFE-STOP is active, we raise immediately (never return None).
    - This prevents downstream NoneType crashes like: q[SPOT_SYMBOL] when quotes are blocked.
    """
    last = None
    for i in range(1, REST_RETRIES + 1):
        if block_stop.is_set():
            raise RuntimeError(
                f"SAFE_STOP active -> {globals().get('SAFE_STOP_REASON','')} | while calling {label}"
            )
        try:
            return fn(*a, **k)
        except Exception as e:
            last = e
            diag("rest_fail", label=label, attempt=i, err=f"{type(e).__name__}: {e}")
            time.sleep(REST_RETRY_SLEEP * i)

    # Final failure: trip SAFE-STOP (fatal=True) and raise
    safe_stop(f"REST failed after retries: {label}", last, fatal=True)

def kite_instruments(exchange: str):
    return _rest_retry(f"instruments({exchange})", kite.instruments, exchange)

def kite_quote(symbols: List[str]):
    return _rest_retry(f"quote({len(symbols)})", kite.quote, symbols)

globals()["kite_instruments"] = kite_instruments
globals()["kite_quote"] = kite_quote

# ----------------------------
# USER INPUT (required)
# ----------------------------
UNDERLYING = input("UNDERLYING (NIFTY / BANKNIFTY): ").strip().upper()
if UNDERLYING not in ["NIFTY", "BANKNIFTY"]:
    safe_stop("UNDERLYING must be NIFTY or BANKNIFTY", fatal=True)

EXPIRY_TYPE = input("EXPIRY TYPE (WEEKLY / MONTHLY): ").strip().upper()
if EXPIRY_TYPE not in ["WEEKLY", "MONTHLY"]:
    safe_stop("EXPIRY TYPE must be WEEKLY or MONTHLY", fatal=True)

LOTS = int(float(input("LOTS (e.g., 1): ").strip() or "1"))

globals()["UNDERLYING"] = UNDERLYING
globals()["EXPIRY_TYPE"] = EXPIRY_TYPE
globals()["LOTS"] = LOTS
globals()["RUN_ID"] = RUN_ID
globals()["TODAY"] = TODAY
globals()["TODAY_DATE"] = TODAY_DATE
globals()["IST"] = IST
globals()["UTC"] = UTC

# ----------------------------
# AUTO PARAM SET (GL & GB) — deterministic (applied ONCE at startup)
# ----------------------------
# IMPORTANT:
# - MAX_SPREAD is controlled ONLY by this param set.
# - No downstream logic is allowed to override MAX_SPREAD.

PARAM_SETS = {
    ("NIFTY", "WEEKLY"): {
        "PARAM_SET_NAME": "NIFTY_WEEKLY",
        "IMB_MIN": 1.70,
        "MAX_SPREAD": 2.29,
        "GL_SL_ATR_MULT": 2.21,
        "GL_MIN_SL_POINTS": 12.91,
        "GL_TRAIL_START": "T4",
        "GB_T2_FLOOR_OFFSET": 5.0,
        "GB_INITIAL_SL_MODE": "FIXED20",
        "LOT_SIZE": 50},
    # Default fallback for other combos (keeps prior behavior unless explicitly tuned)
    ("DEFAULT", "DEFAULT"): {
        "PARAM_SET_NAME": "NIFTY_WEEKLY",
        "IMB_MIN": 1.70,
        "MAX_SPREAD": 2.29,
        "GL_SL_ATR_MULT": 2.21,
        "GL_MIN_SL_POINTS": 12.91,
        "GL_TRAIL_START": "T4",
        "GB_T2_FLOOR_OFFSET": 5.0,
        "GB_INITIAL_SL_MODE": "FIXED20",
        "LOT_SIZE": 50},
    ("BANKNIFTY", "MONTHLY"): {
        "PARAM_SET_NAME": "BANKNIFTY_MONTHLY",
        "IMB_MIN": 1.44,
        "MAX_SPREAD": 8.0,
        "GL_SL_ATR_MULT": 2.00,
        "GL_MIN_SL_POINTS": 28.0,
        "GL_TRAIL_START": "T3",
        "GB_T2_FLOOR_OFFSET": 12.0,
        "GB_INITIAL_SL_MODE": "ATR",
        "LOT_SIZE": 30}}

def apply_param_set(underlying: str, expiry_type: str, *, force: bool = False) -> dict:
    """Apply param set exactly once at startup (no runtime overrides)."""
    if globals().get("_PARAM_SET_LOCKED", False) and not force:
        # Already applied – keep deterministic behavior.
        return dict(globals().get("ACTIVE_PARAM_SET", {}) or {})

    u = str(underlying or "").strip().upper()
    e = str(expiry_type or "").strip().upper()
    ps = PARAM_SETS.get((u, e)) or PARAM_SETS.get(("DEFAULT", "DEFAULT"))
    if not isinstance(ps, dict):
        ps = {}

    # Assign core params
    PARAM_SET_NAME = str(ps.get("PARAM_SET_NAME", f"{u}_{e}") or f"{u}_{e}")
    IMB_MIN = float(ps.get("IMB_MIN", 1.70) or 1.70)
    MAX_SPREAD = float(ps.get("MAX_SPREAD", 2.29) or 2.29)
    GL_SL_ATR_MULT = float(ps.get("GL_SL_ATR_MULT", 2.21) or 2.21)
    GL_MIN_SL_POINTS = float(ps.get("GL_MIN_SL_POINTS", 12.91) or 12.91)
    GL_TRAIL_START = str(ps.get("GL_TRAIL_START", "T4") or "T4")
    GB_T2_FLOOR_OFFSET = float(ps.get("GB_T2_FLOOR_OFFSET", 5.0) or 5.0)
    GB_INITIAL_SL_MODE = str(ps.get("GB_INITIAL_SL_MODE", "FIXED20") or "FIXED20")
    FORCED_LOT_SIZE = int(ps.get("LOT_SIZE", 0) or 0)

    # Publish globals (strategies read from globals())
    globals()["PARAM_SET_NAME"] = PARAM_SET_NAME
    globals()["IMB_MIN"] = IMB_MIN
    globals()["MAX_SPREAD"] = MAX_SPREAD
    globals()["GL_SL_ATR_MULT"] = GL_SL_ATR_MULT
    globals()["GL_MIN_SL_POINTS"] = GL_MIN_SL_POINTS
    globals()["GL_TRAIL_START"] = GL_TRAIL_START
    globals()["GB_T2_FLOOR_OFFSET"] = GB_T2_FLOOR_OFFSET
    globals()["GB_INITIAL_SL_MODE"] = GB_INITIAL_SL_MODE
    globals()["FORCED_LOT_SIZE"] = FORCED_LOT_SIZE

    # Override lot size + compute qty (deterministic; later instrument lookup may confirm lot_size)
    try:
        LOTS_ = int(globals().get("LOTS", 1) or 1)
    except:
        LOTS_ = 1
    if FORCED_LOT_SIZE > 0:
        globals()["LOT_SIZE"] = FORCED_LOT_SIZE
        globals()["QTY"] = int(FORCED_LOT_SIZE) * int(LOTS_)

    active = {
        "PARAM_SET_NAME": PARAM_SET_NAME,
        "IMB_MIN": IMB_MIN,
        "MAX_SPREAD": MAX_SPREAD,
        "GL_SL_ATR_MULT": GL_SL_ATR_MULT,
        "GL_MIN_SL_POINTS": GL_MIN_SL_POINTS,
        "GL_TRAIL_START": GL_TRAIL_START,
        "GB_T2_FLOOR_OFFSET": GB_T2_FLOOR_OFFSET,
        "GB_INITIAL_SL_MODE": GB_INITIAL_SL_MODE,
        "FORCED_LOT_SIZE": FORCED_LOT_SIZE,
        "LOTS": LOTS_,
        "QTY": int(globals().get("QTY", 0) or 0)}
    globals()["ACTIVE_PARAM_SET"] = active
    globals()["_PARAM_SET_LOCKED"] = True

    # Clear startup log (expected by sanity checks)
    print(
        f"[PARAM_SET] {PARAM_SET_NAME} | "
        f"imb={IMB_MIN} spread={MAX_SPREAD} sl_mult={GL_SL_ATR_MULT} min_sl={GL_MIN_SL_POINTS} "
        f"trail={GL_TRAIL_START} lot={FORCED_LOT_SIZE} GB_SL={GB_INITIAL_SL_MODE}"
    )
    return dict(active)

globals()["apply_param_set"] = apply_param_set

# Apply once (startup)
apply_param_set(UNDERLYING, EXPIRY_TYPE)

globals()["market_open_time"] = market_open_time
globals()["market_close_time"] = market_close_time
globals()["SQUARE_OFF_TIME"] = SQUARE_OFF_TIME
globals()["SLIPPAGE_PER_SIDE"] = SLIPPAGE_PER_SIDE
globals()["BROKERAGE_PER_ORDER"] = BROKERAGE_PER_ORDER
globals()["ALLOW_CONCURRENT_LAYERS"] = ALLOW_CONCURRENT_LAYERS
globals()["PAPER_TRADING"] = PAPER_TRADING
globals()["DEMO_MODE"] = DEMO_MODE
globals()["KEEP_NOTEBOOK_ALIVE"] = KEEP_NOTEBOOK_ALIVE
globals()["TICK_LOG_PATH"] = TICK_LOG_PATH
globals()["TRADES_CSV"] = TRADES_CSV

# ----------------------------
# PREMIUM/SPREAD BANDS
# ----------------------------
def _prem_band(underlying: str, expiry_type: str):
    if underlying == "NIFTY" and expiry_type == "WEEKLY":
        return (90, 160)
    if underlying == "NIFTY" and expiry_type == "MONTHLY":
        return (80, 170)
    if underlying == "BANKNIFTY" and expiry_type == "WEEKLY":
        return (180, 320)
    if underlying == "BANKNIFTY" and expiry_type == "MONTHLY":
        return (160, 330)
    return (80, 170)

PREM_MIN, PREM_MAX = _prem_band(UNDERLYING, EXPIRY_TYPE)
# MAX_SPREAD comes exclusively from apply_param_set()

GL_IMB_MIN = float(os.getenv('GL_IMB_MIN', '1.35'))
log(f"📌 PremiumBand={PREM_MIN}-{PREM_MAX} | MaxSpread={MAX_SPREAD:.2f}")

globals()["PREM_MIN"] = PREM_MIN
globals()["PREM_MAX"] = PREM_MAX
# globals()["MAX_SPREAD"] is locked by apply_param_set()

# ----------------------------
# INSTRUMENT RESOLUTION + AUTO STRIKE SELECTION (HARDENED)
# ----------------------------
def resolve_spot_token(underlying: str):
    ins = kite_instruments("NSE")
    df = pd.DataFrame(ins)

    if underlying == "NIFTY":
        candidates = ["NIFTY 50", "NIFTY", "NIFTY_50"]
    else:
        candidates = ["NIFTY BANK", "BANKNIFTY", "NIFTYBANK"]

    for sym in candidates:
        x = df[df["tradingsymbol"] == sym]
        if not x.empty:
            r = x.iloc[0].to_dict()
            return int(r["instrument_token"]), f"NSE:{r['tradingsymbol']}"

    pat = "NIFTY" if underlying == "NIFTY" else "BANK"
    x = df[df["tradingsymbol"].astype(str).str.contains(pat, case=False, na=False)]
    if x.empty:
        safe_stop("Could not resolve SPOT token from NSE instruments.", fatal=True)
    r = x.iloc[0].to_dict()
    return int(r["instrument_token"]), f"NSE:{r['tradingsymbol']}"

SPOT_TOKEN, SPOT_SYMBOL = resolve_spot_token(UNDERLYING)
log(f"📍 SPOT: {SPOT_SYMBOL} | token={SPOT_TOKEN}")

globals()["SPOT_TOKEN"] = int(SPOT_TOKEN)
globals()["SPOT_SYMBOL"] = str(SPOT_SYMBOL)

def spot_ltp_now() -> float:
    """Get current spot LTP safely.

    - Uses wrapped kite_quote (with retries).
    - Falls back to ohlc.close when market is closed / last_price is missing.
    """
    q = kite_quote([SPOT_SYMBOL])
    row = (q or {}).get(SPOT_SYMBOL) or {}
    lp = row.get("last_price")
    if lp is None:
        lp = (row.get("ohlc") or {}).get("close")
    if lp is None:
        raise RuntimeError(f"Spot quote missing last_price and ohlc.close for {SPOT_SYMBOL}. keys={list(row.keys())}")
    return float(lp)

log("📥 Loading NFO instruments (options)…")
nfo_df = pd.DataFrame(kite_instruments("NFO"))

opt_df = nfo_df[(nfo_df["segment"] == "NFO-OPT") &
                (nfo_df["name"].astype(str).str.upper() == UNDERLYING)].copy()
if opt_df.empty:
    safe_stop(f"No option instruments found for {UNDERLYING} in NFO.", fatal=True)

opt_df["expiry_date"] = pd.to_datetime(opt_df["expiry"]).dt.date

def pick_expiry(expiry_type: str) -> datetime.date:
    today = datetime.datetime.now(IST).date()
    expiries = sorted(opt_df["expiry_date"].dropna().unique().tolist())
    if not expiries:
        safe_stop("No expiries found in option instruments.", fatal=True)

    if expiry_type == "WEEKLY":
        for e in expiries:
            if e >= today:
                return e
        return expiries[-1]
    else:
        cur_month, cur_year = today.month, today.year
        month_exps = [e for e in expiries if e.year == cur_year and e.month == cur_month and e >= today]
        if month_exps:
            return max(month_exps)
        for e in expiries:
            if e >= today:
                return e
        return expiries[-1]

EXPIRY_DATE = pick_expiry(EXPIRY_TYPE)
exp_df = opt_df[opt_df["expiry_date"] == EXPIRY_DATE].copy()
if exp_df.empty:
    safe_stop(f"No options for {UNDERLYING} expiry {EXPIRY_DATE}.", fatal=True)

log(f"📅 EXPIRY AUTO: {EXPIRY_DATE} ({EXPIRY_TYPE})")
globals()["EXPIRY_DATE"] = EXPIRY_DATE

def _depth_best_prices_from_quote(qd: dict):
    d = qd.get("depth") or {}
    buy = d.get("buy") or []
    sell = d.get("sell") or []
    bid = ask = None
    try:
        if buy and isinstance(buy[0], dict):
            bid = float(buy[0].get("price") or 0) or None
    except:
        if FAIL_FAST: raise
        pass
    try:
        if sell and isinstance(sell[0], dict):
            ask = float(sell[0].get("price") or 0) or None
    except:
        if FAIL_FAST: raise
        pass
    spr = None
    if bid is not None and ask is not None and ask > bid:
        spr = ask - bid
    return bid, ask, spr

def pick_option_by_premium(opt_type: str, spot_ltp: float) -> dict:
    strikes = sorted(exp_df["strike"].astype(float).unique().tolist())
    if not strikes:
        safe_stop("No strikes found for selected expiry.", fatal=True)

    idx = int(np.argmin([abs(s - spot_ltp) for s in strikes]))
    window = strikes[max(0, idx-12): min(len(strikes), idx+13)]

    candidates = exp_df[(exp_df["instrument_type"] == opt_type) &
                        (exp_df["strike"].astype(float).isin(window))].copy()
    if candidates.empty:
        safe_stop(f"No candidates for {opt_type} near spot {spot_ltp}.", fatal=True)

    syms = ["NFO:" + s for s in candidates["tradingsymbol"].tolist()]
    q = kite_quote(syms)

    rows = []
    for _, r in candidates.iterrows():
        tsym = "NFO:" + r["tradingsymbol"]
        qq = q.get(tsym, {})
        ltp = qq.get("last_price")
        if ltp is None:
            continue
        bid, ask, spr = _depth_best_prices_from_quote(qq)
        lot_size = int(r.get("lot_size") or 0)
        rows.append((tsym, int(r["instrument_token"]), float(r["strike"]), float(ltp), spr, bid, ask, lot_size))

    if not rows:
        safe_stop("No option quotes received (Kite quote returned empty).", fatal=True)

    target = (PREM_MIN + PREM_MAX)/2.0
    band = [x for x in rows if (PREM_MIN <= x[3] <= PREM_MAX) and (x[4] is None or x[4] <= MAX_SPREAD)]
    use = band if band else rows
    use.sort(key=lambda x: abs(x[3] - target))

    tsym, token, strike, ltp, spr, bid, ask, lot_size = use[0]
    return {"symbol": tsym, "token": token, "strike": strike, "ltp": ltp,
            "spread": spr, "bid": bid, "ask": ask, "lot_size": lot_size}

# ============================================================
# ✅ ATM-ONLY STRIKE SELECTION (CE + PE at SAME ATM strike)
#    NOTE: Spot is used ONLY to pick ATM strike. All indicators/logic are on option prices.
# ============================================================
def pick_atm_strike(spot_ltp: float) -> float:
    strikes = sorted(exp_df["strike"].astype(float).unique().tolist())
    if not strikes:
        safe_stop("No strikes found for selected expiry.", fatal=True)
    idx = int(np.argmin([abs(s - float(spot_ltp)) for s in strikes]))
    return float(strikes[idx])

def pick_option_atm(opt_type: str, atm_strike: float) -> dict:
    opt_type = str(opt_type).upper()
    atm_strike = float(atm_strike)
    candidates = exp_df[(exp_df["instrument_type"].astype(str).str.upper() == opt_type) &
                        (exp_df["strike"].astype(float) == atm_strike)].copy()
    if candidates.empty:
        # fallback: nearest strike for that leg
        strikes = sorted(exp_df["strike"].astype(float).unique().tolist())
        idx = int(np.argmin([abs(s - atm_strike) for s in strikes]))
        use_strike = float(strikes[idx])
        candidates = exp_df[(exp_df["instrument_type"].astype(str).str.upper() == opt_type) &
                            (exp_df["strike"].astype(float) == use_strike)].copy()
        if candidates.empty:
            safe_stop(f"No ATM candidates for {opt_type} strike {atm_strike}.", fatal=True)

    # Pick first match (for a given expiry+strike+type there should generally be one)
    r = candidates.iloc[0].to_dict()
    tsym = "NFO:" + str(r["tradingsymbol"])
    q = kite_quote([tsym]).get(tsym, {}) or {}
    ltp = q.get("last_price")
    if ltp is None:
        safe_stop(f"No quote for {tsym}", fatal=True)
    bid, ask, spr = _depth_best_prices_from_quote(q)

    lot_size = int(r.get("lot_size") or 0)
    return {
        "symbol": tsym,
        "token": int(r["instrument_token"]),
        "strike": float(r["strike"]),
        "ltp": float(ltp),
        "spread": spr,
        "bid": bid,
        "ask": ask,
        "lot_size": lot_size}

def _wait_until_time(target_t: datetime.time):
    while True:
        if block_stop.is_set():
            safe_stop("Stopped while waiting for strike selection time.", fatal=True)
        now = datetime.datetime.now(IST)
        if now.time() >= target_t:
            return
        time.sleep(1)

def _select_spot_for_strikes() -> float:
    nowt = datetime.datetime.now(IST).time()
    if nowt < STRIKE_SELECT_START:
        log(f"⏳ Waiting for strike selection window {STRIKE_SELECT_START.strftime('%H:%M:%S')} IST …")
        _wait_until_time(STRIKE_SELECT_START)

    nowt = datetime.datetime.now(IST).time()
    if nowt > STRIKE_SELECT_END:
        px = spot_ltp_now()
        log(f"⚠ Started after strike window end. Using current SPOT for strikes: {px:.2f}")
        return float(px)

    deadline = datetime.datetime.combine(datetime.datetime.now(IST).date(), STRIKE_SELECT_END, tzinfo=IST)
    last_err = None
    while datetime.datetime.now(IST) <= deadline:
        try:
            px = spot_ltp_now()
            if px and float(px) > 0:
                return float(px)
        except Exception as e:
            last_err = str(e)
        time.sleep(STRIKE_SELECT_POLL_SEC)

    px = spot_ltp_now()
    log(f"⚠ Spot read retries ended. Using fallback SPOT={px:.2f}. LastErr={last_err}")
    return float(px)

spot_for_strikes = _select_spot_for_strikes()
ATM_STRIKE = pick_atm_strike(spot_for_strikes)
log(f"🎯 ATM STRIKE SELECTED: {ATM_STRIKE:.0f}")
# ----------------------------
# Build strike candidate shortlist (ATM, ATM±1, ATM±2, ...)
# ----------------------------
def _get_strike_step(underlying: str) -> int:
    u = str(underlying).upper()
    if "BANK" in u:
        return int(STRIKE_STEP_BANKNIFTY)
    return int(STRIKE_STEP_NIFTY)

STRIKE_STEP = _get_strike_step(UNDERLYING)

def _build_candidate_strikes_itm(atm: float, opt_type: str, n_itm: int) -> List[float]:
    """Return [ATM, 1 ITM, 2 ITM, ...] strikes for opt_type.
    For CE: ITM is lower strikes. For PE: ITM is higher strikes.
    """
    opt_type = str(opt_type).upper()
    offsets = [0]
    for i in range(1, int(n_itm) + 1):
        offsets.append(-i if opt_type == "CE" else +i)
    return [float(atm) + float(o) * float(STRIKE_STEP) for o in offsets]

# ============================================================
# ✅ FIXED CE/PE FOR THE DAY (NO HOT-SWAP / NO CANDIDATE ROTATION)
#    We select CE+PE at the same ATM strike once (around 09:09–09:15 window)
#    and keep them unchanged for the whole session to avoid spam + unwanted swaps.
# ============================================================
CE_OPT = pick_option_atm("CE", ATM_STRIKE)
PE_OPT = pick_option_atm("PE", ATM_STRIKE)

CE_TOKEN = int(CE_OPT["token"])
PE_TOKEN = int(PE_OPT["token"])
CE_TSYM  = CE_OPT["symbol"]
PE_TSYM  = PE_OPT["symbol"]
CE_STRIKE= CE_OPT["strike"]
PE_STRIKE= PE_OPT["strike"]

# ----------------------------
# CORE STATE OBJECTS (MANDATORY)
# ----------------------------
# Sanity tests + strategy helpers expect `ce_state` and `pe_state` to exist
# regardless of market hours. Create them NOW (before any seeding / wiring).
global_state = globals().get("global_state")
if global_state is None or not isinstance(global_state, dict):
    global_state = {"CE": {}, "PE": {}}
else:
    global_state.setdefault("CE", {})
    global_state.setdefault("PE", {})

ce_state = global_state["CE"]
pe_state = global_state["PE"]

# Persist canonical references
globals()["global_state"] = global_state

globals()["ce_state"] = ce_state
globals()["pe_state"] = pe_state

# Minimal 15m keys required by GB/GL — must exist even if no live ticks arrive.
_REQUIRED_15M_KEYS = {
    "rsi3_15m": None,
    "rsi3_15m_ts": None,
    "vwap_15m": None,
    "imb15_cur": None,
    "imb15_ts": None,
    "imb_avg20_15m": None,
}
for _k, _v in _REQUIRED_15M_KEYS.items():
    ce_state.setdefault(_k, _v)
    pe_state.setdefault(_k, _v)

print("[STATE] Core CE/PE state objects initialized (mandatory keys present).")
# ----------------------------
# VWAP SEEDING (NEW)
# ----------------------------
_ce_pv_seed = _ce_v_seed = 0.0
_pe_pv_seed = _pe_v_seed = 0.0

# ----------------------------
# OPTION VWAP/ATR ACCUMULATORS (MODULE-SCOPE INIT)
# ----------------------------
# These must exist before the first option tick to avoid NameError in `global` blocks.
_opt_vwap_date = None

_ce_pv = 0.0
_ce_v  = 0.0
_pe_pv = 0.0
_pe_v  = 0.0

# 15m accumulators (if used anywhere)
_ce15_pv = 0.0
_ce15_v  = 0.0
_pe15_pv = 0.0
_pe15_v  = 0.0

# ATR7 rolling queues (points-based)
_ce_tr_q = deque(maxlen=7)
_pe_tr_q = deque(maxlen=7)

def _prev_trading_day(d: datetime.date) -> datetime.date:
    # Simple prev weekday fallback (Kite holiday calendar not available here)
    dd = d - datetime.timedelta(days=1)
    while dd.weekday() >= 5:  # Sat/Sun
        dd = dd - datetime.timedelta(days=1)
    return dd

def _seed_vwap_from_historical(token: int, day: datetime.date) -> tuple[float,float]:
    """Return (pv, v) computed from 1-min candles typical_price*volume.
    If unavailable, returns (0,0).
    """
    try:
        from_dt = datetime.datetime.combine(day, datetime.time(9, 15))
        to_dt   = datetime.datetime.combine(day, datetime.time(15, 30))
        hist = kite.historical_data(int(token), from_dt, to_dt, interval="minute")
        if not hist:
            return 0.0, 0.0
        pv = 0.0
        vv = 0.0
        for c in hist:
            try:
                v = float(c.get("volume") or 0.0)
                if v <= 0:
                    continue
                tp = (float(c["high"]) + float(c["low"]) + float(c["close"])) / 3.0
                pv += tp * v
                vv += v
            except Exception:
                continue
        return float(pv), float(vv)
    except Exception:
        return 0.0, 0.0


class RSIWilder:
    """Wilder RSI on close prices. Returns None until enough data."""
    def __init__(self, period: int):
        self.p = int(period)
        self.prev = None
        self.avg_gain = None
        self.avg_loss = None
        self._seed_g = []
        self._seed_l = []

    def update(self, close: float):
        c = float(close)
        if self.prev is None:
            self.prev = c
            return None
        chg = c - self.prev
        self.prev = c
        gain = chg if chg > 0 else 0.0
        loss = -chg if chg < 0 else 0.0

        if self.avg_gain is None or self.avg_loss is None:
            self._seed_g.append(gain)
            self._seed_l.append(loss)
            if len(self._seed_g) < self.p:
                return None
            self.avg_gain = sum(self._seed_g) / float(self.p)
            self.avg_loss = sum(self._seed_l) / float(self.p)
        else:
            self.avg_gain = (self.avg_gain * (self.p - 1) + gain) / float(self.p)
            self.avg_loss = (self.avg_loss * (self.p - 1) + loss) / float(self.p)

        if self.avg_loss == 0:
            return 100.0
        rs = self.avg_gain / self.avg_loss
        return 100.0 - (100.0 / (1.0 + rs))


def _seed_rsi15_from_historical(token: int, end_day: datetime.date, lookback_days: int = 5) -> float | None:
    """Seed RSI(3) from 15-minute closes using Kite historical candles.

    Design goals:
      - Always produce a usable RSI number at startup (caller applies neutral fallback if needed).
      - Be rate-limit friendly: 1 request per leg in the common case (2 max).
      - Work pre-open / after-hours without depending on live ticks.

    Rules:
      1) Clamp future `end_day` to today's IST date.
      2) If current IST time is before market_open_time, seed from the previous trading day.
      3) Fetch ONE full trading day of 15m candles and compute RSI(3).
         If the day returns empty (rare / intermittent), retry once with previous trading day.

    Returns:
      Latest RSI(3) value, or None if insufficient data.
    """

    def _today_ist_date() -> datetime.date:
        try:
            return datetime.datetime.now(IST).date()
        except Exception:
            return datetime.date.today()

    def _is_td(d: datetime.date) -> bool:
        try:
            return _is_trading_day(d)
        except Exception:
            return d.weekday() < 5

    def _prev_trading_day_best_effort(d: datetime.date) -> datetime.date:
        dd = d - datetime.timedelta(days=1)
        for _ in range(10):  # skip weekends/holidays best-effort
            if _is_td(dd):
                return dd
            dd = dd - datetime.timedelta(days=1)
        # weekday fallback
        while dd.weekday() >= 5:
            dd = dd - datetime.timedelta(days=1)
        return dd

    def _fetch_15m_closes(day: datetime.date) -> list[float]:
        from_dt = datetime.datetime.combine(day, datetime.time(9, 15))
        to_dt   = datetime.datetime.combine(day, datetime.time(15, 30))
        hist15 = kite.historical_data(int(token), from_dt, to_dt, interval="15minute") or []
        closes: list[float] = []
        for c in hist15:
            try:
                closes.append(float(c["close"]))
            except Exception:
                continue
        return closes

    try:
        today_ist = _today_ist_date()
        ed = end_day
        if isinstance(ed, datetime.date) and ed > today_ist:
            ed = today_ist

        # Pre-open: seed from previous trading day (so RSI is meaningful at 09:15)
        try:
            now_ist = datetime.datetime.now(IST)
        except Exception:
            now_ist = datetime.datetime.now()

        try:
            mo = market_open_time  # expected to exist
        except Exception:
            mo = datetime.time(9, 15)

        seed_day = ed
        if isinstance(now_ist, datetime.datetime) and now_ist.time() < mo:
            seed_day = _prev_trading_day_best_effort(ed)
            print(f"[RSI15 SEED] pre-open: using {seed_day} (end_day={ed})")

        closes = _fetch_15m_closes(seed_day)
        print(f"[RSI15 SEED] {seed_day} candles={len(closes)}")

        # Intermittent empty response / rate limit safety: retry once
        if len(closes) < 10:
            retry_day = _prev_trading_day_best_effort(seed_day)
            if retry_day != seed_day:
                print(f"[RSI15 SEED] insufficient candles for {seed_day} (n={len(closes)}). Retrying {retry_day}...")
                closes = _fetch_15m_closes(retry_day)
                print(f"[RSI15 SEED] {retry_day} candles={len(closes)}")

        if len(closes) < 10:
            return None

        rsi = RSIWilder(3)
        last = None
        for x in closes:
            last = rsi.update(float(x))
        return float(last) if last is not None else None

    except Exception as e:
        print(f"[SEED RSI15] error: {e}")
        if FAIL_FAST:
            raise
        return None


# ----------------------------
# IMB (DEPTH) 15m AGG + AVG(20) SEEDING (NEW)
# ----------------------------
# We cannot get historical market-depth from Kite historical candles.
# So we seed Avg IMB from the previous trading day's saved IMB seed file (if present),
# then keep updating live from depth ticks (buy/sell quantities).
#
# Output fields on global_state[CE/PE]:
#   imb15_cur        = running avg IMB for current 15m bucket
#   imb_avg20_15m    = SMA of last 20 completed 15m IMB buckets (seeded if available)
#   imb15_ts         = last update timestamp
#
IMB_SEED_FILE = os.path.join(BASE_FOLDER, "imb_seed.json")

# Rolling bars (completed 15m buckets)
_ce_imb15_bars = deque(maxlen=200)
_pe_imb15_bars = deque(maxlen=200)

# Current bucket accumulators
_ce_imb15_bucket = None
_ce_imb15_sum = 0.0
_ce_imb15_n   = 0

_pe_imb15_bucket = None
_pe_imb15_sum = 0.0
_pe_imb15_n   = 0

def _reports_root_from_base(base_folder: str) -> str:
    try:
        bf = base_folder.rstrip("/").rstrip(chr(92))
        # .../NANDINI19_REPORTS/YYYY-MM-DD
        parent = os.path.dirname(bf)  # .../NANDINI19_REPORTS
        return parent
    except Exception:
        return base_folder

def _load_imb_seed_from_prev_day():
    global _ce_imb15_bars, _pe_imb15_bars
    try:
        root = _reports_root_from_base(BASE_FOLDER)
        prev_d = _prev_trading_day(TODAY_DATE)
        prev_folder = os.path.join(root, prev_d.isoformat())
        prev_file = os.path.join(prev_folder, "imb_seed.json")
        if not os.path.exists(prev_file):
            return
        with open(prev_file, "r") as f:
            j = json.load(f) or {}
        ce_list = j.get("CE") or []
        pe_list = j.get("PE") or []
        for x in ce_list[-200:]:
            try: _ce_imb15_bars.append(float(x))
            except: pass
        for x in pe_list[-200:]:
            try: _pe_imb15_bars.append(float(x))
            except: pass
    except Exception:
        if FAIL_FAST: raise

def _save_imb_seed():
    try:
        payload = {
            "ts": now_ist().isoformat(),
            "CE": list(_ce_imb15_bars),
            "PE": list(_pe_imb15_bars),
        }
        with open(IMB_SEED_FILE, "w") as f:
            json.dump(payload, f, indent=2)
    except Exception:
        if FAIL_FAST: raise

_load_imb_seed_from_prev_day()

def _bucket_15m(ts: datetime.datetime) -> datetime.datetime:
    ts = _as_ist_dt(ts) or ts
    return ts.replace(minute=(ts.minute // 15) * 15, second=0, microsecond=0)

def _avg_last_n(dq: deque, n: int) -> float | None:
    try:
        if not dq:
            return None
        xs = list(dq)[-int(n):]
        if not xs:
            return None
        return float(sum(xs) / len(xs))
    except Exception:
        return None


# Seed IMB 15m fields immediately from loaded seed bars (if present).
# This ensures GB/GL can evaluate even when market is closed / no depth ticks arrive.
# If no prior seed exists (first run / new machine), initialize to neutral numeric defaults.
try:
    _gs = globals().get("global_state")
    if isinstance(_gs, dict):
        _ce = _gs.setdefault("CE", {})
        _pe = _gs.setdefault("PE", {})

        _ce_avg20 = _avg_last_n(_ce_imb15_bars, 20)
        _pe_avg20 = _avg_last_n(_pe_imb15_bars, 20)

        # Neutral default: 1.0 (balanced) — avoids None fields without changing GB/GL logic.
        if _ce.get("imb_avg20_15m") is None:
            _ce["imb_avg20_15m"] = float(_ce_avg20) if _ce_avg20 is not None else 1.0
        if _pe.get("imb_avg20_15m") is None:
            _pe["imb_avg20_15m"] = float(_pe_avg20) if _pe_avg20 is not None else 1.0

        # Current bucket value (fallback to last completed bar; else neutral 1.0)
        if _ce.get("imb15_cur") is None:
            _ce["imb15_cur"] = float(list(_ce_imb15_bars)[-1]) if len(_ce_imb15_bars) > 0 else 1.0
        if _pe.get("imb15_cur") is None:
            _pe["imb15_cur"] = float(list(_pe_imb15_bars)[-1]) if len(_pe_imb15_bars) > 0 else 1.0

        # Timestamp for traceability
        if _ce.get("imb15_ts") is None:
            _ce["imb15_ts"] = now_ist()
        if _pe.get("imb15_ts") is None:
            _pe["imb15_ts"] = now_ist()

        globals()["ce_state"] = _ce
        globals()["pe_state"] = _pe
except Exception as e:
    print(f"[IMB SEED] error: {e}")
    if FAIL_FAST:
        raise

def _update_imb15(kind: str, ts: datetime.datetime, imb_ratio: float):
    """Update 15m bucket running average and SMA(20) for IMB."""
    global _ce_imb15_bucket, _ce_imb15_sum, _ce_imb15_n
    global _pe_imb15_bucket, _pe_imb15_sum, _pe_imb15_n
    kind = str(kind).upper()
    if kind not in ("CE", "PE"):
        return
    try:
        r = float(imb_ratio)
    except Exception:
        return
    b = _bucket_15m(ts)

    if kind == "CE":
        # bucket rollover
        if _ce_imb15_bucket is None:
            _ce_imb15_bucket = b
        if b != _ce_imb15_bucket:
            if _ce_imb15_n > 0:
                bar = float(_ce_imb15_sum / _ce_imb15_n)
                _ce_imb15_bars.append(bar)
                _save_imb_seed()
            _ce_imb15_bucket = b
            _ce_imb15_sum = 0.0
            _ce_imb15_n = 0

        _ce_imb15_sum += r
        _ce_imb15_n += 1
        cur = float(_ce_imb15_sum / _ce_imb15_n) if _ce_imb15_n > 0 else None
        global_state["CE"]["imb15_cur"] = cur
        global_state["CE"]["imb15_ts"] = ts
        global_state["CE"]["imb_avg20_15m"] = _avg_last_n(_ce_imb15_bars, 20)
    else:
        if _pe_imb15_bucket is None:
            _pe_imb15_bucket = b
        if b != _pe_imb15_bucket:
            if _pe_imb15_n > 0:
                bar = float(_pe_imb15_sum / _pe_imb15_n)
                _pe_imb15_bars.append(bar)
                _save_imb_seed()
            _pe_imb15_bucket = b
            _pe_imb15_sum = 0.0
            _pe_imb15_n = 0

        _pe_imb15_sum += r
        _pe_imb15_n += 1
        cur = float(_pe_imb15_sum / _pe_imb15_n) if _pe_imb15_n > 0 else None
        global_state["PE"]["imb15_cur"] = cur
        global_state["PE"]["imb15_ts"] = ts
        global_state["PE"]["imb_avg20_15m"] = _avg_last_n(_pe_imb15_bars, 20)

def _get_leg_imb15(leg: str) -> float | None:
    try:
        return float(global_state[str(leg).upper()].get("imb15_cur"))
    except Exception:
        return None

def _get_leg_imb15_avg20(leg: str) -> float | None:
    try:
        return float(global_state[str(leg).upper()].get("imb_avg20_15m"))
    except Exception:
        return None





def _seed_vwap15_from_historical(token: int, day: datetime.date, up_to: datetime.datetime | None = None) -> tuple[float,float]:
    """Seed (pv,v) for VWAP using historical candles.

    Primary path uses Kite's native 15-minute candles (avoids pandas resample alignment/version issues).
    Fallback uses minute candles and computes VWAP directly at 1-min granularity.

    Robustness:
      - Clamp future `day` to today's IST date (prevents empty results after market close when
        trading-date logic rolls forward).

    Returns:
        (pv, v) where pv = sum(typical_price * volume) and v = sum(volume)
    """
    try:
        try:
            today_ist = datetime.datetime.now(IST).date()
        except Exception:
            today_ist = datetime.date.today()

        d = day
        if isinstance(d, datetime.date) and d > today_ist:
            d = today_ist

        from_dt = datetime.datetime.combine(d, datetime.time(9, 15))
        to_dt   = datetime.datetime.combine(d, datetime.time(15, 30))
        if up_to is not None and isinstance(up_to, datetime.datetime):
            to_dt = min(to_dt, up_to.replace(tzinfo=None))

        pv = 0.0
        vv = 0.0

        # 1) Preferred: native 15m candles
        hist15 = kite.historical_data(int(token), from_dt, to_dt, interval="15minute") or []
        if hist15:
            for c in hist15:
                try:
                    vol = float(c.get("volume") or 0.0)
                    if vol <= 0:
                        continue
                    tp = (float(c["high"]) + float(c["low"]) + float(c["close"])) / 3.0
                    pv += tp * vol
                    vv += vol
                except Exception:
                    continue
            return float(pv), float(vv)

        # 2) Fallback: minute candles (no resampling)
        hist1 = kite.historical_data(int(token), from_dt, to_dt, interval="minute") or []
        if not hist1:
            return 0.0, 0.0

        for c in hist1:
            try:
                vol = float(c.get("volume") or 0.0)
                if vol <= 0:
                    continue
                tp = (float(c["high"]) + float(c["low"]) + float(c["close"])) / 3.0
                pv += tp * vol
                vv += vol
            except Exception:
                continue
        return float(pv), float(vv)

    except Exception as e:
        print(f"[SEED VWAP15] error: {e}")
        if FAIL_FAST:
            raise
        return 0.0, 0.0



try:
    # ✅ Seed RSI(3) on 15m from historical candles (ready immediately; no dependency on live ticks)
    _ce_rsi_seed = _seed_rsi15_from_historical(CE_TOKEN, TODAY_DATE, lookback_days=5)
    _pe_rsi_seed = _seed_rsi15_from_historical(PE_TOKEN, TODAY_DATE, lookback_days=5)

    # Neutral RSI fallback (prevents None fields; does not change GB/GL logic)
    if _ce_rsi_seed is None:
        print("[RSI15 SEED] CE insufficient data -> using neutral 50.0")
        _ce_rsi_seed = 50.0
    if _pe_rsi_seed is None:
        print("[RSI15 SEED] PE insufficient data -> using neutral 50.0")
        _pe_rsi_seed = 50.0

    global_state["CE"]["rsi3_15m"] = float(_ce_rsi_seed)
    global_state["PE"]["rsi3_15m"] = float(_pe_rsi_seed)
    global_state["CE"]["rsi3_15m_ts"] = datetime.datetime.combine(TODAY_DATE, market_open_time, tzinfo=IST)
    global_state["PE"]["rsi3_15m_ts"] = datetime.datetime.combine(TODAY_DATE, market_open_time, tzinfo=IST)
    print(f"[RSI15 SEED] CE={global_state['CE']['rsi3_15m']:.2f} PE={global_state['PE']['rsi3_15m']:.2f}")

    # ✅ Seed VWAP from historical candles up to now (useful if bot starts late / after-hours)
    _now = datetime.datetime.now(IST)
    _ce15_pv, _ce15_v = _seed_vwap15_from_historical(CE_TOKEN, TODAY_DATE, up_to=_now)
    _pe15_pv, _pe15_v = _seed_vwap15_from_historical(PE_TOKEN, TODAY_DATE, up_to=_now)

    def _safe_ltp(tsym: str, fallback: float | None = None) -> float:
        try:
            if fallback is not None and float(fallback) > 0:
                return float(fallback)
        except Exception:
            pass
        try:
            q = kite_quote([tsym]).get(tsym, {}) or {}
            lp = q.get("last_price")
            return float(lp) if lp is not None else 0.0
        except Exception:
            return 0.0

    if float(_ce15_v or 0) > 0:
        global_state["CE"]["vwap_15m"] = float(_ce15_pv / _ce15_v)
    else:
        global_state["CE"]["vwap_15m"] = _safe_ltp(CE_TSYM, CE_OPT.get("ltp"))
        if global_state["CE"]["vwap_15m"] <= 0:
            global_state["CE"]["vwap_15m"] = 0.0
        print(f"[VWAP15 SEED] CE volume=0 -> using LTP fallback {global_state['CE']['vwap_15m']:.2f}")

    if float(_pe15_v or 0) > 0:
        global_state["PE"]["vwap_15m"] = float(_pe15_pv / _pe15_v)
    else:
        global_state["PE"]["vwap_15m"] = _safe_ltp(PE_TSYM, PE_OPT.get("ltp"))
        if global_state["PE"]["vwap_15m"] <= 0:
            global_state["PE"]["vwap_15m"] = 0.0
        print(f"[VWAP15 SEED] PE volume=0 -> using LTP fallback {global_state['PE']['vwap_15m']:.2f}")

    print(f"[VWAP15 SEED] CE_v={float(_ce15_v or 0):.0f} PE_v={float(_pe15_v or 0):.0f}")

except Exception as _e:
    print("[SEED] failed:", _e)
    if FAIL_FAST:
        raise
    # Guarantee numeric fallbacks (no None) even if seeding fails.
    try:
        global_state.setdefault("CE", {})
        global_state.setdefault("PE", {})
        if global_state["CE"].get("rsi3_15m") is None:
            global_state["CE"]["rsi3_15m"] = 50.0
        if global_state["PE"].get("rsi3_15m") is None:
            global_state["PE"]["rsi3_15m"] = 50.0
        if global_state["CE"].get("vwap_15m") is None:
            global_state["CE"]["vwap_15m"] = float(CE_OPT.get("ltp") or 0.0)
        if global_state["PE"].get("vwap_15m") is None:
            global_state["PE"]["vwap_15m"] = float(PE_OPT.get("ltp") or 0.0)
    except Exception:
        pass



# Keep these globals for compatibility with the rest of the notebook.
# (We keep candidate arrays but only containing the active legs.)
CE_CANDIDATES = [CE_OPT]
PE_CANDIDATES = [PE_OPT]
globals()["CE_CANDIDATES"] = CE_CANDIDATES
globals()["PE_CANDIDATES"] = PE_CANDIDATES
globals()["CANDIDATE_TOKENS"] = [int(CE_TOKEN), int(PE_TOKEN)]

LOT_SIZE = int(CE_OPT.get("lot_size") or 0) or int(PE_OPT.get("lot_size") or 0)
if not LOT_SIZE:
    LOT_SIZE = int(float(input("LOT SIZE not found. Enter lot size manually: ").strip()))

# ✅ override lot size based on selected param set (required for BankNifty Monthly)
try:
    _forced = int(globals().get("FORCED_LOT_SIZE") or 0)
    if _forced > 0:
        LOT_SIZE = _forced
except:
    if FAIL_FAST: raise
    pass

QTY = LOT_SIZE * int(LOTS)

globals()["CE_TOKEN"] = CE_TOKEN
globals()["PE_TOKEN"] = PE_TOKEN
globals()["CE_TSYM"] = CE_TSYM
globals()["PE_TSYM"] = PE_TSYM
globals()["CE_STRIKE"] = CE_STRIKE
globals()["PE_STRIKE"] = PE_STRIKE
globals()["LOT_SIZE"] = LOT_SIZE
globals()["QTY"] = QTY

log(
    f"✅ STRIKE SELECTED ({STRIKE_SELECT_START.strftime('%H:%M')}-{STRIKE_SELECT_END.strftime('%H:%M')}) | "
    f"{UNDERLYING} {EXPIRY_TYPE} EXP={EXPIRY_DATE}\n"
    f"SPOT={spot_for_strikes:.2f}\n"
    f"CE {CE_STRIKE} → {CE_TSYM}\n"
    f"PE {PE_STRIKE} → {PE_TSYM}\n"
    f"LOT={LOT_SIZE} LOTS={LOTS} QTY={QTY}"
)

# ------------------------------------------------------------
# GLOBAL STATE (ensure always present)
# ------------------------------------------------------------
globals()["ws_status"] = "INIT"
globals()["reconnects"] = 0
globals()["errors"] = 0
globals()["last_spot_tick_wall"] = 0.0
globals()["last_ce_tick_wall"] = 0.0
globals()["last_pe_tick_wall"] = 0.0

tick_q = globals().get("tick_q")
if tick_q is None or not isinstance(tick_q, queue.Queue):
    tick_q = queue.Queue(maxsize=20000)
globals()["tick_q"] = tick_q
globals()["tickq_drops"] = int(globals().get("tickq_drops", 0) or 0)

# ✅ UPDATED: adds bar_* + atr placeholders for ATR(7) & N19 High-Win
def _mk_opt_state():
    return {
        "ltp": None, "best_bid": None, "best_ask": None, "spread": None,
        "ws_ohlc_open": None, "rest_ohlc_open": None,
        "open": None, "open_ts": None, "open_source": None, "true_open_set": False,
        "open_locked": False, "open_src": None, "open_lock_time": None,
        "open_priority": -1,
        "vol_traded": 0.0,
        "vol_delta": 0.0,

        # dashboard indicator fields
        "ob_bull_low": None, "ob_bull_high": None, "ob_bull_mid": None, "ob_bull_ts": None,
        "ob_bear_low": None, "ob_bear_high": None, "ob_bear_mid": None, "ob_bear_ts": None,

        "imb_ratio": None, "imb_signal": None, "imb_ts": None,
        "imb15_cur": None, "imb15_ts": None, "imb_avg20_15m": None,
        "rsi3_15m_prev": None,
        "vwap_15m": None,
        "rsi3_15m": None, "rsi3_15m_ts": None,
        # (legacy 1m fields kept for compatibility)
        "vwap_1m": None,
        "rsi3_1m": None, "rsi3_ts": None,

        "c15_close": None,
        # (optional debug)

        "c3_close": None,
        # (optional debug)

        # legacy (kept)
        "atr14_1m": None,

        # ✅ REQUIRED for ATR(7) + N19 High-Win
        "bar_minute": None,
        "bar_open": None,
        "bar_high": None,
        "bar_low": None,
        "bar_close": None,
        "bar_ts": None,
        "atr": None,     # numeric ATR used by strategies
        "atr7": None,    # ATRState object will be created in Block-2
    }

def _ensure_keys(dst: dict, template: dict):
    for k, v in template.items():
        if k not in dst:
            dst[k] = v

global_state = globals().get("global_state")
if global_state is None or not isinstance(global_state, dict):
    global_state = {"CE": _mk_opt_state(), "PE": _mk_opt_state()}
else:
    global_state.setdefault("CE", {})
    global_state.setdefault("PE", {})
    tmpl = _mk_opt_state()
    _ensure_keys(global_state["CE"], tmpl)
    _ensure_keys(global_state["PE"], tmpl)

globals()["global_state"] = global_state

# Bind/refresh canonical leg aliases (expected by sanity tests + strategy helpers)
try:
    globals()["ce_state"] = global_state.setdefault("CE", {})
    globals()["pe_state"] = global_state.setdefault("PE", {})
except Exception:
    pass

spot_state = globals().get("spot_state")
if spot_state is None or not isinstance(spot_state, dict):
    spot_state = {
        "ltp": None, "ts": None,
        "atr14_1m": None,
        "atr_last": None,
        "st60_dir": None, "st60_value": None, "st60_ts": None, "st60_err": None,
        "ob_zones": [],
        "spot_5m_bars": deque(maxlen=600),
        "last_5m_bar": None,
        "last_5m_bar_time": None,
        "vwap": None,
        "vwap_ts": None,
        "_vwap_pv": 0.0,
        "_vwap_v": 0.0,
        "_vwap_session_date": None,
        "vwap_hist": deque(maxlen=5000)}
globals()["spot_state"] = spot_state

global_summary = globals().get("global_summary")
if global_summary is None or not isinstance(global_summary, dict):
    global_summary = {"GL": 0.0, "GB": 0.0, "N19": 0.0}
globals()["global_summary"] = global_summary

# ------------------------------------------------------------
# Candidate quote cache + hot-swap helpers (NEW)
# ------------------------------------------------------------
candidate_quote_cache = globals().get("candidate_quote_cache")
if candidate_quote_cache is None or not isinstance(candidate_quote_cache, dict):
    candidate_quote_cache = {}  # token -> {ltp,bid,ask,spread,tsym,strike,delta,ts_wall}
globals()["candidate_quote_cache"] = candidate_quote_cache

def _get_candidates_for_leg(leg: str) -> List[dict]:
    leg = str(leg).upper()
    return list(globals().get("CE_CANDIDATES", [])) if leg == "CE" else list(globals().get("PE_CANDIDATES", []))

def _token_to_symbol(tok: int) -> str:
    try:
        tok = int(tok)
    except:
        return ""
    try:
        for c in (globals().get("CE_CANDIDATES", []) or []) + (globals().get("PE_CANDIDATES", []) or []):
            if int(c.get("token", -1) or -1) == tok:
                return str(c.get("symbol") or "")
    except:
        if FAIL_FAST: raise
        pass
    try:
        q = candidate_quote_cache.get(tok, {}) or {}
        return str(q.get("tsym") or q.get("symbol") or "")
    except:
        return ""

def _rest_fill_candidate_quote(tok: int) -> dict:
    """REST fallback to populate candidate_quote_cache when WS cache is empty (safe, throttled by caller)."""
    sym = _token_to_symbol(tok)
    if not sym:
        return candidate_quote_cache.get(int(tok), {}) or {}
    try:
        qq = kite_quote([sym]).get(sym, {}) or {}
        ltp = qq.get("last_price")
        d = qq.get("depth") or {}
        bid = (d.get("buy") or [{}])[0].get("price")
        ask = (d.get("sell") or [{}])[0].get("price")
        spr = None
        try:
            if bid is not None and ask is not None:
                spr = float(ask) - float(bid)
        except:
            spr = None
        ent = candidate_quote_cache.get(int(tok), {}) or {}
        ent.update({
            "tsym": sym,
            "ltp": float(ltp) if ltp is not None else ent.get("ltp"),
            "bid": float(bid) if bid is not None else ent.get("bid"),
            "ask": float(ask) if ask is not None else ent.get("ask"),
            "spread": float(spr) if spr is not None else ent.get("spread"),
            "oi": int(qq.get("oi")) if qq.get("oi") is not None else ent.get("oi"),
            "ts_wall": time.time()})
        candidate_quote_cache[int(tok)] = ent
        globals()["candidate_quote_cache"] = candidate_quote_cache
        return ent
    except:
        return candidate_quote_cache.get(int(tok), {}) or {}

def _min_premium_for_today() -> float:
    # Currently applied as NIFTY weekly default; customize per instrument/expiry if needed
    return float(MIN_PREMIUM_NIFTY_WEEKLY)

def _min_abs_delta_for_today() -> float:
    return float(MIN_ABS_DELTA_DEFAULT)

def _oi_threshold_for_today() -> int:
    return int(OI_THRESHOLD_NIFTY_WEEKLY)

def _candidate_is_eligible(tok: int) -> Tuple[bool, str]:
    q = candidate_quote_cache.get(int(tok), {}) or {}
    ltp = q.get("ltp")
    spr = q.get("spread")
    oi  = q.get("oi")
    if ltp is None:
        # WS cache may be empty early; try REST fallback once
        try:
            q2 = _rest_fill_candidate_quote(int(tok)) or {}
            ltp = q2.get("ltp") if q2.get("ltp") is not None else ltp
            spr = q2.get("spread") if q2.get("spread") is not None else spr
            oi  = q2.get("oi") if q2.get("oi") is not None else oi
        except:
            if FAIL_FAST: raise
            pass
    if ltp is None:
        return (False, "no_ltp")
    try:
        ltp_f = float(ltp)
    except:
        return (False, "bad_ltp")
    if ltp_f < _min_premium_for_today():
        return (False, f"premium<{_min_premium_for_today():.0f}")

    # OI filter (optional; WS FULL ticks provide "oi" for F&O)
    if bool(globals().get("REQUIRE_OI_FOR_ENTRY", REQUIRE_OI_FOR_ENTRY)):
        if oi is None:
            return (False, "no_oi")
        try:
            if int(oi) < int(_oi_threshold_for_today()):
                return (False, f"oi<{int(_oi_threshold_for_today())}")
        except:
            return (False, "bad_oi")

    # Spread filters
    if spr is None:
        return (False, "no_spread")
    try:
        spr_f = float(spr)
    except:
        return (False, "bad_spread")
    try:
        # absolute caps (backup)
        if float(SPREAD_ABS_CAP) and spr_f > float(SPREAD_ABS_CAP):
            return (False, "spread_abs")
        # existing max spread band (if configured elsewhere)
        if float(globals().get("MAX_SPREAD", 0) or 0) and spr_f > float(globals().get("MAX_SPREAD", 0) or 0):
            return (False, "spread_abs")
        # percent cap (primary)
        if float(SPREAD_PCT_CAP) and (spr_f / max(ltp_f, 1e-9)) > float(SPREAD_PCT_CAP):
            return (False, "spread_pct")
    except:
        if FAIL_FAST: raise
        pass

    return (True, "ok")

def _any_open_position_on_leg(leg: str) -> bool:
    leg = str(leg).upper()
    try:
        if getattr(gl_strategy, "in_trade", False) and getattr(gl_strategy, "leg_in_trade", lambda x: False)(leg):
            return True
    except:
        if FAIL_FAST: raise
        pass
    try:
        gbs = globals().get("gb_strategy")
        if gbs and getattr(getattr(gbs, "trade", None), "in_trade", False) and str(getattr(getattr(gbs, "trade", None), "leg", "")) == leg:
            return True
    except:
        if FAIL_FAST: raise
        pass
    try:
        n19s = globals().get("n19_state")
        if n19s and getattr(n19s, "in_trade", False) and str(getattr(n19s, "leg", "")) == leg:
            return True
    except:
        if FAIL_FAST: raise
        pass
    return False

def _reset_leg_state_on_swap(leg: str):
    # Keep structure, but reset indicator/open fields so TRUE OPEN pipeline can populate correctly for new strike
    leg = str(leg).upper()
    if leg not in ("CE", "PE"):
        return
    st = global_state.get(leg, {}) or {}
    tmpl = _mk_opt_state()
    # preserve some long-lived fields if needed; but safest is to reset to template
    global_state[leg] = copy.deepcopy(tmpl)
    globals()["global_state"] = global_state

def select_and_activate_candidate(leg: str, context: str = "entry") -> Tuple[bool, str]:
    """Pick first eligible candidate for leg and set CE/PE globals accordingly (token, tsym, strike).
    Will NOT change mid-open-position for that leg.
    Returns (changed_or_ok, message).
    """
    leg = str(leg).upper()
    if not bool(globals().get("ENABLE_HOTSWAP", False)):
        return (True, "hotswap_disabled")
    if leg not in ("CE", "PE"):
        return (False, "bad_leg")
    if _any_open_position_on_leg(leg):
        return (True, "locked_in_position")
    candidates = _get_candidates_for_leg(leg)
    if not candidates:
        return (False, "no_candidates")
    cur_tok = int(globals().get("CE_TOKEN") if leg=="CE" else globals().get("PE_TOKEN"))
    # Try current first
    ok, why = _candidate_is_eligible(cur_tok)
    if ok:
        return (True, "current_ok")
    # rotate
    for c in candidates:
        tok = int(c.get("token"))
        ok, why2 = _candidate_is_eligible(tok)
        if ok:
            # swap
            if leg == "CE":
                globals()["CE_TOKEN"] = tok
                globals()["CE_TSYM"]  = c.get("symbol")
                globals()["CE_STRIKE"]= c.get("strike")
            else:
                globals()["PE_TOKEN"] = tok
                globals()["PE_TSYM"]  = c.get("symbol")
                globals()["PE_STRIKE"]= c.get("strike")
            _reset_leg_state_on_swap(leg)
            _log(f"🔁 HOT-SWAP {leg} -> {c.get('symbol')} (token={tok}) reason={why}")
            tg_throttled(f"hotswap_{leg}", f"🔁 HOT-SWAP {leg} -> {str(c.get('symbol')).split(':')[-1]} | {why}", 10)
            diag("hotswap", leg=leg, to=c.get("symbol"), reason=why)
            return (True, f"swapped:{why}")
    _log(f"⛔ ALL CANDIDATES FAIL {leg} | {why}")
    tg_throttled(f"hotswap_fail_{leg}", f"⛔ ALL CANDIDATES FAIL {leg} | {why}", 20)
    diag("hotswap_fail", leg=leg, reason=why)
    return (False, f"no_eligible:{why}")

# ------------------------------------------------------------
# TRUE OPEN (kept same core logic; but with safe wrappers)
# ------------------------------------------------------------
TRUE_OPEN_SCHEMA_VERSION = 122
_true_open_lock = threading.Lock()

OPEN_SOURCE_PRIORITY = {
    "JSON_RESTORE": 100,
    "WS_OHLC_OPEN_DAY": 95,
    "REST_OHLC_OPEN_TODAY": 85,
    "REST_OHLC_OPEN_DAY": 85,
    "CANDLE_0915_LAST_RESORT": 70,
    "CANDLE_EARLIEST_LAST_RESORT": 65,
    "WS_LTP_0915": 10,
    "WS_LTP_FALLBACK": 10}
TRUE_OPEN_SAVE_MIN_PRIORITY = 70

def _expected_tokens():
    return {"SPOT_TOKEN": int(SPOT_TOKEN), "CE_TOKEN": int(CE_TOKEN), "PE_TOKEN": int(PE_TOKEN)}

def save_true_open(ce_open, pe_open, ce_ts, pe_ts, *, leg_sources=None, leg_priorities=None):
    leg_sources = leg_sources or {}
    leg_priorities = leg_priorities or {}
    data = {
        "schema_version": TRUE_OPEN_SCHEMA_VERSION,
        "date": TODAY,
        "UNDERLYING": UNDERLYING,
        "EXPIRY_TYPE": EXPIRY_TYPE,
        "EXPIRY_DATE": str(EXPIRY_DATE),
        "SPOT_TOKEN": int(SPOT_TOKEN),
        "CE_TOKEN": int(CE_TOKEN),
        "PE_TOKEN": int(PE_TOKEN),
        "CE_symbol": CE_TSYM,
        "PE_symbol": PE_TSYM,
        "CE_strike": float(CE_STRIKE),
        "PE_strike": float(PE_STRIKE),
        "CE_open": float(ce_open),
        "PE_open": float(pe_open),
        "CE_ts": ce_ts,
        "PE_ts": pe_ts,
        "CE_source": leg_sources.get("CE"),
        "PE_source": leg_sources.get("PE"),
        "CE_priority": int(leg_priorities.get("CE", -1)),
        "PE_priority": int(leg_priorities.get("PE", -1))}
    try:
        with open(TRUE_OPEN_FILE, "w") as f:
            json.dump(data, f)
        tg_throttled("true_open_saved", f"💾 TRUE OPEN SAVED | CE={data['CE_open']:.2f} PE={data['PE_open']:.2f}", 10)
        diag("true_open_saved", CE=data["CE_open"], PE=data["PE_open"], CE_src=data["CE_source"], PE_src=data["PE_source"])
    except Exception as e:
        tg_throttled("true_open_save_err", f"⚠ TRUE OPEN save failed: {e}", 60)

def load_true_open(expected: dict = None):
    try:
        if not os.path.exists(TRUE_OPEN_FILE):
            return None
        with open(TRUE_OPEN_FILE, "r") as f:
            data = json.load(f)
        if data.get("schema_version") != TRUE_OPEN_SCHEMA_VERSION:
            return None
        if data.get("date") != TODAY:
            return None
        if expected:
            for k, v in expected.items():
                if int(data.get(k, -1)) != int(v):
                    return None
        return data
    except:
        return None

def gann_buy_levels(open_price: float, decimals: int = 2) -> dict:
    """Gann buy levels with full floating precision (NO rounding internally).
    NOTE: `decimals` is kept only for backward compatibility; rounding should be done
    only at display/log time, never for internal comparisons.
    """
    op = float(open_price)
    if op <= 0:
        return {"ENTRY": None, "SL": None, "T1": None, "T2": None, "T3": None, "T4": None, "T5": None, "T6": None, "T7": None, "T8": None}
    r = math.sqrt(op)
    def lvl(f):
        return (r + float(f)) ** 2

    out = {
        "SL":    lvl(0.03125),
        "ENTRY": lvl(0.0625),
        "T1":    lvl(0.125),
        "T2":    lvl(0.25),
        "T3":    lvl(0.375),
        "T4":    lvl(0.5)}
    out["Entry"] = out["ENTRY"]
    return out
def _src_pri(src: str) -> int:
    try:
        return int(OPEN_SOURCE_PRIORITY.get(str(src), 0))
    except:
        return 0

def apply_true_open(leg: str, price: float, ts_str: str, source: str):
    with _true_open_lock:
        st = global_state[leg]
        # If TRUE OPEN is explicitly locked, do not overwrite later.
        if st.get("open_locked") and str(source) != "JSON_RESTORE":
            return
        st.setdefault("open_priority", -1)

        new_pri = _src_pri(source)
        cur_pri = int(st.get("open_priority") or -1)

        if st.get("true_open_set") and new_pri <= cur_pri:
            return

        try:
            px = float(price)
        except:
            return
        if px <= 0:
            return

        was_set = bool(st.get("true_open_set"))

        st["open"] = float(px)
        st["open_ts"] = ts_str
        st["open_source"] = source
        st["open_priority"] = int(new_pri)
        st["true_open_set"] = True
        st["gann"] = gann_buy_levels(st["open"], decimals=2)
        # Lock TRUE OPEN for the day when it is at/above threshold (prevents later overrides)
        try:
            if float(st.get("open") or 0) >= float(globals().get("TRUE_OPEN_FIX_THRESHOLD", 141) or 141):
                st["open_locked"] = True
                st["open_lock_time"] = ts_str
        except:
            if FAIL_FAST: raise
            pass

        msg = f"🔥 TRUE OPEN {leg}={st['open']:.2f} | src={source} | pri={new_pri} | ts={ts_str}"
        print(msg)
        diag("true_open_apply", leg=leg, px=st["open"], src=source, pri=new_pri, ts=ts_str)

        if was_set:
            tg_throttled(f"open_override_{leg}", f"🔁 {msg}", 5)

globals()["apply_true_open"] = apply_true_open

def maybe_save_true_open_if_ready():
    CE = global_state["CE"]; PE = global_state["PE"]
    if not (CE.get("true_open_set") and PE.get("true_open_set")):
        return
    ce_ok = int(CE.get("open_priority") or -1) >= int(TRUE_OPEN_SAVE_MIN_PRIORITY)
    pe_ok = int(PE.get("open_priority") or -1) >= int(TRUE_OPEN_SAVE_MIN_PRIORITY)
    if not (ce_ok and pe_ok):
        return
    saved = load_true_open(expected=_expected_tokens())
    if saved is None:
        save_true_open(
            CE["open"], PE["open"], CE["open_ts"], PE["open_ts"],
            leg_sources={"CE": CE["open_source"], "PE": PE["open_source"]},
            leg_priorities={"CE": CE.get("open_priority"), "PE": PE.get("open_priority")}
        )

globals()["maybe_save_true_open_if_ready"] = maybe_save_true_open_if_ready

saved = load_true_open(expected=_expected_tokens())
if saved:
    apply_true_open("CE", saved["CE_open"], saved.get("CE_ts","NA"), source=saved.get("CE_source","JSON_RESTORE"))
    apply_true_open("PE", saved["PE_open"], saved.get("PE_ts","NA"), source=saved.get("PE_source","JSON_RESTORE"))
    log("📥 TRUE OPEN RESTORED (today, strict tokens match)")

def rest_ohlc_open_day(symbols: list):
    """
    Backup TRUE OPEN capture via REST quote day OHLC open.
    Uses: kite.quote([CE_TSYM, PE_TSYM]) -> quote[symbol]["ohlc"]["open"]
    """
    try:
        q = kite_quote(symbols)
        out = {}
        for s in symbols:
            d = (q.get(s) or {}) if isinstance(q, dict) else {}
            ohlc = (d.get("ohlc") or {}) if isinstance(d, dict) else {}
            op = ohlc.get("open")
            if op is None:
                continue
            try:
                op = float(op)
            except:
                continue
            if op <= 0:
                continue
            out[s] = op
        return out
    except:
        return {}

def _wait_until_dt(target_time: datetime.time):
    today = datetime.datetime.now(IST).date()
    target = datetime.datetime.combine(today, target_time, tzinfo=IST)
    while datetime.datetime.now(IST) < target and not block_stop.is_set():
        time.sleep(0.2)

def true_open_rest_fallback_loop():
    # Secondary backup: if WS not available by ~09:15:03, use REST quote day OHLC open
    if not TRUE_OPEN_USE_REST_OHLC_OPEN:
        return

    # wait until 09:15:03
    _wait_until_dt(datetime.time(9, 15, 3))

    t0 = time.time()
    max_wait = 30.0  # brief retry window (20–30s)
    symbols = [CE_TSYM, PE_TSYM]

    while not block_stop.is_set() and (time.time() - t0) <= max_wait:
        try:
            now_dt = _get_current_ist()
            opens = rest_ohlc_open_day(symbols)

            if (not global_state["CE"].get("open_locked")) and (CE_TSYM in opens):
                ce_open = float(opens[CE_TSYM])
                global_state["CE"]["rest_ohlc_open"] = ce_open
                apply_true_open("CE", ce_open, now_dt.strftime("%H:%M:%S"), "REST_OHLC_OPEN_DAY")
                global_state["CE"]["open_locked"] = True
                global_state["CE"]["open_src"] = "REST_OHLC_OPEN_DAY"
                global_state["CE"]["open_lock_time"] = now_dt

            if (not global_state["PE"].get("open_locked")) and (PE_TSYM in opens):
                pe_open = float(opens[PE_TSYM])
                global_state["PE"]["rest_ohlc_open"] = pe_open
                apply_true_open("PE", pe_open, now_dt.strftime("%H:%M:%S"), "REST_OHLC_OPEN_DAY")
                global_state["PE"]["open_locked"] = True
                global_state["PE"]["open_src"] = "REST_OHLC_OPEN_DAY"
                global_state["PE"]["open_lock_time"] = now_dt

            maybe_save_true_open_if_ready()

            if global_state["CE"].get("open_locked") and global_state["PE"].get("open_locked"):
                return
        except Exception as e:
            tg_throttled("rest_open", f"⚠ REST true-open fallback error: {e}", 60)
        time.sleep(1.0)

# start backup thread
threading.Thread(target=true_open_rest_fallback_loop, daemon=True).start()

log("✅ Block-1 ready (REST hardened + canonical time + strike + paper-lockout + TrueOpen + ATR/N19 fields ready)")

# ============================================================
# BLOCK 2/6 — INDICATORS ✅ (UPDATED v5.2 — N19 + Block-5 COMPAT)
#  ✅ Uses Block-1 canonical as_ist_dt() ONLY (no duplicate converters)
#  ✅ CandleAgg ignores pre-09:15 ticks (fixes ST/OB frozen)
#  ✅ Intraday VWAP resets daily + stores vwap_hist for slope filter
#  ✅ Option VWAP(1m) computed for CE/PE
#  ✅ Option ATR14(1m) Wilder computed for CE/PE (keeps old behavior)
#  ✅ NEW: Option ATR7 SMA(points) stored in global_state[leg]["atr"] (for N19)
#  ✅ NEW: Option 1m candle fields stored: bar_open/high/low/close/bar_ts (for N19/CSV)
# ============================================================

import datetime, math, time, copy
from collections import deque
from typing import Optional, List

IST = globals().get("IST") or datetime.timezone(datetime.timedelta(hours=5, minutes=30))
UTC = datetime.timezone.utc
market_open_time = globals().get("market_open_time", datetime.time(9, 15, 0))

log2 = globals().get("log", print)
tg2  = globals().get("tg", lambda *_: None)
diag = globals().get("diag", lambda *_a, **_k: None)

# -------------------------
# ✅ CANONICAL TIME (from Block-1)
# -------------------------
as_ist_dt = globals().get("as_ist_dt")
if not callable(as_ist_dt):
    raise RuntimeError("Block-2 requires Block-1 as_ist_dt() to exist. Do NOT run Block-2 before Block-1.")

def _as_ist_dt(d: datetime.datetime) -> Optional[datetime.datetime]:
    try:
        return as_ist_dt(d, datetime.datetime.now(IST))
    except:
        return None

globals()["_as_ist_dt"] = _as_ist_dt  # keep compatibility for any older calls

# -------------------------
# Ensure state exists
# -------------------------
spot_state = globals().get("spot_state")
global_state = globals().get("global_state")

if not isinstance(spot_state, dict):
    spot_state = {"ltp": None, "ts": None}
    globals()["spot_state"] = spot_state
if not isinstance(global_state, dict):
    global_state = {"CE": {}, "PE": {}}
    globals()["global_state"] = global_state

# Also ensure canonical CE/PE aliases exist here (in case Block-1 was interrupted).
try:
    globals()["ce_state"] = global_state.setdefault("CE", {})
    globals()["pe_state"] = global_state.setdefault("PE", {})
    _req = globals().get("_REQUIRED_15M_KEYS") or {
        "rsi3_15m": None,
        "rsi3_15m_ts": None,
        "vwap_15m": None,
        "imb15_cur": None,
        "imb15_ts": None,
        "imb_avg20_15m": None,
    }
    for _k, _v in _req.items():
        globals()["ce_state"].setdefault(_k, _v)
        globals()["pe_state"].setdefault(_k, _v)
except Exception:
    pass

# Ensure keys for slope filter + VWAP session
spot_state.setdefault("spot_5m_bars", deque(maxlen=600))
spot_state.setdefault("last_5m_bar", None)
spot_state.setdefault("last_5m_bar_time", None)

spot_state.setdefault("vwap", None)
spot_state.setdefault("vwap_ts", None)
spot_state.setdefault("_vwap_pv", 0.0)
spot_state.setdefault("_vwap_v", 0.0)
spot_state.setdefault("_vwap_session_date", None)
spot_state.setdefault("vwap_hist", deque(maxlen=5000))  # (time, vwap, spot_close)

spot_state.setdefault("atr14_1m", None)
spot_state.setdefault("atr_last", None)
spot_state.setdefault("st60_dir", None)
spot_state.setdefault("st60_value", None)
spot_state.setdefault("st60_ts", None)
spot_state.setdefault("st60_err", None)

spot_state.setdefault("ob_zones", [])

for lg in ("CE", "PE"):
    global_state.setdefault(lg, {})

    # existing indicator fields
    global_state[lg].setdefault("vwap_1m", None)
    global_state[lg].setdefault("imb_ratio", None)
    global_state[lg].setdefault("imb_signal", None)
    global_state[lg].setdefault("imb_ts", None)
    global_state[lg].setdefault("atr14_1m", None)  # option ATR14 Wilder (old)

    # ✅ NEW fields required by updated N19 + Block-5 logging
    global_state[lg].setdefault("atr", None)        # ATR7 SMA(points)
    global_state[lg].setdefault("bar_open", None)
    global_state[lg].setdefault("bar_high", None)
    global_state[lg].setdefault("bar_low", None)
    global_state[lg].setdefault("bar_close", None)
    global_state[lg].setdefault("bar_ts", None)

    global_state[lg].setdefault("c15_close", None)   # last closed slow_tf candle dict

# -------------------------
# Candle Aggregator (session aligned + ignores pre-open)
# -------------------------

def _rsi_preview_wilder(obj: 'RSIWilder', close_preview: float):
    """Preview Wilder RSI without mutating the RSI object.
    Used to update RSI(3,15m) LIVE on every tick using the in-progress candle close (Zerodha-style)."""
    try:
        if obj is None:
            return None
        p = int(getattr(obj, "p", 3) or 3)
        prev = getattr(obj, "prev", None)
        ag = getattr(obj, "avg_gain", None)
        al = getattr(obj, "avg_loss", None)
        if prev is None or ag is None or al is None:
            return None

        c = float(close_preview)
        chg = c - float(prev)
        gain = chg if chg > 0 else 0.0
        loss = -chg if chg < 0 else 0.0

        ag2 = (float(ag) * (p - 1) + gain) / float(p)
        al2 = (float(al) * (p - 1) + loss) / float(p)

        if al2 == 0:
            return 100.0
        rs = ag2 / al2
        return 100.0 - (100.0 / (1.0 + rs))
    except Exception:
        return None


class CandleAgg:
    def __init__(self, tf_min: int):
        self.tf = int(tf_min)
        self.bucket = None
        self.o = self.h = self.l = self.c = None
        self.v = 0.0

    def _session_start(self, ts: datetime.datetime) -> datetime.datetime:
        return datetime.datetime.combine(ts.date(), market_open_time, tzinfo=IST)

    def _bucket_start(self, ts: datetime.datetime) -> datetime.datetime:
        start = self._session_start(ts)
        mins = int((ts - start).total_seconds() // 60)
        b = (mins // self.tf) * self.tf
        return start + datetime.timedelta(minutes=b)

    def update(self, ts: datetime.datetime, price: float, vol_delta: float = 0.0):
        ts = _as_ist_dt(ts) or ts
        if not isinstance(ts, datetime.datetime):
            return None

        start = self._session_start(ts)
        if ts < start:
            # ✅ ignore pre-open ticks
            return None

        b = self._bucket_start(ts)

        if self.bucket is None:
            self.bucket = b
            self.o = self.h = self.l = self.c = float(price)
            self.v = float(vol_delta or 0.0)
            return None

        if b == self.bucket:
            self.h = max(float(self.h), float(price))
            self.l = min(float(self.l), float(price))
            self.c = float(price)
            self.v += float(vol_delta or 0.0)
            return None

        closed = {
            "time": self.bucket,
            "open": float(self.o), "high": float(self.h), "low": float(self.l), "close": float(self.c),
            "volume": float(self.v)}

        self.bucket = b
        self.o = self.h = self.l = self.c = float(price)
        self.v = float(vol_delta or 0.0)
        return closed

globals()["CandleAgg"] = CandleAgg

# -------------------------
# Wilder ATR
# -------------------------
def _true_range(h, l, prev_c):
    if prev_c is None:
        return float(h) - float(l)
    return max(float(h) - float(l), abs(float(h) - float(prev_c)), abs(float(l) - float(prev_c)))

class ATRWilder:
    def __init__(self, period: int):
        self.p = int(period)
        self.prev_c = None
        self.atr = None
        self._seed = []

    def update(self, h, l, c):
        tr = _true_range(h, l, self.prev_c)
        self.prev_c = float(c)

        if self.atr is None:
            self._seed.append(float(tr))
            if len(self._seed) < self.p:
                return None
            self.atr = float(sum(self._seed)) / float(self.p)
            return self.atr

        self.atr = (self.atr * (self.p - 1) + float(tr)) / float(self.p)
        return self.atr



# (SuperTrend removed by request)
class LuxOB:
    def __init__(self, length=5, bull_keep=3, bear_keep=3, mitigation="Wick"):
        self.length = int(length)
        self.bull_keep = int(bull_keep)
        self.bear_keep = int(bear_keep)
        self.mitigation = str(mitigation)

        self.highs  = deque(maxlen=5000)
        self.lows   = deque(maxlen=5000)
        self.closes = deque(maxlen=5000)
        self.hl2s   = deque(maxlen=5000)
        self.vols   = deque(maxlen=5000)
        self.times  = deque(maxlen=5000)

        self.os = 0
        self.bull = deque(maxlen=200)
        self.bear = deque(maxlen=200)

    def _pivot_high_strict(self) -> bool:
        L = self.length
        if len(self.vols) < (2 * L + 1):
            return False
        arr = list(self.vols)
        left  = arr[-(2*L+1):-(L+1)]
        center = arr[-(L+1)]
        right = arr[-L:]
        try:
            c = float(center)
            if not math.isfinite(c) or c <= 0:
                return False
            return (c > max(left)) and (c > max(right))
        except:
            return False

    def _add_zone(self, dq: deque, ztype: str, top: float, btm: float, t: datetime.datetime, pivot_close: float):
        avg = (float(top) + float(btm)) / 2.0
        zone_id = f"{ztype}_{t.strftime('%Y%m%d_%H%M')}_{int(top*100)}"
        dq.appendleft({
            "id": zone_id,
            "type": ztype,
            "top": float(top),
            "bottom": float(btm),
            "eq": float(avg),
            "ts": t,
            "pivot_close": float(pivot_close)})
        keep = self.bull_keep if ztype == "BULL" else self.bear_keep
        while len(dq) > keep:
            dq.pop()

    def _remove_mitigated(self, target: float, bull: bool):
        if bull:
            kept = []
            for z in list(self.bull):
                if float(target) < float(z["bottom"]):
                    continue
                kept.append(z)
            self.bull = deque(kept, maxlen=200)
        else:
            kept = []
            for z in list(self.bear):
                if float(target) > float(z["top"]):
                    continue
                kept.append(z)
            self.bear = deque(kept, maxlen=200)

    def update_bar(self, bar: dict):
        t = bar["time"]
        h = float(bar["high"])
        l = float(bar["low"])
        c = float(bar["close"])
        v = float(bar.get("volume") or 0.0)
        hl2 = (h + l) / 2.0

        self.highs.append(h); self.lows.append(l); self.closes.append(c)
        self.hl2s.append(hl2); self.times.append(t)
        self.vols.append(v)

        L = self.length
        if len(self.highs) < (L + 2):
            return

        upper = max(list(self.highs)[-L:])
        lower = min(list(self.lows)[-L:])

        hL = list(self.highs)[-(L+1)]
        lL = list(self.lows)[-(L+1)]
        if hL > upper:
            self.os = 0
        elif lL < lower:
            self.os = 1

        if self.mitigation.lower() == "close":
            target_bull = min(list(self.closes)[-L:])
            target_bear = max(list(self.closes)[-L:])
        else:
            target_bull = lower
            target_bear = upper

        phv = self._pivot_high_strict()
        if phv:
            pivot_time = list(self.times)[-(L+1)]
            pivot_close = list(self.closes)[-(L+1)]

            if self.os == 1:
                top = list(self.hl2s)[-(L+1)]
                btm = list(self.lows)[-(L+1)]
                self._add_zone(self.bull, "BULL", top, btm, pivot_time, pivot_close)

            if self.os == 0:
                top = list(self.highs)[-(L+1)]
                btm = list(self.hl2s)[-(L+1)]
                self._add_zone(self.bear, "BEAR", top, btm, pivot_time, pivot_close)

        self._remove_mitigated(target_bull, bull=True)
        self._remove_mitigated(target_bear, bull=False)

    def active_zones(self) -> List[dict]:
        return list(self.bull) + list(self.bear)

# -------------------------
# Aggregators
# -------------------------
spot_1m  = CandleAgg(1)
spot_5m  = CandleAgg(5)
spot_60m = CandleAgg(60)

atr_spot_14 = ATRWilder(14)
atr_ce_14   = ATRWilder(14)
atr_pe_14   = ATRWilder(14)
rsi_ce_3    = RSIWilder(3)   # legacy 1m
rsi_pe_3    = RSIWilder(3)   # legacy 1m
rsi_ce_3_15m = RSIWilder(3)
rsi_pe_3_15m = RSIWilder(3)
# --- PRIME RSI(3) 15m objects so LIVE RSI moves from the first tick (Zerodha-style) ---
def _prime_rsi15_objects():
    try:
        now_ist = datetime.datetime.now(IST)
        seed_day = TODAY_DATE
        if now_ist.time() < market_open_time:
            seed_day = _prev_trading_day(seed_day)

        from_dt = datetime.datetime.combine(seed_day, datetime.time(9, 15))
        to_dt   = datetime.datetime.combine(seed_day, datetime.time(15, 30))

        ce15 = kite.historical_data(int(CE_TOKEN), from_dt, to_dt, interval="15minute") or []
        pe15 = kite.historical_data(int(PE_TOKEN), from_dt, to_dt, interval="15minute") or []

        ce_closes = [float(c["close"]) for c in ce15 if "close" in c]
        pe_closes = [float(c["close"]) for c in pe15 if "close" in c]

        for x in ce_closes:
            rsi_ce_3_15m.update(x)
        for x in pe_closes:
            rsi_pe_3_15m.update(x)

        print(f"[RSI15 PRIME] day={seed_day} CE_closes={len(ce_closes)} PE_closes={len(pe_closes)}")
    except Exception as e:
        print("[RSI15 PRIME] skipped:", e)

_prime_rsi15_objects()
ob5      = LuxOB(length=5, bull_keep=3, bear_keep=3, mitigation="Wick")

ce_1m = CandleAgg(1)
pe_1m = CandleAgg(1)

ce_3m = CandleAgg(3)
pe_3m = CandleAgg(3)
ce_15m = CandleAgg(15)
pe_15m = CandleAgg(15)

class _EMA:
    """EMA with SMA seed (matches most charting implementations incl. Zerodha/TradingView)."""
    def __init__(self, period: int):
        self.p = int(period)
        self.k = 2.0 / (float(self.p) + 1.0)
        self.v = None
        self._seed = []
    def update(self, x: float):
        x = float(x)
        if self.v is None:
            self._seed.append(x)
            if len(self._seed) < self.p:
                return None
            if len(self._seed) == self.p:
                self.v = sum(self._seed) / float(self.p)
                return self.v
        self.v = self.v + self.k * (x - self.v)
        return self.v

_ce_prev_close = None
_pe_prev_close = None

def _tr_points(h, l, prev_c):
    h = float(h); l = float(l)
    if prev_c is None:
        return max(0.0, h - l)
    prev_c = float(prev_c)
    return max(h - l, abs(h - prev_c), abs(l - prev_c))

def _depth_imbalance_ratio(depth: dict) -> Optional[float]:
    try:
        buy = (depth or {}).get("buy") or []
        sell = (depth or {}).get("sell") or []
        bq = sum(float(x.get("quantity") or 0.0) for x in buy[:5])
        sq = sum(float(x.get("quantity") or 0.0) for x in sell[:5])
        if bq <= 0 or sq <= 0:
            return None
        return float(bq / sq)
    except:
        return None

def _publish_latest_ob_to_option_cards(zones: List[dict]):
    bull = next((z for z in zones if z.get("type") == "BULL"), None)
    bear = next((z for z in zones if z.get("type") == "BEAR"), None)

    for lg in ("CE", "PE"):
        gs = global_state[lg]
        gs["ob_tf_min"] = 5
        gs["ob_pivot_len"] = 5

        if bull:
            gs["ob_bull_low"]  = bull["bottom"]
            gs["ob_bull_high"] = bull["top"]
            gs["ob_bull_mid"]  = bull["eq"]
            gs["ob_bull_ts"]   = bull["ts"]
            gs["ob_bull_pivot_close"] = bull.get("pivot_close")
        else:
            gs["ob_bull_low"] = gs["ob_bull_high"] = gs["ob_bull_mid"] = gs["ob_bull_ts"] = gs["ob_bull_pivot_close"] = None

        if bear:
            gs["ob_bear_low"]  = bear["bottom"]
            gs["ob_bear_high"] = bear["top"]
            gs["ob_bear_mid"]  = bear["eq"]
            gs["ob_bear_ts"]   = bear["ts"]
            gs["ob_bear_pivot_close"] = bear.get("pivot_close")
        else:
            gs["ob_bear_low"] = gs["ob_bear_high"] = gs["ob_bear_mid"] = gs["ob_bear_ts"] = gs["ob_bear_pivot_close"] = None

# -------------------------
# Seed (warms ST/ATR/OB; VWAP resets intraday)
# -------------------------
def _seed_from_history():
    kite = globals().get("kite")
    spot_token = globals().get("SPOT_TOKEN")
    if kite is None or spot_token is None:
        log2("⚠ Block-2 seed skipped (kite/SPOT_TOKEN missing).")
        return

    spot_token = int(spot_token)
    now_dt = datetime.datetime.now(IST)

    from_1m  = now_dt - datetime.timedelta(days=6)
    from_5m  = now_dt - datetime.timedelta(days=12)
    from_60m = now_dt - datetime.timedelta(days=60)

    try:
        candles_1m = kite.historical_data(spot_token, from_1m, now_dt, interval="minute") or []
        for c in candles_1m[-6000:]:
            atr = atr_spot_14.update(c["high"], c["low"], c["close"])
            if atr is not None:
                spot_state["atr14_1m"] = float(atr)
                spot_state["atr_last"] = float(atr)
    except Exception as e:
        log2(f"⚠ Block-2 seed 1m failed: {e}")

    try:
        ce_token = globals().get("CE_TOKEN")
        pe_token = globals().get("PE_TOKEN")

        def _seed_leg(kind: str, token: int) -> bool:
            if token is None:
                return False
            token = int(token)
            now_dt = datetime.datetime.now(IST)
            from_dt = now_dt - datetime.timedelta(days=12)

            candles = kite.historical_data(token, from_dt, now_dt, interval="15minute") or []
            closes = []
            last_c15 = None

            for c in candles:
                dt = _as_ist_dt(c.get("date")) or c.get("date")
                if not isinstance(dt, datetime.datetime):
                    continue

                # keep only regular session candles (matches Zerodha chart)
                t = dt.time()
                if t < market_open_time or t > market_close_time:
                    continue

                closes.append(float(c["close"]))
                last_c15 = {
                    "time": dt,
                    "open": float(c["open"]), "high": float(c["high"]), "low": float(c["low"]), "close": float(c["close"]),
                    "volume": float(c.get("volume") or 0.0)}

            if len(closes) < 25 or last_c15 is None:
                return False
            st = global_state[kind]
            st["c15_close"] = last_c15
            try:
                candles3 = kite.historical_data(token, from_dt, now_dt, interval="3minute") or []
                closes3 = []
                last_c3 = None
                for c in candles3:
                    dt = _as_ist_dt(c.get("date")) or c.get("date")
                    if not isinstance(dt, datetime.datetime):
                        continue
                    t = dt.time()
                    if t < market_open_time or t > market_close_time:
                        continue
                    closes3.append(float(c["close"]))
                    last_c3 = {
                        "time": dt,
                        "open": float(c["open"]), "high": float(c["high"]), "low": float(c["low"]), "close": float(c["close"]),
                        "volume": float(c.get("volume") or 0.0)}

                if len(closes3) >= 25 and last_c3 is not None:
                    st["c3_close"] = last_c3
            except:
                if FAIL_FAST: raise
                pass

            try:
                gls = globals().get("gl_strategy")
                if gls and hasattr(gls, "update_slow_tf_candle"):
                    gls.update_slow_tf_candle(kind, last_c15)
            except:
                if FAIL_FAST: raise
                pass

            return True

        ok_ce = _seed_leg("CE", ce_token)
        ok_pe = _seed_leg("PE", pe_token)
        if ok_ce or ok_pe:
            log2("✅ Option 15 seeded from history (ready pre-open)")
    except Exception as e:
        log2(f"⚠ Option 15 seed failed: {e}")

    # (SuperTrend 60m seeding removed)

    try:
        candles_5m = kite.historical_data(spot_token, from_5m, now_dt, interval="5minute") or []
        for c in candles_5m[-4000:]:
            dt = _as_ist_dt(c.get("date")) or c.get("date")
            if not isinstance(dt, datetime.datetime):
                continue
            bar = {
                "time": dt,
                "open": c["open"], "high": c["high"], "low": c["low"], "close": c["close"],
                "volume": float(c.get("volume") or 0.0)}
            ob5.update_bar(bar)
            spot_state["spot_5m_bars"].append(bar)
            spot_state["last_5m_bar"] = bar
            spot_state["last_5m_bar_time"] = bar["time"]

        zones = ob5.active_zones()
        spot_state["ob_zones"] = zones
        _publish_latest_ob_to_option_cards(zones)
    except Exception as e:
        log2(f"⚠ Block-2 seed 5m failed: {e}")

    log2("✅ Block-2 SEED done (ATR/ST/OB warmed; VWAP resets intraday)")

_seed_from_history()

def _seed_vwap_from_prev_day_spot(session_date: datetime.date):
    """Seed VWAP accumulators using previous trading day's historical 1m candles.

    Goal: have VWAP ready at market open (09:15) and closer to broker VWAP style.
    Best-effort: if historical fetch fails, VWAP will start from first tick.
    """
    if not SEED_VWAP_FROM_PREV_DAY:
        return

    # previous day date (calendar); broker holiday handling is best-effort
    prev_day = session_date - datetime.timedelta(days=1)

    try:
        token = spot_state.get("token") or spot_state.get("instrument_token")
        if not token:
            return
        if "kite" not in globals():
            return
        kc = globals().get("kite")
        if kc is None:
            return

        # Fetch prev-day 1m candles (historical)
        from_dt = datetime.datetime.combine(prev_day, datetime.time(9, 15))
        to_dt   = datetime.datetime.combine(prev_day, datetime.time(15, 30))
        candles = kc.historical_data(int(token), from_dt, to_dt, "minute")
        pv = 0.0
        vv = 0.0
        for c in candles or []:
            # Kite format: dict with 'high','low','close','volume' or list; handle both
            try:
                h = float(c.get("high")); l = float(c.get("low")); cl = float(c.get("close"))
                v = float(c.get("volume") or 0.0)
            except Exception:
                # list format: [date, open, high, low, close, volume]
                try:
                    h = float(c[2]); l = float(c[3]); cl = float(c[4]); v = float(c[5] or 0.0)
                except Exception:
                    continue
            if v <= 0:
                continue
            tp = (h + l + cl) / 3.0
            pv += tp * v
            vv += v

        if vv > 0:
            spot_state["_vwap_pv"] = float(pv)
            spot_state["_vwap_v"]  = float(vv)
            spot_state["vwap"]     = float(pv / vv)
            spot_state["vwap_ts"]  = None
    except Exception:
        # Silent fail; bot will compute VWAP from live ticks
        return

def _reset_vwap_session_if_needed(ts: datetime.datetime):
    d = ts.date()
    if spot_state.get("_vwap_session_date") != d:
        spot_state["_vwap_session_date"] = d
        spot_state["_vwap_pv"] = 0.0
        spot_state["_vwap_v"] = 0.0
        spot_state["vwap"] = None
        spot_state["vwap_ts"] = None
        spot_state["vwap_hist"].clear()
        _seed_vwap_from_prev_day_spot(d)

def _reset_opt_vwap_if_needed(ts: datetime.datetime):
    global _opt_vwap_date, _ce_pv, _ce_v, _pe_pv, _pe_v, _ce15_pv, _ce15_v, _pe15_pv, _pe15_v
    global _ce_tr_q, _pe_tr_q, _ce_prev_close, _pe_prev_close

    d = ts.date()
    if _opt_vwap_date != d:
        _opt_vwap_date = d
        _ce_pv = float(_ce_pv_seed) if SEED_VWAP_FROM_PREV_DAY else 0.0
        _ce_v  = float(_ce_v_seed)  if SEED_VWAP_FROM_PREV_DAY else 0.0
        _pe_pv = float(_pe_pv_seed) if SEED_VWAP_FROM_PREV_DAY else 0.0
        _pe_v  = float(_pe_v_seed)  if SEED_VWAP_FROM_PREV_DAY else 0.0
        _ce15_pv = _ce15_v = 0.0
        _pe15_pv = _pe15_v = 0.0  # slow_tf VWAP buckets reset
        global_state["CE"]["vwap_1m"] = None
        global_state["PE"]["vwap_1m"] = None
        global_state["CE"]["vwap_slow_tf"] = None
        global_state["PE"]["vwap_slow_tf"] = None

        # ✅ reset ATR7 SMA trackers
        _ce_tr_q.clear()
        _pe_tr_q.clear()
        _ce_prev_close = None
        _pe_prev_close = None

# -------------------------
# MAIN UPDATE FUNCTION (called by Block-5 on every tick)
# -------------------------
def update_indicators_from_tick(kind: str, ts: datetime.datetime, ltp: float, vol_delta: float, depth: dict = None):
    ts = _as_ist_dt(ts) or ts
    if not isinstance(ts, datetime.datetime):
        return
    kind = str(kind).upper()
    global _ce15_pv, _ce15_v, _pe15_pv, _pe15_v, _opt_vwap_date
    global _ce_tr_q, _pe_tr_q, _ce_prev_close, _pe_prev_close


    # -------- SPOT --------
    if kind == "SPOT":
        spot_state["ltp"] = float(ltp)
        spot_state["ts"]  = ts

        _reset_vwap_session_if_needed(ts)

        c1 = spot_1m.update(ts, ltp, vol_delta)
        if c1:
            atr = atr_spot_14.update(c1["high"], c1["low"], c1["close"])
            if atr is not None:
                spot_state["atr14_1m"] = float(atr)
                spot_state["atr_last"] = float(atr)

            v = float(c1.get("volume") or 0.0)
            if v > 0:
                tp = (float(c1["high"]) + float(c1["low"]) + float(c1["close"])) / 3.0
                spot_state["_vwap_pv"] += tp * v
                spot_state["_vwap_v"]  += v
                if spot_state["_vwap_v"] > 0:
                    vw = spot_state["_vwap_pv"] / spot_state["_vwap_v"]
                    spot_state["vwap"] = float(vw)
                    spot_state["vwap_ts"] = c1["time"]
                    spot_state["vwap_hist"].append((c1["time"], float(vw), float(c1["close"])))

        c5 = spot_5m.update(ts, ltp, vol_delta)
        if c5:
            ob5.update_bar(c5)
            zones = ob5.active_zones()
            spot_state["ob_zones"] = zones
            _publish_latest_ob_to_option_cards(zones)

            spot_state["spot_5m_bars"].append(c5)
            spot_state["last_5m_bar"] = c5
            spot_state["last_5m_bar_time"] = c5["time"]

        c60 = spot_60m.update(ts, ltp, vol_delta)
        if c60:
            # (SuperTrend removed) keep placeholders for dashboard/backward-compat
            spot_state["st60_dir"] = None
            spot_state["st60_value"] = None
            spot_state["st60_ts"] = c60.get("time")
            spot_state["st60_err"] = None
        return

    # -------- OPTIONS (CE/PE) --------
    if kind in ("CE", "PE"):
        _reset_opt_vwap_if_needed(ts)

        st = global_state[kind]
        st["vol_delta"] = float(vol_delta or 0.0)

        # ✅ IMB ratio from market depth (required by GL/GB/N19)
        # depth is expected like: {"buy":[{"quantity":...},...], "sell":[...]}
        try:
            r = _depth_imbalance_ratio(depth) if depth is not None else None
            if r is not None:
                st["imb_ratio"]  = float(r)
                st["imb_signal"] = "BULL" if float(r) >= 1.0 else "BEAR"
                st["imb_ts"]     = ts
                try:
                    _update_imb15(kind, ts, float(r))
                except Exception:
                    pass
        except Exception:
            if FAIL_FAST:
                raise

                    # ✅ Tick-based VWAP (Zerodha-style: value/volume)
        vd = float(vol_delta or 0.0)
        try:
            if vd > 0:
                if kind == "CE":
                    _ce_pv += float(ltp) * vd
                    _ce_v  += vd
                    st["vwap"] = (_ce_pv / _ce_v) if _ce_v > 0 else None
                else:
                    _pe_pv += float(ltp) * vd
                    _pe_v  += vd
                    st["vwap"] = (_pe_pv / _pe_v) if _pe_v > 0 else None

                # ✅ Mirror live VWAP into strategy field
                st["vwap_15m"] = st.get("vwap")

                # timestamps
                st["vwap_ts"] = ts
                st["vwap_15m_ts"] = ts
        except Exception:
            pass

        # 🔴 THIS MUST BE DEDENTED (same level as vd = ...)
        agg = ce_1m if kind == "CE" else pe_1m
        c1 = agg.update(ts, ltp, vol_delta)

        if c1:
            # ✅ publish latest option candle fields (needed by N19/CSV)
            st["bar_open"]  = float(c1["open"])
            st["bar_high"]  = float(c1["high"])
            st["bar_low"]   = float(c1["low"])
            st["bar_close"] = float(c1["close"])
            st["bar_ts"]    = c1["time"]

            # ✅ Option ATR14 Wilder (old behavior)
            atr_obj = atr_ce_14 if kind == "CE" else atr_pe_14
            atrv = atr_obj.update(c1["high"], c1["low"], c1["close"])
            if atrv is not None:
                st["atr14_1m"] = float(atrv)

            # ✅ RSI(3) on option close (Wilder)
            try:
                rsi_obj = rsi_ce_3 if kind == "CE" else rsi_pe_3
                rsi_v = rsi_obj.update(c1["close"])
                if rsi_v is not None:
                    st["rsi3_1m"] = float(rsi_v)
                    st["rsi3_ts"] = c1["time"]
            except Exception:
                pass

            # ✅ Option VWAP 1m
            tp = (c1["high"] + c1["low"] + c1["close"]) / 3.0
            v  = float(c1.get("volume") or 0.0)
            if v > 0:
                if kind == "CE":
                    _ce_pv += float(tp) * v
                    _ce_v  += v
                    st["vwap_1m"] = (_ce_pv / _ce_v) if _ce_v > 0 else None
                else:
                    _pe_pv += float(tp) * v
                    _pe_v  += v
                    st["vwap_1m"] = (_pe_pv / _pe_v) if _pe_v > 0 else None

            # ✅ Option ATR7 SMA(points) for updated N19
            global _ce_prev_close, _pe_prev_close, _ce_tr_q, _pe_tr_q
            if kind == "CE":
                tr = _tr_points(c1["high"], c1["low"], _ce_prev_close)
                _ce_tr_q.append(tr)
                _ce_prev_close = float(c1["close"])
                st["atr"] = (sum(_ce_tr_q) / len(_ce_tr_q)) if len(_ce_tr_q) >= 2 else None
            else:
                tr = _tr_points(c1["high"], c1["low"], _pe_prev_close)
                _pe_tr_q.append(tr)
                _pe_prev_close = float(c1["close"])
                st["atr"] = (sum(_pe_tr_q) / len(_pe_tr_q)) if len(_pe_tr_q) >= 2 else None

        agg3 = ce_3m if kind == "CE" else pe_3m
        c3 = agg3.update(ts, ltp, vol_delta)

        if c3:
            try:
                st["c3_close"] = float(c3["close"])
                st["c3_ts"] = c3.get("time")
            except Exception:
                st["c3_close"] = None
                st["c3_ts"] = None

        # ✅ 15-min candle (for RSI(3) + VWAP on 15m as requested)
        agg15 = ce_15m if kind == "CE" else pe_15m
        c15 = agg15.update(ts, ltp, vol_delta)
        if c15:
            try:
                st["c15_close"] = float(c15["close"])
                st["c15_ts"] = c15.get("time")
            except Exception:
                st["c15_close"] = None
                st["c15_ts"] = None

            # RSI(3) on 15m close (continuous)
            try:
                rsi15 = rsi_ce_3_15m if kind == "CE" else rsi_pe_3_15m
                rsi_v = rsi15.update(c15["close"])
                if rsi_v is not None:
                    st["rsi3_15m"] = float(rsi_v)
                    st["rsi3_15m_ts"] = c15.get("time")
            except Exception:
                pass

            # VWAP computed from 15m candles (typical_price * volume) - matches chart VWAP logic per timeframe
            try:
                v = float(c15.get("volume") or 0.0)
                if v > 0:
                    tp = (float(c15["high"]) + float(c15["low"]) + float(c15["close"])) / 3.0
                    if kind == "CE":
                        _ce15_pv += tp * v
                        _ce15_v  += v
                        st["vwap_15m"] = (_ce15_pv / _ce15_v) if _ce15_v > 0 else None
                    else:
                        _pe15_pv += tp * v
                        _pe15_v  += v
                        st["vwap_15m"] = (_pe15_pv / _pe15_v) if _pe15_v > 0 else None
            except Exception:
                pass


        # ✅ LIVE RSI(3,15m) — update on EVERY tick using in-progress 15m candle close (Zerodha-style)
        try:
            rsi15_obj = rsi_ce_3_15m if kind == "CE" else rsi_pe_3_15m
            cur_close = float(getattr(agg15, "c", ltp) or ltp)   # current forming 15m candle close
            rsi_live = _rsi_preview_wilder(rsi15_obj, cur_close)
            if rsi_live is not None:
                st["rsi3_15m"] = float(rsi_live)
                st["rsi3_15m_ts"] = ts
        except Exception:
            pass


# ----------------------------
# Defensive globals
# ----------------------------
IST = globals().get("IST") or datetime.timezone(datetime.timedelta(hours=5, minutes=30))
UTC = datetime.timezone.utc

SQUARE_OFF_TIME = globals().get("SQUARE_OFF_TIME", datetime.time(15, 15))
SLIPPAGE_PER_SIDE = float(globals().get("SLIPPAGE_PER_SIDE", 0.18) or 0.18)
BROKERAGE_PER_ORDER = float(globals().get("BROKERAGE_PER_ORDER", 20) or 20)

QTY = int(globals().get("QTY", 0) or 0)
RUN_ID = str(globals().get("RUN_ID", "RUN"))
TRADES_CSV = str(globals().get("TRADES_CSV", "trades.csv"))
UNDERLYING = str(globals().get("UNDERLYING", "NIFTY")).upper()

PAPER_TRADING = bool(globals().get("PAPER_TRADING", True))
DEMO_MODE = bool(globals().get("DEMO_MODE", False))
ALLOW_CONCURRENT_LAYERS = bool(globals().get("ALLOW_CONCURRENT_LAYERS", True))
MAX_SPREAD = float(globals().get("MAX_SPREAD", 1e9))

kite = globals().get("kite", None)
CE_TSYM = str(globals().get("CE_TSYM", ""))
PE_TSYM = str(globals().get("PE_TSYM", ""))

global_state = globals().get("global_state", {}) or {}
spot_state = globals().get("spot_state", {}) or {}
global_summary = globals().get("global_summary", {"GL": 0.0, "GB": 0.0, "N19": 0.0}) or {"GL": 0.0, "GB": 0.0, "N19": 0.0}
globals()["global_summary"] = global_summary
global_summary.setdefault("GL", 0.0)

global_summary.setdefault("GB", 0.0)
global_summary.setdefault("N19", 0.0)
tg = globals().get("tg", lambda *_a, **_k: None)
log = globals().get("log", print)
diag = globals().get("diag", lambda *_a, **_k: None)
tg_throttled = globals().get("tg_throttled", lambda *_a, **_k: None)

# ----------------------------
# Canonical time (must come from Block-1)
# ----------------------------
as_ist_dt = globals().get("as_ist_dt")
if not callable(as_ist_dt):
    raise RuntimeError("Block-3 requires Block-1 as_ist_dt(). Run Block-1 first.")

def _as_ist_dt(d):
    try:
        return as_ist_dt(d, datetime.datetime.now(IST))
    except:
        try:
            return as_ist_dt(d)
        except:
            return None

globals()["_as_ist_dt"] = _as_ist_dt

now_ist = globals().get("now_ist")
if not callable(now_ist):
    def now_ist():
        return datetime.datetime.now(IST)
    globals()["now_ist"] = now_ist

# ----------------------------
# Signal emitter (Block-4 adds emit_signal later)
# ----------------------------
def _emit(strategy: str, event: str, leg: Optional[str] = None,
          price: Optional[float] = None, reason: str = "", meta: Optional[dict] = None):
    fn = globals().get("emit_signal")
    if callable(fn):
        try:
            fn(strategy=strategy, event=event, leg=leg, price=price, reason=reason, meta=(meta or {}))
        except:
            if FAIL_FAST: raise
            pass

# ----------------------------
# Safe stop (only meaningful in live mode; paper mode should NOT crash)
# ----------------------------
safe_stop = globals().get("safe_stop")
if not callable(safe_stop):
    def safe_stop(reason: str, exc: Exception = None, fatal: bool = True):
        msg = reason if exc is None else f"{reason}: {type(exc).__name__}: {exc}"
        raise RuntimeError(msg)
    globals()["safe_stop"] = safe_stop

# ----------------------------
# Last-known LTP cache (exit fallback)
# ----------------------------
_last_ltp = globals().get("_last_ltp")
if not isinstance(_last_ltp, dict):
    _last_ltp = {"CE": None, "PE": None}
globals()["_last_ltp"] = _last_ltp

def _update_last_ltp(leg: str, ltp: float):
    try:
        leg = str(leg).upper()
        if leg in ("CE", "PE"):
            v = float(ltp)
            if v > 0 and math.isfinite(v):
                _last_ltp[leg] = v
    except:
        if FAIL_FAST: raise
        pass

# ----------------------------
# Helpers: marketable prices
# ----------------------------
def get_marketable_entry(leg: str) -> Optional[float]:
    st = global_state.get(str(leg).upper()) or {}
    ask = st.get("best_ask")
    ltp = st.get("ltp")
    try:
        if ask is not None and float(ask) > 0:
            return float(ask)
    except:
        if FAIL_FAST: raise
        pass
    try:
        if ltp is not None and float(ltp) > 0:
            return float(ltp)
    except:
        if FAIL_FAST: raise
        pass
    v = _last_ltp.get(str(leg).upper())
    return float(v) if v is not None else None

def get_marketable_exit(leg: str) -> Optional[float]:
    st = global_state.get(str(leg).upper()) or {}
    bid = st.get("best_bid")
    ltp = st.get("ltp")
    try:
        if bid is not None and float(bid) > 0:
            return float(bid)
    except:
        if FAIL_FAST: raise
        pass
    try:
        if ltp is not None and float(ltp) > 0:
            return float(ltp)
    except:
        if FAIL_FAST: raise
        pass
    v = _last_ltp.get(str(leg).upper())
    return float(v) if v is not None else None

globals()["get_marketable_entry"] = get_marketable_entry
globals()["get_marketable_exit"] = get_marketable_exit

# ----------------------------
# Trades CSV
# ----------------------------
def _ensure_trades_header():
    try:
        if os.path.exists(TRADES_CSV) and os.path.getsize(TRADES_CSV) > 0:
            return
        with open(TRADES_CSV, "w", newline="") as f:
            csv.writer(f).writerow(["dt", "layer", "leg", "side", "price", "qty", "pnl_net", "reason", "run_id"])
    except:
        if FAIL_FAST: raise
        pass

_ensure_trades_header()

def log_trade(layer, leg, side, price, qty, pnl_net="", reason=""):
    dt = now_ist().strftime("%Y-%m-%d %H:%M:%S")
    try:
        with open(TRADES_CSV, "a", newline="") as f:
            csv.writer(f).writerow([dt, layer, leg, side, float(price), int(qty), pnl_net, reason, RUN_ID])
    except:
        if FAIL_FAST: raise
        pass

globals()["log_trade"] = log_trade

# ----------------------------
# Order executor (Paper-safe)
# ----------------------------
def _tsym_to_tradingsymbol(tsym: str) -> str:
    return str(tsym).split(":", 1)[-1]

def place_order_live(leg: str, side: str, qty: int) -> Optional[str]:
    # ✅ Paper mode: never touch broker
    if PAPER_TRADING or DEMO_MODE:
        return "PAPER"
    try:
        if kite is None:
            raise RuntimeError("kite not initialized")
        tsym = CE_TSYM if str(leg).upper() == "CE" else PE_TSYM
        tradingsymbol = _tsym_to_tradingsymbol(tsym)
        tx = "BUY" if str(side).upper() == "BUY" else "SELL"
        oid = kite.place_order(
            variety=kite.VARIETY_REGULAR,
            exchange=kite.EXCHANGE_NFO,
            tradingsymbol=tradingsymbol,
            transaction_type=tx,
            quantity=int(qty),
            order_type=kite.ORDER_TYPE_MARKET,
            product=kite.PRODUCT_MIS,
            validity=kite.VALIDITY_DAY
        )
        return str(oid)
    except Exception as e:
        tg_throttled("order_err", f"❌ ORDER ERROR {leg} {side}: {type(e).__name__}: {e}", 30)
        return None

def _place_or_revert(ok_if_paper: bool, leg: str, side: str, qty: int,
                     on_fail_revert_fn=None, fail_tag="order_fail"):
    oid = place_order_live(leg, side, qty)

    # ✅ Paper: always OK
    if (PAPER_TRADING or DEMO_MODE) and ok_if_paper:
        return True, oid

    # Live mode failure => revert + safe-stop (but you said you won't run live)
    if oid is None:
        try:
            if callable(on_fail_revert_fn):
                on_fail_revert_fn()
        except:
            if FAIL_FAST: raise
            pass
        tg_throttled(fail_tag, f"🛑 LIVE ORDER FAILED => SAFE-STOP ({leg} {side} qty={qty})", 30)
        safe_stop(f"Live order failed: {leg} {side} qty={qty}", fatal=True)
        return False, None

    return True, oid

globals()["place_order_live"] = place_order_live
globals()["_place_or_revert"] = _place_or_revert

# ============================================================
#  ✅ Uses Block-2 ATR7 points: global_state[leg]["atr"]
# ============================================================

import datetime, math
from dataclasses import dataclass
from typing import Optional, Dict, Any

IST = globals().get("IST") or datetime.timezone(datetime.timedelta(hours=5, minutes=30))

global_state = globals().get("global_state", {}) or {}
global_summary = globals().get("global_summary", {"GL": 0.0, "GB": 0.0, "N19": 0.0}) or {"GL": 0.0, "GB": 0.0, "N19": 0.0}
global_summary.setdefault("GL", 0.0)
globals()["global_summary"] = global_summary

tg = globals().get("tg", lambda *_a, **_k: None)
diag = globals().get("diag", lambda *_a, **_k: None)
tg_throttled = globals().get("tg_throttled", lambda *_a, **_k: None)

SLIPPAGE_PER_SIDE = float(globals().get("SLIPPAGE_PER_SIDE", 0.18) or 0.18)
BROKERAGE_PER_ORDER = float(globals().get("BROKERAGE_PER_ORDER", 20) or 20)
MAX_SPREAD = float(globals().get("MAX_SPREAD", 1e9))
QTY = int(globals().get("QTY", 0) or 0)
ALLOW_CONCURRENT_LAYERS = bool(globals().get("ALLOW_CONCURRENT_LAYERS", True))

_as_ist_dt = globals().get("_as_ist_dt") or globals().get("as_ist_dt")
now_ist = globals().get("now_ist") or (lambda: datetime.datetime.now(IST))

def _emit(strategy: str, event: str, leg: Optional[str] = None,
          price: Optional[float] = None, reason: str = "", meta: Optional[dict] = None):
    fn = globals().get("emit_signal")
    if callable(fn):
        try:
            fn(strategy=strategy, event=event, leg=leg, price=price, reason=reason, meta=(meta or {}))
        except:
            if FAIL_FAST: raise
            pass

class GannLevelsGL:
    @staticmethod
    def calculate_buy_levels(open_price: float) -> Dict[str, Optional[float]]:
        try:
            op = float(open_price)
        except:
            return {"ENTRY": None, "SL": None, "T1": None, "T2": None, "T3": None, "T4": None, "T5": None, "T6": None, "T7": None, "T8": None}
        if op <= 0:
            return {"ENTRY": None, "SL": None, "T1": None, "T2": None, "T3": None, "T4": None}

        r = math.sqrt(op)
        return {
            "SL": ((r + 0.0625) ** 2),
            "ENTRY": ((r + 0.125) ** 2),
            "T1": ((r + 0.25) ** 2),
            "T2": ((r + 0.50) ** 2),
            "T3": ((r + 0.75) ** 2),
            "T4": ((r + 1.00) ** 2),
            "T5": ((r + 1.25) ** 2),
            "T6": ((r + 1.50) ** 2),
            "T7": ((r + 1.75) ** 2),
            "T8": ((r + 2.00) ** 2)}

@dataclass

class GLTradeLegState:
    in_trade: bool = False
    entry_price: Optional[float] = None
    entry_ts: Optional[datetime.datetime] = None

    sl: Optional[float] = None

    # trailing (GL)
    trailing_sl: Optional[float] = None
    trail_active: bool = False
    trail_activated_ts: Optional[datetime.datetime] = None
    entry_level: Optional[float] = None
    t1: Optional[float] = None
    t2: Optional[float] = None
    t3: Optional[float] = None
    t4: Optional[float] = None
    t5: Optional[float] = None
    t6: Optional[float] = None
    t7: Optional[float] = None
    t8: Optional[float] = None

    qty_initial: int = 0
    qty_remaining: int = 0

    # milestone flags
    t1_hit: bool = False
    t2_hit: bool = False
    t3_hit: bool = False
    t4_hit: bool = False
    t5_hit: bool = False
    t6_hit: bool = False
    t7_hit: bool = False
    t8_hit: bool = False

    realized_pnl: float = 0.0

    def reset(self):
        self.__init__()

@dataclass
class GLDayState:
    date: Optional[datetime.date] = None
    trades_today: int = 0
    ce_traded: bool = False
    pe_traded: bool = False
    max_trades_per_day: int = 3  # maximum 3 trades/day (CE+PE combined)

    disabled: bool = False
    disabled_reason: Optional[str] = None
    max_loss: float = 950.0  # GL-only kill-switch
    day_pnl: float = 0.0

    def reset_for_day(self, date: datetime.date):
        if self.date != date:
            self.date = date
            self.trades_today = 0
            self.ce_traded = False
            self.pe_traded = False
            self.disabled = False
            self.disabled_reason = None
            self.day_pnl = 0.0

class GLStrategy:
    def __init__(self):
        self.gann = GannLevelsGL()

        # per-leg trades (CE + PE concurrently allowed)
        self.trades = {"CE": GLTradeLegState(), "PE": GLTradeLegState()}
        self.day_state = GLDayState()

        # time windows
        self.entry_start = datetime.time(9, 15, 0)
        self.entry_end   = datetime.time(14, 30, 0)
        self.first_candle_end = datetime.time(9, 30, 0)
        self.square_off_time = datetime.time(15, 15, 0)

        # lunch no-trade window for GL entries only
        self.lunch_start = datetime.time(11, 45, 0)
        self.lunch_end   = datetime.time(13, 0, 0)

        self.last_slow_tf_candle = {"CE": None, "PE": None}

        # pending orders per leg (GL only)
        self.pending = {"CE": None, "PE": None}
        self.pending_ttl = datetime.timedelta(minutes=45)
        self.pending_cancel_pct = 0.0025  # 0.25%

        # active params (auto selected at startup)
        self.imb_min = float(globals().get("IMB_MIN", 1.70) or 1.70)
        self.max_spread = float(globals().get("MAX_SPREAD", 2.29) or 2.29)
        self.sl_atr_mult = float(globals().get("GL_SL_ATR_MULT", 2.21) or 2.21)
        self.min_sl_points = float(globals().get("GL_MIN_SL_POINTS", 12.91) or 12.91)
        self.trail_start_tag = str(globals().get("GL_TRAIL_START", "T4")).upper()

    def reset_day(self, date: datetime.date):
        for leg in ("CE", "PE"):
            self.trades[leg].reset()
            self.pending[leg] = None
        self.day_state.reset_for_day(date)
        self.last_slow_tf_candle = {"CE": None, "PE": None}

    def leg_in_trade(self, leg: str) -> bool:
        leg = str(leg).upper()
        return bool(self.trades.get(leg, GLTradeLegState()).in_trade)

    @property
    def any_in_trade(self) -> bool:
        return bool(self.trades["CE"].in_trade or self.trades["PE"].in_trade)

    def update_slow_tf_candle(self, leg: str, candle: dict):
        try:
            leg = str(leg).upper()
            if leg in ("CE", "PE") and isinstance(candle, dict):
                self.last_slow_tf_candle[leg] = {
                    "time": candle.get("time"),
                    "open": float(candle.get("open", 0) or 0),
                    "high": float(candle.get("high", 0) or 0),
                    "low": float(candle.get("low", 0) or 0),
                    "close": float(candle.get("close", 0) or 0)}
        except:
            if FAIL_FAST: raise
            pass

    def _within_entry_window(self, ts: datetime.datetime) -> bool:
        t = ts.time()
        if t < self.entry_start or t > self.entry_end:
            return False
        if self.lunch_start <= t < self.lunch_end:
            return False
        return True

    def _daily_limit_ok(self) -> bool:
        try:
            return int(self.day_state.trades_today) < int(self.day_state.max_trades_per_day)
        except:
            return True

    def _leg_reentry_ok(self, leg: str, ts: datetime.datetime) -> bool:
        # Before 09:30 re-entry allowed. After 09:30, block if that leg traded today.
        if ts.time() < self.first_candle_end:
            return True
        if leg == "CE":
            return not bool(getattr(self.day_state, "ce_traded", False))
        return not bool(getattr(self.day_state, "pe_traded", False))

    def _check_spread(self, st: dict) -> bool:
        spr = st.get("spread")
        try:
            if spr is None:
                return True
            return float(spr) <= float(self.max_spread)
        except:
            return True

    def _filters_and_sl(self, ts: datetime.datetime, leg: str, st: dict, entry_level: float, ltp: float):
        """
        Returns (ok, details). details contains 'sl' when computable.
        Used by both DIRECT entry and PENDING execution checks.
        """
        out = {"ok": True, "fail": []}
        vwap = st.get("vwap")
        if vwap is None:
            vwap = st.get("vwap_1m")
        try:
            vwap = float(vwap) if vwap is not None else None
        except:
            vwap = None
        if vwap is None:
            out["ok"] = False
            out["fail"].append("NO_VWAP")
        else:
            if float(vwap) >= float(entry_level):
                out["ok"] = False
                out["fail"].append("VWAP>=ENTRY")
        out["vwap"] = vwap

        # IMB ratio
        imb = st.get("imb_ratio")
        try:
            imb = float(imb) if imb is not None else None
        except:
            imb = None
        if imb is None or float(imb) < float(self.imb_min):
            out["ok"] = False
            out["fail"].append("IMB_LOW")
        out["imb_ratio"] = imb

        # spread check
        if not self._check_spread(st):
            out["ok"] = False
            out["fail"].append("SPREAD_WIDE")

        # ATR ready
        atr = st.get("atr")
        try:
            atr = float(atr) if atr is not None else None
        except:
            atr = None
        if atr is None or atr <= 0:
            out["ok"] = False
            out["fail"].append("ATR_MISSING")
        out["atr"] = atr

        # initial SL (VWAP_1m - buffer)
        vwap1 = st.get("vwap_1m")
        try:
            vwap1 = float(vwap1) if vwap1 is not None else None
        except:
            vwap1 = None
        out["vwap_1m"] = vwap1

        sl = None
        if (vwap1 is not None) and (atr is not None) and (atr > 0):
            atr_buf = max(float(self.min_sl_points), float(self.sl_atr_mult) * float(atr))
            sl = float(vwap1) - float(atr_buf)
            out["sl"] = sl
            out["atr_buffer"] = atr_buf
        else:
            out["ok"] = False
            out["fail"].append("SL_NOT_READY")

        # SL must be valid vs entry price (ltp + slippage)
        if sl is not None:
            entry_est = float(ltp) + float(SLIPPAGE_PER_SIDE)
            if float(sl) >= float(entry_est):
                out["ok"] = False
                out["fail"].append("SL_INVALID>=ENTRY")

        return bool(out["ok"]), out

    def _pending_housekeeping(self, ts: datetime.datetime, leg: str, ltp: float):
        p = self.pending.get(leg)
        if not p:
            return
        try:
            if ts >= p.get("expiry_ts"):
                self.pending[leg] = None
                return
            cancel_below = float(p.get("cancel_below") or 0.0)
            if cancel_below > 0 and float(ltp) <= cancel_below:
                self.pending[leg] = None
                return
        except:
            return

    def check_entry(self, ts: datetime.datetime, leg: str, prev_ltp, ltp):
        leg = str(leg).upper()
        if leg not in ("CE", "PE"):
            return None

        # GL disabled => no new entries
        if self.day_state.disabled:
            return None

        # pending housekeeping even outside entry window
        self._pending_housekeeping(ts, leg, float(ltp))

        # block if already in trade on this leg
        if self.trades[leg].in_trade:
            return None

        # daily trade limit (CE+PE combined)
        if not self._daily_limit_ok():
            return None

        # re-entry rule per leg (after 09:30)
        if not self._leg_reentry_ok(leg, ts):
            return None

        # pending execution path
        p = self.pending.get(leg)
        if p and self._within_entry_window(ts):
            try:
                entry_level = float(p.get("entry_level") or 0.0)
            except:
                entry_level = 0.0
            if entry_level > 0 and float(ltp) >= entry_level:
                st = global_state.get(leg, {}) or {}
                ok, details = self._filters_and_sl(ts, leg, st, float(entry_level), float(ltp))
                if ok:
                    g = p.get("gann")
                    if not isinstance(g, dict) or g.get("ENTRY") is None:
                        true_open = st.get("open")
                        try:
                            g = self.gann.calculate_buy_levels(float(true_open))
                        except:
                            return None
                    return {"leg": leg, "gann": g, "entry_level": float(entry_level), "sl": float(details.get("sl")), "pending": True}
            return None

        # entry time / lunch block
        if not self._within_entry_window(ts):
            return None

        # fresh cross only
        if prev_ltp is None:
            return None

        st = global_state.get(leg, {}) or {}
        true_open = st.get("open")
        try:
            if true_open is None or float(true_open) <= 0:
                return None
        except:
            return None

        g = self.gann.calculate_buy_levels(float(true_open))
        entry_level = g.get("ENTRY")
        if entry_level is None:
            return None
        entry_level = float(entry_level)

        try:
            if float(prev_ltp) >= float(entry_level):
                return None
            if float(ltp) < float(entry_level):
                return None
        except:
            return None

        ok, details = self._filters_and_sl(ts, leg, st, float(entry_level), float(ltp))
        if ok:
            return {"leg": leg, "gann": g, "entry_level": float(entry_level), "sl": float(details.get("sl")), "pending": False}

        # create pending (45m) if cross happened but filters fail or SL invalid
        try:
            expiry = ts + self.pending_ttl
            cancel_below = float(entry_level) * (1.0 - float(self.pending_cancel_pct))
            self.pending[leg] = {
                "created_ts": ts,
                "expiry_ts": expiry,
                "entry_level": float(entry_level),
                "cancel_below": float(cancel_below),
                "gann": g,
                "fail": list(details.get("fail") or [])}
        except:
            if FAIL_FAST: raise
            pass
        return None

    def execute_entry(self, ts: datetime.datetime, leg: str, option_price: float, qty: int, entry_signal: dict):
        leg = str(leg).upper()
        tr = self.trades[leg]
        g = entry_signal["gann"]

        tr.in_trade = True
        tr.entry_price = float(option_price)
        tr.entry_ts = ts

        tr.entry_level = float(g["ENTRY"])
        tr.t1 = float(g["T1"]); tr.t2 = float(g["T2"]); tr.t3 = float(g["T3"]); tr.t4 = float(g["T4"])
        tr.t5 = float(g.get("T5")); tr.t6 = float(g.get("T6")); tr.t7 = float(g.get("T7")); tr.t8 = float(g.get("T8"))

        tr.qty_initial = int(qty)
        tr.qty_remaining = int(qty)

        tr.sl = float(entry_signal["sl"])
        tr.trailing_sl = None
        tr.trail_active = False
        tr.trail_activated_ts = None

        if leg == "CE":
            self.day_state.ce_traded = True
        else:
            self.day_state.pe_traded = True
        self.day_state.trades_today += 1

        # clear pending once we enter
        self.pending[leg] = None

    def check_exit(self, ts: datetime.datetime, leg: str, ltp: float):
        leg = str(leg).upper()
        tr = self.trades.get(leg)
        if not tr or not tr.in_trade:
            return None

        if ts.time() >= self.square_off_time:
            return {"reason": "SQUARE_OFF", "qty": tr.qty_remaining}

        try:
            ltp = float(ltp)
        except:
            return None

        # HARD SL
        if tr.sl is not None and float(ltp) <= float(tr.sl):
            return {"reason": "SL_HIT", "qty": tr.qty_remaining}

        # TARGET T4
        if tr.t4 is not None and float(ltp) >= float(tr.t4):
            return {"reason": "TARGET_T4", "qty": tr.qty_remaining}

        # activate trailing at milestone (T4 for Nifty, T3 for BN Monthly)
        trail_start = tr.t3 if self.trail_start_tag == "T3" else tr.t4
        if (not tr.trail_active) and (trail_start is not None) and float(ltp) >= float(trail_start):
            tr.trail_active = True
            tr.trailing_sl = float(trail_start)
            tr.trail_activated_ts = ts

        if tr.trail_active:
            st = global_state.get(leg, {}) or {}
            atr = st.get("atr")
            vwap = st.get("vwap") or st.get("vwap_1m")
            try:
                atr = float(atr) if atr is not None else None
                vwap = float(vwap) if vwap is not None else None
            except:
                atr, vwap = None, None

            if (atr is not None and atr > 0) and (vwap is not None and vwap > 0):
                buf = max(float(self.min_sl_points), float(self.sl_atr_mult) * float(atr))
                new_trail = float(vwap) - float(buf)
                tr.trailing_sl = max(float(tr.trailing_sl), float(new_trail))

            # avoid instant trail exit on activation tick
            if tr.trail_activated_ts is not None and ts == tr.trail_activated_ts:
                return None

            if tr.trailing_sl is not None and float(ltp) <= float(tr.trailing_sl):
                return {"reason": "TRAIL_SL", "qty": tr.qty_remaining}

        return None

# instantiate GL strategy
gl_strategy = GLStrategy()
globals()["gl_strategy"] = gl_strategy

def compute_buy_targets(true_open: float, decimals: int = 2):
    try:
        lv = gann_buy_levels(float(true_open), decimals=int(decimals))
        buy = lv.get("ENTRY") if isinstance(lv, dict) else None
        if buy is None and isinstance(lv, dict):
            buy = lv.get("Entry")
        out = {
            "BUY": buy,
            "ENTRY": buy,
            "Entry": buy,
            "SL":  lv.get("SL") if isinstance(lv, dict) else None,
            "T1":  lv.get("T1") if isinstance(lv, dict) else None,
            "T2":  lv.get("T2") if isinstance(lv, dict) else None,
            "T3":  lv.get("T3") if isinstance(lv, dict) else None,
            "T4":  lv.get("T4") if isinstance(lv, dict) else None}
        return buy, out
    except Exception as e:
        return None, {"BUY": None, "ENTRY": None, "Entry": None, "SL": None, "T1": None, "T2": None, "T3": None, "T4": None}

# ------------------------------------------------------------
# GB STATE (FIX) — required by GBStrategy / gb_on_tick
# ------------------------------------------------------------
@dataclass
class GBTradeState:
    in_trade: bool = False
    leg: Optional[str] = None
    entry_price: Optional[float] = None
    entry_ts: Optional[datetime.datetime] = None

    sl: Optional[float] = None
    t1: Optional[float] = None
    t2: Optional[float] = None
    t3: Optional[float] = None

    qty_initial: int = 0
    qty_remaining: int = 0

    t2_hit: bool = False  # trailing activates after T2

    def reset(self):
        self.__init__()

@dataclass
class GBDayState:
    date: Optional[datetime.date] = None
    ce_traded: bool = False
    pe_traded: bool = False
    levels: Dict[str, Any] = None

    def __post_init__(self):
        if self.levels is None:
            self.levels = {"CE": None, "PE": None}

    def reset_for_day(self, date: datetime.date):
        if self.date != date:
            self.date = date
            self.ce_traded = False
            self.pe_traded = False
            self.levels = {"CE": None, "PE": None}

class GBStrategy:
    def __init__(self):
        self.trade = GBTradeState()
        self.day = GBDayState(levels={"CE": None, "PE": None})
        self.entry_start = datetime.time(9, 15, 0)
        self.entry_end   = datetime.time(14, 30, 0)
        self.square_off_time = datetime.time(15, 15, 0)

        # active params (auto selected at startup)
        self.imb_min = float(globals().get("IMB_MIN", 1.70) or 1.70)
        self.max_spread = float(globals().get("MAX_SPREAD", 2.29) or 2.29)
        self.sl_atr_mult = float(globals().get("GL_SL_ATR_MULT", 2.21) or 2.21)   # from table
        self.min_sl_points = float(globals().get("GL_MIN_SL_POINTS", 12.91) or 12.91)
        self.t2_floor_offset = float(globals().get("GB_T2_FLOOR_OFFSET", 5.0) or 5.0)
        self.initial_sl_mode = str(globals().get("GB_INITIAL_SL_MODE", "FIXED20")).upper()

    def reset_day(self, date: datetime.date):
        self.trade.reset()
        self.day.reset_for_day(date)

    def _within_entry_window(self, ts: datetime.datetime) -> bool:
        t = ts.time()
        return (t >= self.entry_start) and (t <= self.entry_end)

    def _daily_limit_ok(self, leg: str) -> bool:
        # max 1 trade per leg per day
        leg = str(leg).upper()
        if leg == "CE":
            return not bool(self.day.ce_traded)
        return not bool(self.day.pe_traded)

    def _check_spread(self, st: dict) -> bool:
        spr = st.get("spread")
        try:
            if spr is None:
                return True
            return float(spr) <= float(self.max_spread)
        except:
            return True

    def _ensure_levels(self, leg: str):
        lv = (self.day.levels or {}).get(leg)
        if isinstance(lv, dict) and lv.get("BUY") is not None:
            return lv

        st = global_state.get(leg, {}) or {}
        true_open = st.get("open")
        try:
            if true_open is None or float(true_open) <= 0:
                return None
        except:
            return None

        _, out = compute_buy_targets(float(true_open))
        self.day.levels[leg] = out
        return out

    def check_entry(self, ts: datetime.datetime, leg: str, prev_ltp, ltp):
        if self.trade.in_trade:
            return None

        leg = str(leg).upper()
        if leg not in ("CE", "PE"):
            return None

        if not self._within_entry_window(ts):
            return None

        if not self._daily_limit_ok(leg):
            return None

        lv = self._ensure_levels(leg)
        if not lv:
            return None

        st = global_state.get(leg, {}) or {}

        # ATR ready
        atr = st.get("atr")
        try:
            atr = float(atr) if atr is not None else None
        except:
            atr = None
        if atr is None or atr <= 0:
            return None

        # IMB ratio
        imb = st.get("imb_ratio")
        try:
            imb = float(imb) if imb is not None else None
        except:
            imb = None
        if imb is None or imb < float(self.imb_min):
            return None

        # spread
        if not self._check_spread(st):
            return None
        # ( removed) 15>0 filter removed

        # VWAP filter (option 1m VWAP)
        vw = st.get("vwap_1m")
        try:
            if vw is None or float(vw) <= 0:
                return None
            if float(ltp) <= float(vw):
                return None
        except:
            return None

        # fresh cross above BUY: prev < buy AND current >= buy
        buy = float(lv["BUY"])
        try:
            if prev_ltp is None:
                return None
            if float(prev_ltp) >= buy:
                return None
            if float(ltp) < buy:
                return None
        except:
            return None

        return {"leg": leg, "levels": lv, "atr": atr}

    def execute_entry(self, ts: datetime.datetime, leg: str, entry_px: float, qty: int, sig: dict):
        lv = sig["levels"]
        atr = float(sig.get("atr") or 0.0)

        self.trade.in_trade = True
        self.trade.leg = leg
        self.trade.entry_price = float(entry_px)
        self.trade.entry_ts = ts
        self.trade.qty_initial = int(qty)
        self.trade.qty_remaining = int(qty)

        # initial SL
        if self.initial_sl_mode == "ATR":
            dist = max(float(self.min_sl_points), float(self.sl_atr_mult) * float(atr))
            self.trade.sl = float(entry_px) - float(dist)
        else:
            self.trade.sl = float(entry_px) - 20.0

        self.trade.t1 = float(lv["T1"])
        self.trade.t2 = float(lv["T2"])
        self.trade.t3 = float(lv["T3"])
        self.trade.t2_hit = False

        if leg == "CE":
            self.day.ce_traded = True
        else:
            self.day.pe_traded = True

    def update_trailing_and_check_exit(self, ts: datetime.datetime, ltp: float):
        if not self.trade.in_trade:
            return None

        if ts.time() >= self.square_off_time:
            return {"reason": "SQUARE_OFF", "qty": int(self.trade.qty_remaining)}

        leg = self.trade.leg
        st = global_state.get(leg, {}) or {}

        # target at T3
        try:
            if self.trade.t3 is not None and float(ltp) >= float(self.trade.t3):
                return {"reason": "TARGET_T3", "qty": int(self.trade.qty_remaining)}
        except:
            if FAIL_FAST: raise
            pass

        # T2 hit -> lock floor
        try:
            if (not self.trade.t2_hit) and self.trade.t2 is not None and float(ltp) >= float(self.trade.t2):
                self.trade.t2_hit = True
                floor = float(self.trade.t2) - float(self.t2_floor_offset)
                self.trade.sl = max(float(self.trade.sl or -1e9), floor)
        except:
            if FAIL_FAST: raise
            pass

        # ATR trailing after T2
        if self.trade.t2_hit:
            atr7 = st.get("atr")
            try:
                atr7 = float(atr7) if atr7 is not None else 0.0
            except:
                atr7 = 0.0

            floor = (float(self.trade.t2) - float(self.t2_floor_offset)) if self.trade.t2 is not None else float(self.trade.sl or -1e9)
            trail_val = float(ltp) - (1.5 * float(atr7))
            self.trade.sl = max(float(self.trade.sl or -1e9), trail_val, floor)

            if self.trade.sl is not None and float(ltp) <= float(self.trade.sl):
                return {"reason": "TRAIL_SL", "qty": int(self.trade.qty_remaining)}

        # pre-T2 SL
        try:
            if (not self.trade.t2_hit) and self.trade.sl is not None and float(ltp) <= float(self.trade.sl):
                return {"reason": "SL_HIT", "qty": int(self.trade.qty_remaining)}
        except:
            if FAIL_FAST: raise
            pass

        return None

# instantiate GB strategy
gb_strategy = GBStrategy()
globals()["gb_strategy"] = gb_strategy

class GBStateCompat:
    @property
    def in_trade(self): return gb_strategy.trade.in_trade
    @property
    def leg(self): return gb_strategy.trade.leg
    @property
    def entry(self): return gb_strategy.trade.entry_price
    @property
    def sl(self): return gb_strategy.trade.sl
    @property
    def qty(self): return gb_strategy.trade.qty_remaining
    @property
    def t1(self): return gb_strategy.trade.t1
    @property
    def t2(self): return gb_strategy.trade.t2
    @property
    def t3(self): return gb_strategy.trade.t3

# ----------------------------
# GL COMPAT STATE + TICK HANDLER (MANDATORY)
# Ensures deterministic wiring (no cell-order dependency).
# ----------------------------
class GLStateCompat:
    """Compatibility view for UI/engine checks (single object)."""
    def __init__(self, gls):
        self._gls = gls

    @property
    def in_trade(self) -> bool:
        try:
            return bool(self._gls.any_in_trade())
        except:
            return False

    @property
    def leg(self):
        try:
            legs = [l for l in ("CE", "PE") if self._gls.leg_in_trade(l)]
            if len(legs) == 1:
                return legs[0]
            if len(legs) >= 2:
                return "BOTH"
            return None
        except:
            return None

    @property
    def entry(self):
        try:
            lg = self.leg
            if lg in ("CE", "PE"):
                return self._gls.trades[lg].entry_price
            return None
        except:
            return None

    @property
    def sl(self):
        try:
            lg = self.leg
            if lg in ("CE", "PE"):
                tr = self._gls.trades[lg]
                if tr.trail_active and tr.trailing_sl is not None:
                    return tr.trailing_sl
                return tr.sl
            return None
        except:
            return None

    @property
    def execution_preference(self):
        return None

gl_state = GLStateCompat(globals().get("gl_strategy"))
globals()["gl_state"] = gl_state

def gl_on_tick(ts: datetime.datetime, leg: str, *args):
    """GL tick handler.
    Supports:
      gl_on_tick(ts, leg, ltp)
      gl_on_tick(ts, leg, prev_ltp, ltp)
    """
    try:
        ts = (_as_ist_dt(ts) if callable(globals().get("_as_ist_dt")) else ts) or ts
        if not isinstance(ts, datetime.datetime):
            return
    except:
        return

    leg = str(leg).upper()
    if leg not in ("CE", "PE"):
        return

    # args parse
    prev_ltp = None
    ltp = None
    if len(args) == 1:
        ltp = args[0]
    elif len(args) >= 2:
        prev_ltp, ltp = args[0], args[1]
    else:
        return

    try:
        ltp = float(ltp)
    except:
        return

    gls = globals().get("gl_strategy")
    if gls is None:
        return

    # day reset
    try:
        if gls.day_state.date != ts.date():
            gls.reset_day(ts.date())
    except:
        if FAIL_FAST: raise
        pass

    get_entry = globals().get("get_marketable_entry")
    get_exit  = globals().get("get_marketable_exit")
    placer    = globals().get("_place_or_revert")
    logger    = globals().get("log_trade")
    tg_fn     = globals().get("tg") or (lambda *_: None)
    diag_fn   = globals().get("diag") or (lambda *_a, **_k: None)
    emit_fn   = globals().get("_emit") or (lambda *_a, **_k: None)

    SLIPPAGE = float(globals().get("SLIPPAGE_PER_SIDE", 0.18) or 0.18)
    BROKER   = float(globals().get("BROKERAGE_PER_ORDER", 20) or 20)

    # EXIT
    try:
        if gls.leg_in_trade(leg):
            ex = gls.check_exit(ts, leg, float(ltp))
            if ex:
                exit_px = get_exit(leg) if callable(get_exit) else None
                exit_px = float(exit_px) if exit_px else float(ltp)
                qty = int(ex.get("qty") or 0)
                if qty <= 0:
                    return

                ok, _ = placer(True, leg, "SELL", qty, None, "gl_sell_fail") if callable(placer) else (True, None)
                if ok:
                    tr = gls.trades.get(leg)
                    entry_px = float(getattr(tr, "entry_price", 0.0) or 0.0)
                    pnl = (exit_px - entry_px) * qty
                    pnl -= SLIPPAGE * 2 * qty
                    pnl -= BROKER * 2

                    gs = globals().get("global_summary")
                    if isinstance(gs, dict):
                        gs["GL"] = float(gs.get("GL", 0.0)) + float(pnl)

                    reason = str(ex.get("reason", "EXIT"))
                    if callable(logger):
                        logger("GL", leg, "EXIT", exit_px, qty, pnl_net=pnl, reason=reason)

                    tg_fn(f"🟥 GL EXIT {leg} @ {exit_px:.2f} | NET ₹{pnl:+,.0f} | {reason}")
                    emit_fn("GL", "EXIT", leg, exit_px, reason, {"pnl": pnl})
                    diag_fn("gl_exit", leg=leg, exit=exit_px, reason=reason)

                    try:
                        gls.trades[leg].reset()
                    except:
                        if FAIL_FAST: raise
                        pass
            return
    except:
        if FAIL_FAST: raise
        pass

    # ENTRY: block if any other strategy is in trade (if concurrency disabled)
    if not bool(globals().get("ALLOW_CONCURRENT_LAYERS", True)):
        try:
            n19s = globals().get("n19_state")
            gbs = globals().get("gb_state")
            if (n19s and getattr(n19s, "in_trade", False)) or (gbs and getattr(gbs, "in_trade", False)):
                return
        except:
            if FAIL_FAST: raise
            pass

    sig = None
    try:
        sig = gls.check_entry(ts, leg, prev_ltp, ltp)
    except:
        sig = None

    if not sig:
        return

    # ✅ EXTRA FILTERS (requested): VWAP + RSI(3) for GL
    try:
        vw = _get_leg_vwap(leg)
        r3 = _get_leg_rsi3(leg)
        if vw is None or r3 is None:
            return
        if not (float(ltp) > float(vw) and float(r3) > 50.0):
            return
    except Exception:
        return

    # Ensure eligible strike at entry-time (candidate hot-swap)
    try:
        select_and_activate_candidate(leg, context="GL_entry_place")
    except:
        if FAIL_FAST: raise
        pass

    entry_px = get_entry(leg) if callable(get_entry) else None
    if entry_px is None:
        return
    try:
        entry_px = float(entry_px)
    except:
        return

    qty = int(globals().get("QTY", 0) or 0)
    if qty <= 0:
        return

    def _revert():
        try:
            gls.trades[leg].reset()
        except:
            if FAIL_FAST: raise
            pass

    ok, _ = placer(True, leg, "BUY", qty, _revert, "gl_buy_fail") if callable(placer) else (True, None)
    if not ok:
        return

    try:
        gls.execute_entry(ts, leg, entry_px, qty, sig)
    except:
        return

    # Display/log rounding only (internal levels keep full precision)
    g = sig.get("gann") or {}
    reason = f"GANN BUY={float(g.get('ENTRY') or entry_px):.2f} IMB>={float(globals().get('IMB_MIN', 0) or 0):.2f}"

    if callable(logger):
        logger("GL", leg, "ENTRY", entry_px + SLIPPAGE, qty, reason=reason)

    tg_fn(f"🟩 GL ENTRY {leg} @ {entry_px:.2f}")
    try:
        sl_now = float(gls.trades[leg].sl or 0.0)
        tg_fn(
            f"📍 ENTRY {float(g.get('ENTRY') or entry_px):.2f} | SL {sl_now:.2f} | "
            f"T1 {float(g.get('T1') or 0):.2f} T2 {float(g.get('T2') or 0):.2f} "
            f"T3 {float(g.get('T3') or 0):.2f} T4 {float(g.get('T4') or 0):.2f}"
        )
    except:
        if FAIL_FAST: raise
        pass

    emit_fn("GL", "ENTRY", leg, entry_px, reason, {"gann": g})
    diag_fn("gl_entry", leg=leg, entry=entry_px, sl=float(gls.trades[leg].sl or 0.0))

globals()["gl_on_tick"] = gl_on_tick

gb_state = GBStateCompat()
globals()["gb_state"] = gb_state

def gb_on_tick(ts: datetime.datetime, leg: str, *args):
    """
    Supports both call styles:
      gb_on_tick(ts, leg, ltp)
      gb_on_tick(ts, leg, prev_ltp, ltp)
    """
    try:
        ts = (_as_ist_dt(ts) if callable(_as_ist_dt) else ts) or ts
        if not isinstance(ts, datetime.datetime):
            return
    except:
        return

    leg = str(leg).upper()
    if leg not in ("CE", "PE"):
        return

    # args parse
    prev_ltp = None
    ltp = None
    if len(args) == 1:
        ltp = args[0]
    elif len(args) >= 2:
        prev_ltp, ltp = args[0], args[1]
    else:
        return

    try:
        ltp = float(ltp)
    except:
        return

    # day reset
    if gb_strategy.day.date != ts.date():
        gb_strategy.reset_day(ts.date())

    get_entry = globals().get("get_marketable_entry")
    get_exit  = globals().get("get_marketable_exit")
    placer    = globals().get("_place_or_revert")
    logger    = globals().get("log_trade")

    # EXIT
    if gb_strategy.trade.in_trade and gb_strategy.trade.leg == leg:
        ex = gb_strategy.update_trailing_and_check_exit(ts, ltp)
        if ex:
            exit_px = get_exit(leg) if callable(get_exit) else None
            exit_px = float(exit_px) if exit_px else float(ltp)
            qty = int(ex["qty"])

            ok, _ = placer(True, leg, "SELL", qty, None, "gb_sell_fail") if callable(placer) else (True, None)
            if ok:
                entry_px = float(gb_strategy.trade.entry_price or 0.0)
                pnl = (exit_px - entry_px) * qty
                pnl -= SLIPPAGE_PER_SIDE * 2 * qty
                pnl -= BROKERAGE_PER_ORDER * 2

                global_summary["GB"] = float(global_summary.get("GB", 0.0)) + float(pnl)

                reason = str(ex.get("reason", "EXIT"))
                if callable(logger):
                    logger("GB", leg, "EXIT", exit_px, qty, pnl_net=pnl, reason=reason)

                tg(f"🟥 GB EXIT {leg} @ {exit_px:.2f} | NET ₹{pnl:+,.0f} | {reason}")
                _emit("GB", "EXIT", leg, exit_px, reason, {"pnl": pnl})

                gb_strategy.trade.reset()
        return

    # ENTRY: block if any other strategy is in trade (if concurrency disabled)
    if not ALLOW_CONCURRENT_LAYERS:
        n19s = globals().get("n19_state")
        gls = globals().get("gl_state")
        if (n19s and getattr(n19s, "in_trade", False)) or (gls and getattr(gls, "in_trade", False)):
            return

    if gb_strategy.trade.in_trade:
        return

    sig = gb_strategy.check_entry(ts, leg, prev_ltp, ltp)
    if not sig:
        return

    # ✅ EXTRA FILTERS (requested): VWAP + RSI(3) cross + Avg IMB(20)*1.5 for GB
    try:
        vw = _get_leg_vwap(leg)
        r3 = _get_leg_rsi3(leg)
        r3_prev = None
        try:
            r3_prev = global_state[leg].get("rsi3_15m_prev")
        except Exception:
            r3_prev = None

        imb_cur  = _get_leg_imb15(leg)
        imb_avg20 = _get_leg_imb15_avg20(leg)

        if vw is None or r3 is None or imb_cur is None or imb_avg20 is None:
            return

        # RSI(3) crossed above 66 (fresh momentum)
        if r3_prev is None:
            return
        crossed = (float(r3_prev) < 66.0 and float(r3) >= 66.0)

        if not crossed:
            return

        if not (float(ltp) > float(vw)):
            return

        if not (float(imb_cur) > float(imb_avg20) * 1.5):
            return
    except Exception:
        return

    # Ensure eligible strike at entry-time (candidate hot-swap)
    try:
        select_and_activate_candidate(leg, context="GB_entry_place")
    except:
        if FAIL_FAST: raise
        pass

    entry_px = get_entry(leg) if callable(get_entry) else None
    if entry_px is None:
        return
    entry_px = float(entry_px)

    qty = int(QTY) if int(QTY) > 0 else 0
    if qty <= 0:
        return

    def _revert():
        gb_strategy.trade.reset()

    ok, _ = placer(True, leg, "BUY", qty, _revert, "gb_buy_fail") if callable(placer) else (True, None)
    if not ok:
        return

    gb_strategy.execute_entry(ts, leg, entry_px, qty, sig)

    lv = sig["levels"]
    reason = f"15>0 + VWAP + GANN BUY={float(lv['BUY']):.2f}"

    if callable(logger):
        logger("GB", leg, "ENTRY", entry_px + SLIPPAGE_PER_SIDE, qty, reason=reason)

    tg(f"🟩 GB ENTRY {leg} @ {entry_px:.2f}")
    sl_now = float(gb_strategy.trade.sl or (entry_px - 20.0))
    tg(f"📍 BUY {float(lv['BUY']):.2f} | SL {sl_now:.2f} | T1 {float(lv['T1']):.2f} T2 {float(lv['T2']):.2f} T3 {float(lv['T3']):.2f}")
    _emit("GB", "ENTRY", leg, entry_px, reason, {"levels": lv})
    diag("gb_entry", leg=leg, entry=entry_px, sl=sl_now)

def gb_exit(ts: datetime.datetime, reason: str = "FORCED"):
    if not gb_strategy.trade.in_trade:
        return
    try:
        ts = (_as_ist_dt(ts) if callable(_as_ist_dt) else ts) or ts
    except:
        ts = now_ist()

    leg = gb_strategy.trade.leg
    get_exit = globals().get("get_marketable_exit")
    placer   = globals().get("_place_or_revert")
    logger   = globals().get("log_trade")

    exit_px = get_exit(leg) if callable(get_exit) else None
    exit_px = float(exit_px) if exit_px else float(gb_strategy.trade.entry_price or 0.0)

    qty = int(gb_strategy.trade.qty_remaining)
    ok, _ = placer(True, leg, "SELL", qty, None, "gb_force_exit") if callable(placer) else (True, None)
    if not ok:
        return

    entry_px = float(gb_strategy.trade.entry_price or 0.0)
    pnl = (exit_px - entry_px) * qty
    pnl -= SLIPPAGE_PER_SIDE * 2 * qty
    pnl -= BROKERAGE_PER_ORDER * 2

    global_summary["GB"] = float(global_summary.get("GB", 0.0)) + float(pnl)
    if callable(logger):
        logger("GB", leg, "EXIT", exit_px, qty, pnl_net=pnl, reason=reason)

    tg(f"🟥 GB FORCE EXIT {leg} @ {exit_px:.2f} | NET ₹{pnl:+,.0f} | {reason}")
    _emit("GB", "EXIT", leg, exit_px, reason, {"pnl": pnl})
    gb_strategy.trade.reset()

globals()["gb_on_tick"] = gb_on_tick
globals()["gb_exit"] = gb_exit

print("✅ GB loaded (ATM-only, CE+PE; VWAP915 + ATR7 trail)")

# ============================================================
# Block-3 done
# ============================================================
log("✅ Block-3 ready (GL/GB/N19) — safe for PAPER_TRADING=True")

# ============================================================

# ============================================================

# ============================================================

def _get_leg_state(leg: str):
    leg = str(leg).upper()
    return globals().get("ce_state") if leg == "CE" else globals().get("pe_state")

def _get_leg_vwap(leg: str):
    """Preferred VWAP is 15m (chart-style). Falls back to tick VWAP then 1m VWAP."""
    st = _get_leg_state(leg) or {}
    vw = st.get("vwap_15m")
    if vw is None:
        vw = st.get("vwap")
    if vw is None:
        vw = st.get("vwap_1m")
    return float(vw) if vw is not None else None

def _get_leg_rsi3(leg: str):
    """Preferred RSI(3) is 15m; falls back to 1m if 15m not ready."""
    st = _get_leg_state(leg) or {}
    v = st.get("rsi3_15m")
    if v is None:
        v = st.get("rsi3_1m")
    return float(v) if v is not None else None

# SHARED UTILITIES — MARKETABLE ENTRY/EXIT + SIGNAL EMIT + WRAPPER GUARDS
# ============================================================

import datetime, math
from dataclasses import dataclass
from typing import Optional, Dict, Any

IST = globals().get("IST") or datetime.timezone(datetime.timedelta(hours=5, minutes=30))

global_state = globals().get("global_state", {}) or {}
global_summary = globals().get("global_summary", {"GL": 0.0, "GB": 0.0, "N19": 0.0}) or {"GL": 0.0, "GB": 0.0, "N19": 0.0}
globals()["global_summary"] = global_summary

tg = globals().get("tg", lambda *_a, **_k: None)
diag = globals().get("diag", lambda *_a, **_k: None)

SLIPPAGE_PER_SIDE = float(globals().get("SLIPPAGE_PER_SIDE", 0.18) or 0.18)
BROKERAGE_PER_ORDER = float(globals().get("BROKERAGE_PER_ORDER", 20) or 20)
QTY = int(globals().get("QTY", 0) or 0)
ALLOW_CONCURRENT_LAYERS = bool(globals().get("ALLOW_CONCURRENT_LAYERS", True))
SQUARE_OFF_TIME = globals().get("SQUARE_OFF_TIME", datetime.time(15, 15))

_as_ist_dt = globals().get("_as_ist_dt") or globals().get("as_ist_dt")
now_ist = globals().get("now_ist") or (lambda: datetime.datetime.now(IST))

def _emit(strategy: str, event: str, leg: Optional[str] = None,
          price: Optional[float] = None, reason: str = "", meta: Optional[dict] = None):
    fn = globals().get("emit_signal")
    if callable(fn):
        try:
            fn(strategy=strategy, event=event, leg=leg, price=price, reason=reason, meta=(meta or {}))
        except:
            if FAIL_FAST: raise
            pass

def _update_last_ltp(leg: str, ltp: float):
    try:
        leg = str(leg).upper()
        _last_ltp = globals().get("_last_ltp", {})
        if leg in ("CE", "PE"):
            v = float(ltp)
            if v > 0 and math.isfinite(v):
                _last_ltp[leg] = v
                globals()["_last_ltp"] = _last_ltp
    except:
        if FAIL_FAST: raise
        pass

def get_marketable_entry(leg: str) -> Optional[float]:
    st = global_state.get(str(leg).upper()) or {}
    ask = st.get("best_ask")
    ltp = st.get("ltp")
    try:
        if ask is not None and float(ask) > 0:
            return float(ask)
    except:
        if FAIL_FAST: raise
        pass
    try:
        if ltp is not None and float(ltp) > 0:
            return float(ltp)
    except:
        if FAIL_FAST: raise
        pass
    _last_ltp = globals().get("_last_ltp", {})
    v = _last_ltp.get(str(leg).upper())
    return float(v) if v is not None else None

def get_marketable_exit(leg: str) -> Optional[float]:
    st = global_state.get(str(leg).upper()) or {}
    bid = st.get("best_bid")
    ltp = st.get("ltp")
    try:
        if bid is not None and float(bid) > 0:
            return float(bid)
    except:
        if FAIL_FAST: raise
        pass
    try:
        if ltp is not None and float(ltp) > 0:
            return float(ltp)
    except:
        if FAIL_FAST: raise
        pass
    _last_ltp = globals().get("_last_ltp", {})
    v = _last_ltp.get(str(leg).upper())
    return float(v) if v is not None else None

# --- FIX: prevent RecursionError (self-referential log_trade wrapper) ---
if "_LOG_TRADE_CORE" not in globals():
    globals()["_LOG_TRADE_CORE"] = globals().get("log_trade")

def log_trade(layer, leg, side, price, qty, pnl_net="", reason=""):
    fn = globals().get("_LOG_TRADE_CORE")
    if callable(fn) and fn is not log_trade:
        try:
            fn(layer, leg, side, price, qty, pnl_net, reason)
        except:
            if FAIL_FAST: raise
            pass

# --- FIX: prevent RecursionError (self-referential _place_or_revert wrapper) ---
if "_PLACE_OR_REVERT_CORE" not in globals():
    globals()["_PLACE_OR_REVERT_CORE"] = globals().get("_place_or_revert")

def _place_or_revert(ok_if_paper: bool, leg: str, side: str, qty: int,
                     on_fail_revert_fn=None, fail_tag="order_fail"):
    fn = globals().get("_PLACE_OR_REVERT_CORE")
    if callable(fn) and fn is not _place_or_revert:
        return fn(ok_if_paper, leg, side, qty, on_fail_revert_fn, fail_tag)
    return (True, None)

# ============================================================
# N19 — IMB Pullback → VWAP → Reclaim (IMPROVED PE LOGIC)
#   ✅ PE demand filter: PE LTP must be ABOVE PE VWAP
#   ✅ BEAR IMB range filter: avoids exhausted moves
#   ✅ PE candle strength filter (close near lows)
#   ✅ PE requires bearish SPOT environment (SPOT < SPOT VWAP)
#   ✅ Wider SL for PE only
#   ✅ Consecutive loss protection + post-loss cooldown
# ============================================================

class N19LegState:
    def __init__(self):
        self.phase = "IDLE"  # IDLE -> WAIT_PULLBACK -> ARMED
        self.impulse_ts = None
        self.pullback_ts = None
        self.armed_ts = None

class N19State:
    def __init__(self):
        self.in_trade = False
        self.leg = None
        self.entry = None
        self.qty = 0
        self.sl = None
        self.tp1 = None
        self.tp2 = None
        self.peak = None
        self.partial_done = False

        # NEW: entry regime context (for conditional exits)
        self.entry_scenario = None
        self.special_exit_mode = False

        self.trades_today = 0
        self.trades_date = None
        self.cooldown_until = None

        self.leg_state = {"CE": N19LegState(), "PE": N19LegState()}

        # NEW: loss control
        self.consecutive_losses = 0
        self.last_loss_time = None

n19_state = globals().get("n19_state")
if not isinstance(n19_state, N19State):
    n19_state = N19State()
globals()["n19_state"] = n19_state

# ----------------------------
# TUNING
# ----------------------------
N19_MAX_TRADES_PER_DAY = 5

# --- BULL (CE) thresholds ---
N19_IMPULSE_BULL = 1.80
N19_REIMP_BULL   = 1.35

# --- BEAR (PE) thresholds — IMPROVED (range-limited to avoid exhaustion) ---
N19_BEAR_IMPULSE_MAX = 0.55
N19_BEAR_REIMP_MAX = 0.60

# (kept for legacy compatibility; not used for BEAR after the change below)
N19_IMPULSE_BEAR = N19_BEAR_IMPULSE_MAX
N19_REIMP_BEAR   = N19_BEAR_REIMP_MAX

# --- VWAP parameters ---
N19_PULLBACK_BAND_PCT = 0.0060
N19_RECLAIM_GAP_PCT = 0.0040

# NEW: PE-specific environment + demand confirmation
N19_PE_MIN_VWAP_GAP        = 0.0035   # SPOT must be >= 0.35% below SPOT VWAP
N19_PE_VWAP_DEMAND_MIN     = 0.0030   # PE LTP must be >= 0.30% above PE VWAP
N19_PE_CANDLE_CLOSE_MAX    = 0.30     # close must be in bottom 30% of range

# --- Time windows ---
N19_TIME_START = datetime.time(9, 20)
N19_TIME_END   = datetime.time(14, 20)
N19_LUNCH_START = datetime.time(11, 30)
N19_LUNCH_END   = datetime.time(13, 0)
N19_PHASE_EXPIRY_MIN = 25
N19_COOLDOWN_SEC = 240

# --- Stops/Targets ---
# (CE unchanged; PE wider)
N19_SL_ATR_MULT_CE = 1.50
N19_SL_ATR_MULT_PE = 1.50
N19_TRAIL_ATR_MULT    = 1.20
N19_MIN_SL_POINTS_CE = 8.0
N19_MIN_SL_POINTS_PE  = 8.0
N19_TP1_R = 1.00
N19_TP2_R = 2.00
N19_PARTIAL_FRAC = 0.33

# NEW: loss prevention
N19_MAX_CONSECUTIVE_LOSSES = 2
N19_MIN_TIME_AFTER_LOSS_SEC = 1800  # 30 minutes

# ============================================================
#  Applies ONLY when N19 is about to enter (setup already ARMED + reclaim).
# ============================================================

def _n19_get_gb_buy_level(true_open: float) -> Optional[float]:
    """Compute GB BUY level from True Open (same compute_buy_targets used by GB strategy)."""
    try:
        if true_open is None:
            return None
        to = float(true_open)
        if to <= 0:
            return None
        # compute_buy_targets is defined in GB block
        _, out = compute_buy_targets(to)
        buy = out.get("BUY") if isinstance(out, dict) else None
        return float(buy) if buy is not None else None
    except:
        return None

def _n19_trend_filter_ok(leg: str, ltp: float) -> Tuple[bool, Optional[int], str]:
    """Lightweight trend filter ( removed, slow timeframe removed).

    Returns (ok, scenario, reason_if_blocked)
      scenario: 1 = bullish alignment, 2 = bearish alignment
    """
    st = global_state.get(leg, {}) or {}

    # True Open (consume only; do not modify the true-open engine)
    true_open = st.get("open")
    try:
        true_open = float(true_open) if true_open is not None else None
    except:
        true_open = None

    if true_open is None or true_open <= 0:
        return (False, None, "TrueOpen missing")

    px = float(ltp)

    # Prefer option tick-vwap; fall back to 1m VWAP if needed
    vwap = st.get("vwap")
    if vwap is None:
        vwap = st.get("vwap_1m")

    if vwap is None:
        return (False, None, "VWAP not ready")

    try:
        vwap = float(vwap)
    except:
        return (False, None, "VWAP invalid")

    # Alignment rule: price & true_open on same side of VWAP
    if px >= true_open and px >= vwap:
        return (True, 1, "OK (bullish)")
    if px <= true_open and px <= vwap:
        return (True, 2, "OK (bearish)")

    return (False, None, "No trend alignment")

def _n19_reset_daily(ts: datetime.datetime):
    """Reset daily counters at start of new trading day"""
    d = ts.date()
    if n19_state.trades_date != d:
        n19_state.trades_date = d
        n19_state.trades_today = 0
        n19_state.cooldown_until = None
        n19_state.leg_state = {"CE": N19LegState(), "PE": N19LegState()}
        n19_state.consecutive_losses = 0
        n19_state.last_loss_time = None

def _n19_cooldown_ok(ts: datetime.datetime):
    """Check if cooldown period has passed"""
    if n19_state.cooldown_until is None:
        return True
    try:
        return ts >= n19_state.cooldown_until
    except:
        return True

def _n19_phase_expire_if_needed(ls: N19LegState, ts: datetime.datetime):
    """Expire armed/pullback phase if too much time has passed"""
    base = ls.impulse_ts or ls.pullback_ts or ls.armed_ts
    if base is None:
        return
    try:
        if (ts - base).total_seconds() > (N19_PHASE_EXPIRY_MIN * 60):
            ls.__init__()
    except:
        if FAIL_FAST: raise
        pass

def _n19_direction_ok(leg: str, sig: str) -> bool:
    """Verify leg matches signal direction (CE=BULL, PE=BEAR)"""
    leg = str(leg).upper()
    sig = str(sig).upper() if sig is not None else ""
    return (leg == "CE" and sig == "BULL") or (leg == "PE" and sig == "BEAR")

def _n19_impulse_ok(sig: str, ratio: float) -> bool:
    sig = str(sig).upper()
    if sig == 'BULL':
        return ratio >= N19_IMPULSE_BULL
    if sig == 'BEAR':
        return ratio <= N19_BEAR_IMPULSE_MAX
    return False

def _n19_reimpulse_ok(sig: str, ratio: float) -> bool:
    sig = str(sig).upper()
    if sig == 'BULL':
        return ratio >= N19_REIMP_BULL
    if sig == 'BEAR':
        return ratio <= N19_BEAR_REIMP_MAX
    return False

def _n19_get_atr_points(st: dict) -> Optional[float]:
    """Get ATR value from state, with validation"""
    try:
        v = st.get("atr")
        if v is None:
            return None
        v = float(v)
        if v <= 0 or (not math.isfinite(v)):
            return None
        return v
    except:
        return None

def n19_on_tick(ts: datetime.datetime, leg: str, ltp: float):
    """
    Main N19 strategy tick handler (called on every CE/PE option tick)

    IMPROVED PE LOGIC:
    - Avoid exhausted BEAR IMB (range filter)
    - Require bearish SPOT environment (SPOT < SPOT VWAP)
    - Require PE demand confirmation (PE LTP > PE VWAP by min gap)
    - Candle strength filter (PE close near lows)
    - Wider SL for PE
    - Consecutive loss throttling + post-loss cooldown
    """
    ts = _as_ist_dt(ts) or ts
    if not isinstance(ts, datetime.datetime):
        return

    t = ts.time()
    leg = str(leg).upper()
    px = float(ltp)

    if leg in ("CE", "PE"):
        _update_last_ltp(leg, px)

    _n19_reset_daily(ts)

    # Square-off
    if n19_state.in_trade and t >= SQUARE_OFF_TIME:
        n19_exit(ts, "SQUARE_OFF_TIME")
        return

    # ----------------------------
    # Manage open trade
    # ----------------------------
    if n19_state.in_trade and n19_state.leg == leg:
        cur = px
        n19_state.peak = max(float(n19_state.peak or cur), cur)

        # SL
        if n19_state.sl is not None and cur <= float(n19_state.sl):
            n19_exit(ts, "STOP_HIT")
            return

        # TP1
        if (not n19_state.partial_done) and n19_state.tp1 is not None and cur >= float(n19_state.tp1):
            book_qty = max(1, min(int(n19_state.qty * float(N19_PARTIAL_FRAC)), n19_state.qty))
            n19_partial(ts, book_qty, "TP1_1R")
            n19_state.qty -= book_qty
            n19_state.partial_done = True
            n19_state.sl = max(float(n19_state.sl), float(n19_state.entry))

        # TP2
        if n19_state.partial_done and n19_state.tp2 is not None and n19_state.qty > 0:
            if cur >= float(n19_state.tp2):
                n19_exit(ts, "TP2_2R")
                return

        # Trail
        if n19_state.partial_done and n19_state.qty > 0:
            st = global_state.get(leg, {}) or {}
            atr = _n19_get_atr_points(st)
            if atr is not None:
                trail = float(n19_state.peak) - float(N19_TRAIL_ATR_MULT) * float(atr)
                n19_state.sl = max(float(n19_state.sl), float(trail))
        return

    # ----------------------------
    # Entry filters (not in trade)
    # ----------------------------
    if not ALLOW_CONCURRENT_LAYERS:
        gls = globals().get("gl_state")
        gbs = globals().get("gb_state")
        if (gls and getattr(gls, "in_trade", False)) or (gbs and getattr(gbs, "in_trade", False)):
            return

    if n19_state.trades_today >= N19_MAX_TRADES_PER_DAY:
        return

    if not _n19_cooldown_ok(ts):
        return

    if not (N19_TIME_START <= t <= N19_TIME_END):
        return
    # Lunch block: no new N19 entries during low-volume chop
    if N19_LUNCH_START <= t <= N19_LUNCH_END:
        return

    if leg not in ("CE", "PE"):
        return

    # NEW: loss throttles
    if n19_state.consecutive_losses >= N19_MAX_CONSECUTIVE_LOSSES:
        tg_throttled("n19_max_losses",
                     f"🛑 N19 stopped - {N19_MAX_CONSECUTIVE_LOSSES} consecutive losses. Review setup!",
                     300)
        return

    if n19_state.last_loss_time is not None:
        try:
            if (ts - n19_state.last_loss_time).total_seconds() < N19_MIN_TIME_AFTER_LOSS_SEC:
                return
        except:
            if FAIL_FAST: raise
            pass

    # Market data (option leg state)
    st = global_state.get(leg, {}) or {}
    ratio = st.get("imb_ratio")
    sig   = st.get("imb_signal")
    opt_vwap = st.get("vwap_1m")  # option VWAP (CE/PE)

    if ratio is None or sig is None or opt_vwap is None:
        return
    try:
        ratio = float(ratio)
        opt_vwap = float(opt_vwap)
    except:
        return
    if opt_vwap <= 0:
        return

    if not _n19_direction_ok(leg, sig):
        return

    # ATR for SL sizing (option ATR7 points from Block-2)
    atr = _n19_get_atr_points(st)
    if atr is None:
        return

    # Candle data (option 1m candle)
    bar_high = st.get("bar_high")
    bar_low  = st.get("bar_low")
    bar_close = st.get("bar_close")
    if bar_high is None or bar_low is None or bar_close is None:
        return
    try:
        bar_high = float(bar_high)
        bar_low = float(bar_low)
        bar_close = float(bar_close)
    except:
        return

    # Leg phase machine
    ls = n19_state.leg_state.get(leg) or N19LegState()
    n19_state.leg_state[leg] = ls
    _n19_phase_expire_if_needed(ls, ts)

    # ------------------------------------------------------------
    # PHASE 1: IDLE (initial impulse)
    # ------------------------------------------------------------
    if ls.phase == "IDLE":
        if not _n19_impulse_ok(sig, ratio):
            return

        # Use option VWAP for the pullback/reclaim structure (existing behavior)
        if str(sig).upper() == "BULL":
            if px > opt_vwap * (1 + N19_RECLAIM_GAP_PCT):
                ls.phase = "WAIT_PULLBACK"
                ls.impulse_ts = ts
                tg_throttled(f"n19_imp_{leg}",
                             f"⚡ N19 IMPULSE {leg} BULL r={ratio:.2f} px={px:.2f} optVWAP={opt_vwap:.2f}",
                             20)
        else:
            # PE requires bearish SPOT environment even to arm the setup
            if leg == "PE":
                spot_ltp = (spot_state or {}).get("ltp")
                spot_vwap = (spot_state or {}).get("vwap")
                if spot_ltp is None or spot_vwap is None:
                    return
                try:
                    spot_ltp = float(spot_ltp); spot_vwap = float(spot_vwap)
                except:
                    return
                if spot_vwap <= 0:
                    return
                if ((spot_vwap - spot_ltp) / spot_vwap) < N19_PE_MIN_VWAP_GAP:
                    return

            if px < opt_vwap * (1 - N19_RECLAIM_GAP_PCT):
                ls.phase = "WAIT_PULLBACK"
                ls.impulse_ts = ts
                tg_throttled(f"n19_imp_{leg}",
                             f"⚡ N19 IMPULSE {leg} BEAR r={ratio:.2f} px={px:.2f} optVWAP={opt_vwap:.2f}",
                             20)
        return

    # ------------------------------------------------------------
    # PHASE 2: WAIT_PULLBACK (touch VWAP band)
    # ------------------------------------------------------------
    if ls.phase == "WAIT_PULLBACK":
        band_lo = opt_vwap * (1 - N19_PULLBACK_BAND_PCT)
        band_hi = opt_vwap * (1 + N19_PULLBACK_BAND_PCT)
        touched = (bar_low <= band_hi and bar_high >= band_lo)

        if touched:
            ls.phase = "ARMED"
            ls.pullback_ts = ts
            ls.armed_ts = ts
            tg_throttled(f"n19_pb_{leg}",
                         f"🎯 N19 PULLBACK OK {leg} touched VWAP band | optVWAP={opt_vwap:.2f}",
                         20)
        return

    # ------------------------------------------------------------
    # PHASE 3: ARMED (confirm + reclaim + entry)
    # ------------------------------------------------------------
    if ls.phase == "ARMED":
        if not _n19_reimpulse_ok(sig, ratio):
            return

        # ============================================================
        # CE candle strength: close must be in top 35% of candle range
        if str(sig).upper() == 'BULL' and leg == 'CE':
            candle_range = bar_high - bar_low
            if candle_range > 0:
                close_pos = (bar_close - bar_low) / candle_range
                if close_pos < 0.65:
                    tg_throttled('n19_ce_weak_candle',
                                 f'⚠️ N19 CE SKIP - Weak candle | close@{close_pos*100:.0f}% (need >65%)',
                                 20)
                    return

        # PE-SPECIFIC FILTERS (NEW LOGIC)
        # ============================================================
        if str(sig).upper() == "BEAR" and leg == "PE":
            # A) SPOT environment must be bearish (SPOT < SPOT VWAP)
            spot_ltp = (spot_state or {}).get("ltp")
            spot_vwap = (spot_state or {}).get("vwap")
            if spot_ltp is None or spot_vwap is None:
                return
            try:
                spot_ltp = float(spot_ltp)
                spot_vwap = float(spot_vwap)
            except:
                return
            if spot_vwap <= 0:
                return

            spot_vwap_gap = (spot_vwap - spot_ltp) / spot_vwap
            if spot_vwap_gap < N19_PE_MIN_VWAP_GAP:
                tg_throttled("n19_pe_spot_vwap_close",
                             f"⚠️ N19 PE SKIP - SPOT too close to SPOT VWAP | "
                             f"spot={spot_ltp:.2f} vwap={spot_vwap:.2f} gap={spot_vwap_gap*100:.2f}% "
                             f"(need >{N19_PE_MIN_VWAP_GAP*100:.2f}%)",
                             20)
                return

            # B) PE demand confirmation: PE LTP must be ABOVE PE VWAP
            if px <= opt_vwap:
                tg_throttled("n19_pe_below_optvwap",
                             f"🚫 N19 PE REJECTED - No demand | PE={px:.2f} <= PE_VWAP={opt_vwap:.2f} | IMB={ratio:.2f}",
                             15)
                # invalidate the armed setup (exhaustion/profit booking)
                ls.__init__()
                return

            pe_vwap_gap = (px - opt_vwap) / opt_vwap
            if pe_vwap_gap < N19_PE_VWAP_DEMAND_MIN:
                tg_throttled("n19_pe_weak_demand",
                             f"⚠️ N19 PE REJECTED - Weak demand | gap={pe_vwap_gap*100:.2f}% "
                             f"(need >{N19_PE_VWAP_DEMAND_MIN*100:.2f}%) | PE={px:.2f} VWAP={opt_vwap:.2f}",
                             15)
                return

            # C) Candle strength: close near lows
            candle_range = bar_high - bar_low
            if candle_range > 0:
                close_pos = (bar_close - bar_low) / candle_range  # 0=low, 1=high
                if close_pos > N19_PE_CANDLE_CLOSE_MAX:
                    tg_throttled("n19_pe_weak_candle",
                                 f"⚠️ N19 PE SKIP - Weak candle | close@{close_pos*100:.0f}% "
                                 f"(need <{N19_PE_CANDLE_CLOSE_MAX*100:.0f}%)",
                                 20)
                    return

        # Reclaim condition (option reclaim vs option VWAP)
        reclaim = (px > opt_vwap * (1 + N19_RECLAIM_GAP_PCT)) if str(sig).upper() == "BULL" else (px < opt_vwap * (1 - N19_RECLAIM_GAP_PCT))
        if not reclaim:
            return

        # ----------------------------
        #  Applies ONLY after N19 setup is ACTIVE (ARMED + reclaim).
        # ----------------------------
        ok_tf, scenario, block_reason = _n19_trend_filter_ok(leg, px)
        if not ok_tf:
            # Trend filter blocked — reset setup entirely, do not delay
            ls.__init__()
            tg_throttled(f'n19_tf_block_{leg}',
                         f'⛔ N19 TREND FILTER BLOCKED {leg} | {block_reason} | Setup reset.',
                         15)
            return

        # ----------------------------
        # ENTRY
        # ----------------------------
        # Ensure eligible strike at entry-time (candidate hot-swap; N19 core logic unchanged)
        try:
            select_and_activate_candidate(leg, context="N19_entry_place")
        except:
            if FAIL_FAST: raise
            pass
        opt_entry = get_marketable_entry(leg)
        if opt_entry is None:
            return

        entry = float(opt_entry) + SLIPPAGE_PER_SIDE

        # Leg-specific SL sizing
        sl_atr_mult = float(N19_SL_ATR_MULT_PE if leg == "PE" else N19_SL_ATR_MULT_CE)
        min_sl_pts  = float(N19_MIN_SL_POINTS_PE if leg == "PE" else N19_MIN_SL_POINTS_CE)

        sl_dist = max(min_sl_pts, sl_atr_mult * float(atr))
        sl = entry - sl_dist
        tp1 = entry + float(N19_TP1_R) * sl_dist
        tp2 = entry + float(N19_TP2_R) * sl_dist

        if sl >= entry:
            return

        def _revert():
            keep_leg_state = n19_state.leg_state
            keep_date = n19_state.trades_date
            keep_trades = n19_state.trades_today
            keep_cd = n19_state.cooldown_until
            keep_losses = n19_state.consecutive_losses
            keep_loss_time = n19_state.last_loss_time
            n19_state.__init__()
            n19_state.leg_state = keep_leg_state
            n19_state.trades_date = keep_date
            n19_state.trades_today = keep_trades
            n19_state.cooldown_until = keep_cd
            n19_state.consecutive_losses = keep_losses
            n19_state.last_loss_time = keep_loss_time

        n19_state.in_trade = True
        n19_state.leg = leg
        n19_state.entry = entry
        n19_state.qty = int(QTY)
        n19_state.sl = sl
        n19_state.tp1 = tp1
        n19_state.tp2 = tp2
        n19_state.peak = entry
        n19_state.partial_done = False

        # NEW: store entry regime context for conditional exit rule
        try:
            n19_state.entry_scenario = int(scenario) if scenario is not None else None
            st0 = global_state.get(leg, {}) or {}

            n19_state.special_exit_mode = bool(
                n19_state.entry_scenario == 2
            )

            if n19_state.special_exit_mode:
                tg_throttled(f"n19_special_exit_{leg}",
                             f"🧩 N19 SPECIAL EXIT MODE {leg} | Scenario-2 entry (3 bullish<0, 15>0). Exit @ TP1 or when 15 drops below 0.",
                             30)
        except:
            n19_state.entry_scenario = None
            n19_state.special_exit_mode = False

        n19_state.trades_today += 1
        ls.__init__()

        ok, _ = _place_or_revert(True, leg, "BUY", n19_state.qty,
                                on_fail_revert_fn=_revert,
                                fail_tag=f"n19_buy_fail_{leg}")
        if not ok:
            return

        log_trade("N19", leg, "ENTRY", n19_state.entry, n19_state.qty,
                  reason=f"IMB+VWAP reclaim | sig={sig} r={ratio:.2f} optVWAP={opt_vwap:.2f} atr7={atr:.2f} SLd={sl_dist:.2f}")

        tg(f"🟦 N19 ENTRY {leg} @ {n19_state.entry:.2f} | SL={n19_state.sl:.2f} | "
           f"TP1={n19_state.tp1:.2f} TP2={n19_state.tp2:.2f} | sig={sig} r={ratio:.2f} optVWAP={opt_vwap:.2f} atr7={atr:.2f}")
        _emit("N19", "ENTRY", leg, n19_state.entry, "IMB_VWAP_RECLAIM",
              {"sig": sig, "imb": ratio, "opt_vwap": opt_vwap, "atr7": atr, "sl_dist": sl_dist})
        return

def n19_partial(ts: datetime.datetime, qty: int, reason: str):
    """Partial exit (book profits at TP1)"""
    leg = n19_state.leg
    exit_px = get_marketable_exit(leg)
    if exit_px is None:
        exit_px = _last_ltp.get(leg) or n19_state.entry
        tg_throttled("n19_partial_fallback", f"⚠️ N19 PARTIAL fallback px used for {leg} | {reason}", 20)

    pnl = (float(exit_px) - float(n19_state.entry)) * qty - (SLIPPAGE_PER_SIDE * 2 * qty) - (BROKERAGE_PER_ORDER * 2)
    global_summary["N19"] = float(global_summary.get("N19", 0.0)) + float(pnl)

    _place_or_revert(True, leg, "SELL", qty, on_fail_revert_fn=None, fail_tag="n19_sell_fail")

    log_trade("N19", leg, "PARTIAL", float(exit_px), qty, pnl_net=pnl, reason=reason)
    tg(f"🟨 N19 PARTIAL {leg} @ {float(exit_px):.2f} qty={qty} NET ₹{pnl:+.0f} | {reason}")
    _emit("N19", "INFO", leg, float(exit_px), reason, {"pnl": pnl})

def n19_exit(ts: datetime.datetime, reason: str):
    """
    Full exit (stop loss, take profit, or square-off)
    IMPROVED: Tracks consecutive losses to prevent overtrading after bad setups.
    """
    if not n19_state.in_trade or not n19_state.leg:
        return

    leg = n19_state.leg
    exit_px = get_marketable_exit(leg)
    if exit_px is None:
        exit_px = _last_ltp.get(leg) or n19_state.entry
        tg_throttled("n19_exit_fallback", f"⚠️ N19 EXIT fallback px used for {leg} | {reason}", 20)

    qty = int(n19_state.qty)
    pnl = (float(exit_px) - float(n19_state.entry)) * qty - (SLIPPAGE_PER_SIDE * 2 * qty) - (BROKERAGE_PER_ORDER * 2)
    global_summary["N19"] = float(global_summary.get("N19", 0.0)) + float(pnl)

    _place_or_revert(True, leg, "SELL", qty, on_fail_revert_fn=None, fail_tag="n19_sell_fail")

    log_trade("N19", leg, "EXIT", float(exit_px), qty, pnl_net=pnl, reason=reason)
    tg(f"🟥 N19 EXIT {leg} @ {float(exit_px):.2f} NET ₹{pnl:+.0f} | {reason}")
    _emit("N19", "EXIT", leg, float(exit_px), reason, {"pnl": pnl})

    # NEW: loss tracking (prefer true PnL; fallback to reason keywords)
    try:
        is_loss = (float(pnl) < 0)
    except:
        is_loss = ("STOP" in str(reason).upper()) or ("SL" in str(reason).upper())

    if is_loss:
        n19_state.consecutive_losses += 1
        n19_state.last_loss_time = ts
        if n19_state.consecutive_losses >= N19_MAX_CONSECUTIVE_LOSSES:
            tg(f"🛑 N19 - {N19_MAX_CONSECUTIVE_LOSSES} consecutive losses! New entries paused. Review setup.")
    else:
        n19_state.consecutive_losses = 0
        n19_state.last_loss_time = None

    n19_state.cooldown_until = ts + datetime.timedelta(seconds=N19_COOLDOWN_SEC)

    # Reset state while preserving important counters
    keep_leg_state = n19_state.leg_state
    keep_date = n19_state.trades_date
    keep_trades = n19_state.trades_today
    keep_cd = n19_state.cooldown_until
    keep_losses = n19_state.consecutive_losses
    keep_loss_time = n19_state.last_loss_time

    n19_state.__init__()
    n19_state.leg_state = keep_leg_state
    n19_state.trades_date = keep_date
    n19_state.trades_today = keep_trades
    n19_state.cooldown_until = keep_cd
    n19_state.consecutive_losses = keep_losses
    n19_state.last_loss_time = keep_loss_time

globals()["n19_on_tick"] = n19_on_tick
globals()["n19_exit"] = n19_exit

log("✅ N19 loaded (IMPROVED PE LOGIC: IMB range + SPOT VWAP env + PE demand(VWAP) + candle strength + PE wider SL + loss throttles)")

# BLOCK 4/6 — SIGNAL AGGREGATOR (GL/GB/N19) + SQUARE-OFF ROUTER ✅
#  ✅ Signal bus used by dashboard
#  ✅ square_off_all() calls gl_exit/gb_exit/n19_exit if available
# ============================================================

import datetime, threading, queue
from dataclasses import dataclass
from typing import Optional, Dict, Any

IST = globals().get("IST") or datetime.timezone(datetime.timedelta(hours=5, minutes=30))

# --- Shared stop event ---
if "block_stop" not in globals() or globals().get("block_stop") is None:
    globals()["block_stop"] = threading.Event()

block_stop = globals()["block_stop"]

# --- Signal bus ---
signal_q = globals().get("signal_q")
if signal_q is None or not isinstance(signal_q, queue.Queue):
    signal_q = queue.Queue(maxsize=5000)
globals()["signal_q"] = signal_q

@dataclass
class SignalEvent:
    ts: datetime.datetime
    strategy: str          # "GL","GB","N19"
    event: str             # "ENTRY","EXIT","PARTIAL","INFO"
    leg: Optional[str]     # "CE"/"PE"/None
    price: Optional[float] # option/spot price (optional)
    reason: str = ""
    meta: Dict[str, Any] = None

# last signal for dashboard
if "LAST_SIGNALS" not in globals() or not isinstance(globals().get("LAST_SIGNALS"), dict):
    globals()["LAST_SIGNALS"] = {"GL": None, "GB": None, "N19": None}

def emit_signal(strategy: str, event: str, leg: Optional[str] = None,
                price: Optional[float] = None, reason: str = "", meta: Optional[dict] = None):
    try:
        se = SignalEvent(
            ts=datetime.datetime.now(IST),
            strategy=str(strategy),
            event=str(event),
            leg=(str(leg).upper() if leg else None),
            price=(float(price) if price is not None else None),
            reason=str(reason or ""),
            meta=(meta or {})
        )
        globals()["LAST_SIGNALS"][str(strategy)] = se
        try:
            signal_q.put_nowait(se)
        except:
            if FAIL_FAST: raise
            pass
    except:
        if FAIL_FAST: raise
        pass

def get_last_signal_text(strategy: str) -> str:
    try:
        se = (globals().get("LAST_SIGNALS", {}) or {}).get(strategy)
        if se is None:
            return "—"
        t = se.ts.strftime("%H:%M:%S")
        px = f"{se.price:.2f}" if isinstance(se.price, (int, float)) else "—"
        leg = se.leg or "—"
        rs = (se.reason or "")[:40]
        return f"{t} {se.event} {leg} @ {px} {rs}".strip()
    except:
        return "—"

globals()["emit_signal"] = emit_signal
globals()["get_last_signal_text"] = get_last_signal_text

# ------------------------
# Square-off router (15:15 IST)
# ------------------------
SQUARE_OFF_TIME = globals().get("SQUARE_OFF_TIME", datetime.time(15, 15))
globals()["SQUARE_OFF_TIME"] = SQUARE_OFF_TIME

def square_off_all(reason: str = "SQUARE_OFF_1515"):
    """
    Calls exit functions if available.
    ✅ Calls: gl_exit, gb_exit, n19_exit (your current naming)
    ✅ Keeps backward-compat with older hooks too
    """
    ts = datetime.datetime.now(IST)

    # Preferred hooks in your bot
    for fn_name in ("gl_exit", "gb_exit", "n19_exit"):
        fn = globals().get(fn_name)
        if callable(fn):
            try:
                fn(ts, reason)
            except:
                try:
                    fn(reason)
                except:
                    if FAIL_FAST: raise
                    pass

    # Backward compat hooks (if any older names exist)
    for fn_name in ("exit_n19_trade", "exit_l1_trade", "exit_l1a_trade", "exit_l1b_trade"):
        fn = globals().get(fn_name)
        if callable(fn):
            try:
                fn(ts, reason)
            except:
                try:
                    fn(reason)
                except:
                    if FAIL_FAST: raise
                    pass

globals()["square_off_all"] = square_off_all

print("✅ Block-4 ready: Signal bus + LAST_SIGNALS + square_off_all()")

# ============================================================
# BLOCK 5/6 — WEBSOCKET + ENGINE ✅ (GOOGLE COLAB OPTIMIZED)
#  ✅ Compatible with new N19 (atr + candle fields)
# ============================================================

import os, csv, time, queue, threading, datetime, traceback, sys
import pandas as pd

# ----------------------------
# TIMEZONE & LOGGING
# ----------------------------
IST = globals().get("IST") or datetime.timezone(datetime.timedelta(hours=5, minutes=30))

def _log(msg: str):
    try:
        ts = datetime.datetime.now(IST).strftime("%H:%M:%S")
        full_msg = f"[{ts}] {msg}"
        print(full_msg, flush=True)
        fn = globals().get("log")
        if callable(fn) and fn != _log:
            try:
                fn(msg)
            except:
                if FAIL_FAST: raise
                pass
    except:
        if FAIL_FAST: raise
        pass

# Throttled telegram
tg_throttled = globals().get("tg_throttled")
if not callable(tg_throttled):
    _TG_CACHE = {}
    _TG_LOCK = threading.Lock()
    def tg_throttled(key: str, msg: str, cooldown_sec: int = 60):
        try:
            noww = time.time()
            with _TG_LOCK:
                last = _TG_CACHE.get(key, 0.0)
                if (noww - last) < float(cooldown_sec):
                    return
                _TG_CACHE[key] = noww
            fn = globals().get("tg")
            if callable(fn):
                fn(msg)
        except:
            if FAIL_FAST: raise
            pass
    globals()["tg_throttled"] = tg_throttled

# Safe stop
block_stop = globals().get("block_stop")
if block_stop is None or not isinstance(block_stop, threading.Event):
    block_stop = threading.Event()
globals()["block_stop"] = block_stop

safe_stop = globals().get("safe_stop")
if not callable(safe_stop):
    def safe_stop(reason: str, exc: Exception = None, fatal: bool = True):
        msg = f"SAFE_STOP: {reason}"
        if exc:
            msg += f" | {type(exc).__name__}: {exc}"
        _log(f"🛑 {msg}")
        globals()["SAFE_STOP_REASON"] = reason
        globals()["ws_status"] = "SAFE_STOP"
        try:
            block_stop.set()
        except:
            if FAIL_FAST: raise
            pass
        if fatal:
            raise RuntimeError(msg)
    globals()["safe_stop"] = safe_stop

# ----------------------------
# MARKET HOURS
# ----------------------------
market_open_time = globals().get("market_open_time", datetime.time(9, 15))
market_close_time = globals().get("market_close_time", datetime.time(15, 30))
WS_ALLOW_START = datetime.time(8, 55)
WS_ALLOW_END = datetime.time(15, 35)

def _within_ws_hours(nowt: datetime.time = None):
    if nowt is None:
        nowt = datetime.datetime.now(IST).time()
    return (nowt >= WS_ALLOW_START) and (nowt <= WS_ALLOW_END)

def _get_current_ist():
    return datetime.datetime.now(IST)

# ----------------------------
# VALIDATE REQUIRED GLOBALS
# ----------------------------
required = ["kite", "API_KEY", "ACCESS_TOKEN",
            "SPOT_TOKEN", "CE_TOKEN", "PE_TOKEN",
            "SPOT_SYMBOL", "CE_TSYM", "PE_TSYM"]
missing = [x for x in required if not globals().get(x)]
if missing:
    _log(f"❌ Missing: {missing}. Run Block-1 first!")
    raise RuntimeError(f"Block-5 missing globals: {missing}")

kite = globals()["kite"]
API_KEY = str(globals()["API_KEY"])
ACCESS_TOKEN = str(globals()["ACCESS_TOKEN"])
SPOT_TOKEN = int(globals()["SPOT_TOKEN"])
CE_TOKEN = int(globals()["CE_TOKEN"])
PE_TOKEN = int(globals()["PE_TOKEN"])
SPOT_SYMBOL = str(globals()["SPOT_SYMBOL"])
CE_TSYM = str(globals()["CE_TSYM"])
PE_TSYM = str(globals()["PE_TSYM"])
RUN_ID = str(globals().get("RUN_ID", _get_current_ist().strftime("%H%M%S")))
UNDERLYING = str(globals().get("UNDERLYING", "NIFTY"))

_log(f"📌 SPOT={SPOT_TOKEN} CE={CE_TOKEN} PE={PE_TOKEN}")

# ----------------------------
# CANONICAL TIMESTAMP (from Block-1)
# ----------------------------
as_ist_dt = globals().get("as_ist_dt") or globals().get("_as_ist_dt")
if not callable(as_ist_dt):
    def as_ist_dt(d, now_dt=None):
        if d is None:
            return None
        if isinstance(d, datetime.datetime):
            if d.tzinfo is None:
                return d.replace(tzinfo=IST)
            return d.astimezone(IST)
        return None
    globals()["as_ist_dt"] = as_ist_dt
    _log("⚠ Using fallback as_ist_dt()")

def as_ist_dt_safe(raw, now_dt=None):
    if raw is None:
        return None
    try:
        if now_dt is not None:
            return as_ist_dt(raw, now_dt)
        return as_ist_dt(raw)
    except:
        return None

# ----------------------------
# STATUS TRACKERS
# ----------------------------
globals()["ws_status"] = globals().get("ws_status", "INIT")
globals()["reconnects"] = int(globals().get("reconnects", 0) or 0)
globals()["errors"] = int(globals().get("errors", 0) or 0)
globals()["last_spot_tick_wall"] = float(globals().get("last_spot_tick_wall", 0.0) or 0.0)
globals()["last_ce_tick_wall"] = float(globals().get("last_ce_tick_wall", 0.0) or 0.0)
globals()["last_pe_tick_wall"] = float(globals().get("last_pe_tick_wall", 0.0) or 0.0)
globals()["engine_last_wall"] = float(globals().get("engine_last_wall", 0.0) or 0.0)
globals()["tickq_drops"] = int(globals().get("tickq_drops", 0) or 0)
globals()["ws_last_ping"] = float(globals().get("ws_last_ping", 0.0) or 0.0)

# Tick queue
tick_q = globals().get("tick_q")
if tick_q is None or not isinstance(tick_q, queue.Queue):
    tick_q = queue.Queue(maxsize=20000)
globals()["tick_q"] = tick_q

# ----------------------------
# STATE STRUCTURES (UPDATED for new N19)
# ----------------------------
def _mk_opt_state():
    return {
        "ltp": None,
        "best_bid": None, "best_ask": None, "spread": None,
        "vol_traded": 0.0, "vol_delta": 0.0,

        # open pipeline
        "ws_ohlc_open": None, "rest_ohlc_open": None,
        "open": None, "open_ts": None, "open_source": None,
        "true_open_set": False, "open_priority": -1,

        # imbalance + vwap (Block-2)
        "imb_ratio": None, "imb_signal": None, "imb_ts": None,
        "imb15_cur": None, "imb15_ts": None, "imb_avg20_15m": None,
        "rsi3_15m_prev": None,
        "vwap_15m": None,
        "rsi3_15m": None, "rsi3_15m_ts": None,
        "vwap_1m": None,
        "rsi3_1m": None, "rsi3_ts": None,

        "c15_close": None,

        "c3_close": None,

        # old ATR (legacy support)
        "atr14_1m": None,

        # ✅ NEW for N19 candle-based logic
        "bar_open": None,
        "bar_high": None,
        "bar_low": None,
        "bar_close": None,
        "bar_ts": None,     # last bar timestamp

        # ✅ NEW ATR(7) SMA points for N19
        "atr": None}

def _ensure_keys(dst: dict, template: dict):
    for k, v in template.items():
        if k not in dst:
            dst[k] = v

# Global state
if "global_state" not in globals() or not isinstance(globals().get("global_state"), dict):
    globals()["global_state"] = {"CE": _mk_opt_state(), "PE": _mk_opt_state()}
else:
    gs = globals()["global_state"]
    gs.setdefault("CE", {})
    gs.setdefault("PE", {})
    _ensure_keys(gs["CE"], _mk_opt_state())
    _ensure_keys(gs["PE"], _mk_opt_state())

global_state = globals()["global_state"]

# Spot state
if "spot_state" not in globals() or not isinstance(globals().get("spot_state"), dict):
    globals()["spot_state"] = {"ltp": None, "ts": None, "vwap": None, "vwap_ts": None}
spot_state = globals()["spot_state"]

# ----------------------------
# CSV PATHS
# ----------------------------
TICK_LOG_PATH = globals().get("TICK_LOG_PATH", os.path.join(os.getcwd(), "ticks.csv"))
TRADES_CSV = globals().get("TRADES_CSV", os.path.join(os.getcwd(), "trades.csv"))
globals()["TICK_LOG_PATH"] = TICK_LOG_PATH
globals()["TRADES_CSV"] = TRADES_CSV

# ✅ UPDATED HEADER (adds atr + candle fields)
TICK_HEADER = [
    "date", "time", "exch_time",
    "leg", "symbol", "token",
    "ltp", "bid", "ask", "spread",
    "vol_traded_cum", "vol_delta",
    "spot_ltp", "spot_vwap",
    "GL_inTrade", "GL_leg",
    "GB_inTrade", "GB_leg",
    "N19_inTrade", "N19_leg",
    "imb_ratio", "imb_signal", "vwap",
    "rsi3_15m", "vwap_15m", "imb15_cur", "imb_avg20_15m",
    "opt_atr7", "bar_high", "bar_low", "bar_close",
    "run_id"
]

def ensure_csv_header(path: str, header: list):
    try:
        if (not os.path.exists(path)) or os.path.getsize(path) == 0:
            with open(path, "w", newline="") as f:
                csv.writer(f).writerow(header)
            return
        with open(path, "r", newline="") as f:
            first = f.readline().strip()
        cols = [c.strip().replace('"', '') for c in first.split(",")] if first else []
        if cols != header:
            bak = path.replace(".csv", f"_{RUN_ID}.csv")
            try:
                os.rename(path, bak)
            except:
                if FAIL_FAST: raise
                pass
            with open(path, "w", newline="") as f:
                csv.writer(f).writerow(header)
    except Exception as e:
        _log(f"⚠ CSV header error: {e}")

ensure_csv_header(TICK_LOG_PATH, TICK_HEADER)

# Tick buffer
tick_buffer = globals().get("tick_buffer")
tick_buffer_lock = globals().get("tick_buffer_lock")
if tick_buffer is None or not isinstance(tick_buffer, list):
    tick_buffer = []
if tick_buffer_lock is None:
    tick_buffer_lock = threading.Lock()
globals()["tick_buffer"] = tick_buffer
globals()["tick_buffer_lock"] = tick_buffer_lock
globals()["ticks_saved"] = int(globals().get("ticks_saved", 0) or 0)
globals()["tick_buffer_size"] = 0
globals()["last_flush_time"] = ""

# ----------------------------
# TICK WRITER THREAD
# ----------------------------
def _tick_writer_loop():
    while not block_stop.is_set():
        try:
            rows = None
            with tick_buffer_lock:
                globals()["tick_buffer_size"] = len(tick_buffer)
                if tick_buffer:
                    rows = tick_buffer[:]
                    tick_buffer.clear()
                    globals()["tick_buffer_size"] = 0

            if rows:
                try:
                    with open(TICK_LOG_PATH, "a", newline="") as f:
                        w = csv.writer(f)
                        for r in rows:
                            if len(r) < len(TICK_HEADER):
                                r = list(r) + [""] * (len(TICK_HEADER) - len(r))
                            w.writerow(r)
                        f.flush()
                    globals()["ticks_saved"] = int(globals().get("ticks_saved", 0)) + len(rows)
                    globals()["last_flush_time"] = _get_current_ist().strftime("%H:%M:%S")
                except Exception as e:
                    _log(f"⚠ Tick write error: {e}")

            time.sleep(0.1)

        except Exception as e:
            _log(f"⚠ Tick writer error: {e}")
            time.sleep(1)

if not globals().get("_TICK_WRITER_STARTED", False):
    globals()["_TICK_WRITER_STARTED"] = True
    t = threading.Thread(target=_tick_writer_loop, daemon=True, name="TickWriter")
    t.start()
    _log("📝 Tick writer started")

# ----------------------------
# TICK HELPERS
# ----------------------------
def _best_bid_ask_from_tick(tk: dict):
    d = tk.get("depth") or {}
    buy = d.get("buy") or []
    sell = d.get("sell") or []
    bid = ask = None
    try:
        if buy and isinstance(buy[0], dict):
            bid = float(buy[0].get("price") or 0) or None
    except:
        if FAIL_FAST: raise
        pass
    try:
        if sell and isinstance(sell[0], dict):
            ask = float(sell[0].get("price") or 0) or None
    except:
        if FAIL_FAST: raise
        pass
    spr = None
    if bid is not None and ask is not None and ask > bid:
        spr = ask - bid
    return bid, ask, spr

_last_cum_vol = {}

def _delta_vol(token: int, cum: float):
    try:
        cum = float(cum or 0.0)
    except:
        return 0.0
    prev = _last_cum_vol.get(token)
    _last_cum_vol[token] = cum
    if prev is None:
        return 0.0
    d = cum - float(prev)
    return float(d) if d >= 0 else 0.0

# Block-2 indicator update (REQUIRED)
_update_ind = globals().get("update_indicators_from_tick")
if not callable(_update_ind):
    _log("⚠ update_indicators_from_tick() not found. Run Block-2 first.")

# ----------------------------
# FUT TOKEN FOR VOLUME PROXY (optional)
# ----------------------------
def _resolve_fut_token():
    if globals().get("FUT_TOKEN"):
        return int(globals()["FUT_TOKEN"]), str(globals().get("FUT_TSYM", ""))

    if UNDERLYING not in ("NIFTY", "BANKNIFTY"):
        return None, None

    try:
        fn = globals().get("kite_instruments")
        nfo = fn("NFO") if callable(fn) else kite.instruments("NFO")
        df = pd.DataFrame(nfo)
        fut = df[(df["segment"] == "NFO-FUT") & (df["name"].astype(str).str.upper() == UNDERLYING)].copy()
        if fut.empty:
            return None, None
        fut["expiry_date"] = pd.to_datetime(fut["expiry"]).dt.date
        today = _get_current_ist().date()
        fut = fut[fut["expiry_date"] >= today].sort_values("expiry_date")
        if fut.empty:
            return None, None
        r = fut.iloc[0].to_dict()
        token = int(r["instrument_token"])
        tsym = "NFO:" + str(r["tradingsymbol"])
        globals()["FUT_TOKEN"] = token
        globals()["FUT_TSYM"] = tsym
        _log(f"📌 FUT proxy: {tsym}")
        return token, tsym
    except Exception as e:
        _log(f"⚠ FUT resolver: {e}")
        return None, None

FUT_TOKEN, FUT_TSYM = _resolve_fut_token()
_fut_vol_accum = 0.0

# ----------------------------
# WEBSOCKET INSTANCE
# ----------------------------
from kiteconnect import KiteTicker

ws = None
globals()["ws"] = None
_ws_lock = threading.Lock()

def _ist_today_date() -> datetime.date:
    return datetime.datetime.now(IST).date()

def _is_tick_today_and_after_open(exch_ts: datetime.datetime) -> bool:
    try:
        if not isinstance(exch_ts, datetime.datetime):
            return False
        exch_ts = _as_ist_dt(exch_ts) or exch_ts
        if exch_ts.date() != _ist_today_date():
            return False
        return exch_ts.time() >= market_open_time
    except:
        return False

def _tick_age_sec(now_dt: datetime.datetime, exch_ts: datetime.datetime) -> float:
    try:
        if not isinstance(now_dt, datetime.datetime) or not isinstance(exch_ts, datetime.datetime):
            return 9999.0
        return abs((now_dt - exch_ts).total_seconds())
    except:
        return 9999.0

def _can_lock_true_open(now_dt: datetime.datetime, exch_ts: datetime.datetime, *, max_age_sec: float = 3.0) -> bool:
    # ✅ Guard against stale ticks (reconnect) + previous-day ticks + pre-open ticks
    if not _is_tick_today_and_after_open(exch_ts):
        return False
    age = _tick_age_sec(now_dt, exch_ts)
    return age <= float(max_age_sec)

# ----------------------------
# ON_TICKS HANDLER
# ----------------------------
def on_ticks(_ws, ticks):
    global _fut_vol_accum

    noww_wall = time.time()
    globals()["ws_last_ping"] = noww_wall

    for tk in (ticks or []):
        try:
            token = int(tk.get("instrument_token") or 0)
            if token <= 0:
                continue

            now_dt = _get_current_ist()

            exch_raw = tk.get("exchange_timestamp") or tk.get("timestamp") or tk.get("last_trade_time")
            exch_ts = as_ist_dt_safe(exch_raw, now_dt) if exch_raw else now_dt
            if exch_ts is None:
                exch_ts = now_dt

            exch_ts_str = exch_ts.strftime("%H:%M:%S")

            ltp = tk.get("last_price")
            if ltp is None:
                continue
            ltp = float(ltp)

            cum_vol = float(tk.get("volume_traded") or 0)
            vol_delta = _delta_vol(token, cum_vol)

            # FUT volume proxy
            if FUT_TOKEN and token == int(FUT_TOKEN):
                _fut_vol_accum += vol_delta
                continue

            # ---------------- SPOT ----------------
            if token == SPOT_TOKEN:
                spot_state["ltp"] = ltp
                spot_state["ts"] = exch_ts
                globals()["last_spot_tick_wall"] = noww_wall

                proxy_vol = float(_fut_vol_accum)
                _fut_vol_accum = 0.0

                if callable(_update_ind):
                    try:
                        _update_ind("SPOT", exch_ts, ltp, proxy_vol, None)
                    except Exception as e:
                        tg_throttled("ind_spot_err", f"⚠ SPOT ind: {e}", 120)
                continue

            # --------------- CANDIDATE QUOTES (cache for hot-swap) ---------------
            try:
                cand_tokens = set(globals().get("CANDIDATE_TOKENS", []) or [])
                if token in cand_tokens:
                    q = tk.get("last_price")
                    bid = None; ask = None
                    dpth = tk.get("depth") or {}
                    try:
                        bb = (dpth.get("buy") or [{}])[0]
                        ba = (dpth.get("sell") or [{}])[0]
                        bid = bb.get("price")
                        ask = ba.get("price")
                    except:
                        if FAIL_FAST: raise
                        pass
                    spr = None
                    try:
                        if bid is not None and ask is not None:
                            spr = float(ask) - float(bid)
                    except:
                        spr = None
                    cqc = globals().get("candidate_quote_cache", {}) or {}
                    ent = cqc.get(int(token), {}) or {}
                    ent.update({
                        "ltp": float(q) if q is not None else ent.get("ltp"),
                        "bid": float(bid) if bid is not None else ent.get("bid"),
                        "ask": float(ask) if ask is not None else ent.get("ask"),
                        "spread": float(spr) if spr is not None else ent.get("spread"),
                        "oi": int(tk.get("oi")) if tk.get("oi") is not None else ent.get("oi"),
                        "ts_wall": noww_wall})
                    # delta hook: if you compute/pull delta elsewhere, write into ent["delta"]
                    cqc[int(token)] = ent
                    globals()["candidate_quote_cache"] = cqc
            except:
                if FAIL_FAST: raise
                pass

            # --------------- OPTIONS ---------------
            if token == CE_TOKEN:
                leg = "CE"
                sym = CE_TSYM
                globals()["last_ce_tick_wall"] = noww_wall
            elif token == PE_TOKEN:
                leg = "PE"
                sym = PE_TSYM
                globals()["last_pe_tick_wall"] = noww_wall
            else:
                continue

            st = global_state[leg]
            prev_ltp = st.get("ltp")

            st["ltp"] = ltp
            st["vol_traded"] = cum_vol
            st["vol_delta"] = vol_delta

            bid, ask, spr = _best_bid_ask_from_tick(tk)
            st["best_bid"] = bid
            st["best_ask"] = ask
            st["spread"] = spr

            # TRUE OPEN via WS day OHLC open (must match Kite "Open")
            try:
                # ✅ Ignore pre-open / previous-day / stale ticks
                if not _is_tick_today_and_after_open(exch_ts):
                    pass
                else:
                    # PRIMARY: day OHLC open from WS tick["ohlc"]["open"]
                    if globals().get("TRUE_OPEN_USE_WS_OHLC_OPEN", True) and (not st.get("open_locked")):
                        ohlc = tk.get("ohlc") or {}
                        op = ohlc.get("open")
                        if op is not None and float(op) > 0 and _can_lock_true_open(now_dt, exch_ts, max_age_sec=3.0):
                            st["ws_ohlc_open"] = float(op)
                            fn_apply = globals().get("apply_true_open")
                            fn_save  = globals().get("maybe_save_true_open_if_ready")
                            if callable(fn_apply):
                                fn_apply(leg, float(op), exch_ts_str, "WS_OHLC_OPEN_DAY")
                            # lock (do not overwrite later)
                            st["open_locked"] = True
                            st["open_src"] = "WS_OHLC_OPEN_DAY"
                            st["open_lock_time"] = exch_ts
                            if callable(fn_save):
                                fn_save()

                    # FALLBACK (only if still not locked and very late): use LTP with strict freshness
                    if (not st.get("open_locked")) and _can_lock_true_open(now_dt, exch_ts, max_age_sec=3.0):
                        # allow only after REST has had time (~30s) to fetch day open
                        if exch_ts.time() >= (datetime.time(9, 15, 30)):
                            fn_apply = globals().get("apply_true_open")
                            fn_save  = globals().get("maybe_save_true_open_if_ready")
                            if callable(fn_apply):
                                fn_apply(leg, float(ltp), exch_ts_str, "WS_LTP_FALLBACK")
                            st["open_locked"] = True
                            st["open_src"] = "WS_LTP_FALLBACK"
                            st["open_lock_time"] = exch_ts
                            if callable(fn_save):
                                fn_save()

                # announce once when both CE/PE are locked
                try:
                    if (global_state["CE"].get("open_locked") and global_state["PE"].get("open_locked")
                        and not globals().get("_OPEN_LOCK_ANNOUNCED", False)):
                        globals()["_OPEN_LOCK_ANNOUNCED"] = True
                        ce = global_state["CE"].get("open")
                        pe = global_state["PE"].get("open")
                        ce_src = global_state["CE"].get("open_src") or global_state["CE"].get("open_source")
                        pe_src = global_state["PE"].get("open_src") or global_state["PE"].get("open_source")
                        lt = global_state["CE"].get("open_lock_time") or global_state["PE"].get("open_lock_time")
                        lt_str = lt.strftime("%Y-%m-%d %H:%M:%S") if isinstance(lt, datetime.datetime) else str(lt)
                        msg = f"✅ TRUE OPEN LOCKED | CE={ce} ({ce_src}) | PE={pe} ({pe_src}) | lock_time={lt_str} IST"
                        print(msg)
                        tg_throttled("true_open_locked", msg, 3600)
                except:
                    if FAIL_FAST: raise
                    pass
            except:
                if FAIL_FAST: raise
                pass

            # ✅ Update indicators (Block-2)
            if callable(_update_ind):
                try:
                    _update_ind(leg, exch_ts, ltp, vol_delta, tk.get("depth"))
                except:
                    if FAIL_FAST: raise
                    pass

            # Push to engine queue
            try:
                tick_q.put_nowait(("OPT", exch_ts, leg, prev_ltp, ltp, tk))
            except queue.Full:
                globals()["tickq_drops"] = int(globals().get("tickq_drops", 0) or 0) + 1
            except:
                if FAIL_FAST: raise
                pass

            # ✅ CSV row (includes candle/atr fields)
            try:
                spot_ltp = spot_state.get("ltp")
                spot_vwap = spot_state.get("vwap")

                def _in_leg(obj):
                    try:
                        return bool(getattr(obj, "in_trade", False)), getattr(obj, "leg", None)
                    except:
                        return False, None

                # Strategy states (only GL/GB/N19)
                gls = globals().get("gl_state")
                gbs = globals().get("gb_state")
                n19s = globals().get("n19_state")

                gl_in, gl_leg = _in_leg(gls)
                gb_in, gb_leg = _in_leg(gbs)
                n19_in, n19_leg = _in_leg(n19s)

                row = [
                    now_dt.strftime("%Y-%m-%d"),
                    now_dt.strftime("%H:%M:%S"),
                    exch_ts_str,
                    leg, sym, token,
                    ltp, bid, ask, spr,
                    cum_vol, vol_delta,
                    spot_ltp, spot_vwap,
                    gl_in, gl_leg,
                    gb_in, gb_leg,
                    n19_in, n19_leg,
                    st.get("imb_ratio"), st.get("imb_signal"), (st.get("vwap") if st.get("vwap") is not None else st.get("vwap_1m")),
                    st.get("rsi3_15m"), st.get("vwap_15m"), st.get("imb15_cur"), st.get("imb_avg20_15m"),
                    st.get("atr"), st.get("bar_high"), st.get("bar_low"), st.get("bar_close"),
                    RUN_ID
                ]

                with tick_buffer_lock:
                    tick_buffer.append(row)
            except:
                if FAIL_FAST: raise
                pass

        except Exception as e:
            globals()["errors"] = int(globals().get("errors", 0) or 0) + 1

# ----------------------------
# WEBSOCKET CALLBACKS
# ----------------------------
def on_connect(_ws, response):
    globals()["ws_status"] = "CONNECTED"
    globals()["ws_last_ping"] = time.time()

    tokens = [SPOT_TOKEN] + list(globals().get('CANDIDATE_TOKENS', []) or [])
    if FUT_TOKEN:
        tokens.append(int(FUT_TOKEN))

    try:
        tokens = list(dict.fromkeys([int(t) for t in tokens if t]))
        _ws.subscribe(tokens)
        _ws.set_mode(_ws.MODE_FULL, tokens)
        _log(f"✅ WS CONNECTED | Subscribed {len(tokens)} tokens")
        tg_throttled("ws_conn", f"✅ WS connected", 30)
    except Exception as e:
        _log(f"⚠ Subscribe error: {e}")

def on_close(_ws, code, reason):
    globals()["ws_status"] = f"CLOSED:{code}"
    _log(f"⚠ WS closed: {code} - {reason}")
    tg_throttled("ws_close", f"⚠ WS closed: {code}", 120)

def on_error(_ws, code, reason):
    globals()["ws_status"] = f"ERROR:{code}"
    globals()["errors"] = int(globals().get("errors", 0) or 0) + 1
    _log(f"❌ WS error: {code} - {reason}")
    tg_throttled("ws_err", f"❌ WS error: {code}", 120)

def on_reconnect(_ws, attempts):
    globals()["ws_status"] = f"RECONNECTING:{attempts}"
    globals()["reconnects"] = int(globals().get("reconnects", 0) or 0) + 1
    _log(f"🔄 Reconnecting... attempt {attempts}")

# ----------------------------
# ENGINE LOOP
# ----------------------------
SQUARE_OFF_TIME = globals().get("SQUARE_OFF_TIME", datetime.time(15, 15))

import traceback

# disable flags (so one broken strategy doesn't spam & block others)
globals().setdefault("_STRAT_DISABLED", {})

def _safe_call(name: str, fn, *args):
    # Strategy execution wrapper:
    # - prevents Telegram spam on repeating errors
    # - can auto-disable a strategy if it keeps erroring to keep the bot healthy
    if globals()["_STRAT_DISABLED"].get(name):
        return None

    if not callable(fn):
        # This usually happens if cells were run out-of-order and the strategy block isn't loaded yet.
        tg_throttled(f"missing_{name}", f"⚠ {name} skipped: handler not loaded (run blocks in order / restart & Run all).", 300)
        return None

    try:
        return fn(*args)

    except RecursionError as e:
        # Disable this strategy for the rest of the session to keep the bot healthy
        globals()["_STRAT_DISABLED"][name] = True
        tg_throttled(f"rec_{name}", f"🛑 {name} DISABLED (RecursionError): {e}", 600)
        return None

    except Exception as e:
        # Repeating error guard (prevents TG spam)
        if not hasattr(_safe_call, "_err_mem"):
            _safe_call._err_mem = {}
            _safe_call._err_lock = threading.Lock()

        err_sig = f"{type(e).__name__}:{str(e)[:120]}"
        noww = time.time()
        with _safe_call._err_lock:
            rec = _safe_call._err_mem.get(name, {"sig": None, "cnt": 0, "since": noww, "last": 0.0})
            # reset window if different error or window expired
            if rec["sig"] != err_sig or (noww - rec["since"]) > 300:
                rec = {"sig": err_sig, "cnt": 0, "since": noww, "last": 0.0}
            rec["cnt"] += 1
            rec["last"] = noww
            _safe_call._err_mem[name] = rec

        # Special hint for the most common GB crash
        if name == "GB" and ("compute_buy_targets" in str(e)):
            tg_throttled(
                "gb_missing_compute_buy_targets",
                "⛔ GB blocked: compute_buy_targets is not loaded. Fix: Runtime → Restart runtime → Run all (do not skip Block-3).",
                900
            )

        # After 3 errors in 5 minutes, disable to avoid runaway spam and protect other strategies
        if rec["cnt"] >= 3:
            globals()["_STRAT_DISABLED"][name] = True
            tg_throttled(f"disable_{name}", f"🛑 {name} DISABLED after repeated errors: {err_sig}", 900)
            return None

        tb = traceback.format_exc(limit=6)
        tg_throttled(f"err_{name}", f"⚠ {name} ERROR: {err_sig}\n{tb}", 600)
        return None

def _squareoff_all():
    ts = _get_current_ist()
    _safe_call("GL", globals().get("gl_exit"), ts, "SQUARE_OFF_1515")
    _safe_call("GB", globals().get("gb_exit"), ts, "SQUARE_OFF_1515")
    _safe_call("N19", globals().get("n19_exit"), ts, "SQUARE_OFF_1515")
    _log("✅ 15:15 square-off done")
    tg_throttled("sq_done", "✅ 15:15 square-off complete", 60)

def engine_loop():
    _log("🧠 Engine started")
    did_squareoff = False

    while not block_stop.is_set():
        try:
            globals()["engine_last_wall"] = time.time()

            now_dt = _get_current_ist()
            if not did_squareoff and now_dt.time() >= SQUARE_OFF_TIME:
                _squareoff_all()
                did_squareoff = True

            try:
                item = tick_q.get(timeout=0.5)
            except queue.Empty:
                continue

            if not item or item[0] != "OPT":
                continue

            _, ts, leg, prev_ltp, ltp, raw = item

            _safe_call("GL", globals().get("gl_on_tick"), ts, leg, prev_ltp, ltp)
            _safe_call("GB", globals().get("gb_on_tick"), ts, leg, prev_ltp, ltp)
            _safe_call("N19", globals().get("n19_on_tick"), ts, leg, ltp)

        except Exception as e:
            _log(f"⚠ Engine error: {e}")
            time.sleep(0.5)

if not globals().get("_ENGINE_STARTED", False):
    globals()["_ENGINE_STARTED"] = True
    t = threading.Thread(target=engine_loop, daemon=True, name="Engine")
    t.start()
    _log("✅ Engine thread started")

# ----------------------------
# WS MANAGEMENT
# ----------------------------
_ws_lock = threading.Lock()

def stop_ws():
    global ws
    with _ws_lock:
        w = globals().get("ws")
        if w is not None:
            try:
                w.close()
            except:
                if FAIL_FAST: raise
                pass
        globals()["ws"] = None
        ws = None
        globals()["ws_status"] = "STOPPED"
    _log("🔌 WS stopped")

def start_ws():
    global ws
    with _ws_lock:
        if globals().get("ws") is not None:
            _log("ℹ WS already exists")
            return True

        nowt = _get_current_ist().time()
        if not _within_ws_hours(nowt):
            globals()["ws_status"] = "MARKET_CLOSED"
            _log(f"ℹ Outside market hours ({nowt.strftime('%H:%M')})")
            return False

        try:
            _log("🔌 Creating KiteTicker...")
            ws = KiteTicker(API_KEY, ACCESS_TOKEN)
            globals()["ws"] = ws
        except Exception as e:
            globals()["ws_status"] = "CREATE_FAILED"
            _log(f"❌ KiteTicker failed: {e}")
            return False

        try:
            ws.enable_reconnect(reconnect=True, reconnect_max_tries=100, reconnect_max_delay=30)
        except Exception as e:
            _log(f"⚠ enable_reconnect: {e}")

        ws.on_ticks = on_ticks
        ws.on_connect = on_connect
        ws.on_close = on_close
        ws.on_error = on_error
        ws.on_reconnect = on_reconnect

        try:
            globals()["ws_status"] = "CONNECTING"
            _log("🔌 Connecting to Kite...")
            ws.connect(threaded=True)
        except Exception as e:
            globals()["ws_status"] = "CONNECT_FAILED"
            _log(f"❌ Connect failed: {e}")
            return False

    for i in range(15):
        time.sleep(1)
        status = str(globals().get("ws_status", ""))
        if status == "CONNECTED":
            _log("✅ WS connected successfully!")
            return True
        if block_stop.is_set():
            return False
        if i == 7:
            _log(f"⏳ Still connecting... status={status}")

    _log(f"⚠ Connection timeout. Status: {globals().get('ws_status')}")
    return False

globals()["start_ws"] = start_ws
globals()["stop_ws"] = stop_ws

# ----------------------------
# HEALTH MONITOR
# ----------------------------
def _ws_health_monitor():
    consecutive_failures = 0
    max_failures = 3

    while not block_stop.is_set():
        time.sleep(20)
        try:
            nowt = _get_current_ist().time()
            if not _within_ws_hours(nowt):
                consecutive_failures = 0
                continue

            ws_status = str(globals().get("ws_status", ""))
            last_ping = float(globals().get("ws_last_ping", 0) or 0)
            ping_age = time.time() - last_ping if last_ping > 0 else 999

            is_connected = "CONNECTED" in ws_status.upper()
            is_receiving = ping_age < 45

            if is_connected and is_receiving:
                consecutive_failures = 0
                continue

            consecutive_failures += 1
            _log(f"⚠ Health check failed ({consecutive_failures}/{max_failures}) | status={ws_status} ping_age={ping_age:.0f}s")

            if consecutive_failures >= max_failures:
                _log("🔄 Restarting WS...")
                tg_throttled("ws_restart", f"🔄 Auto-restart (ping_age={ping_age:.0f}s)", 300)
                try:
                    stop_ws()
                    time.sleep(3)
                    if start_ws():
                        consecutive_failures = 0
                        _log("✅ WS restarted successfully")
                    else:
                        _log("⚠ WS restart failed, will retry...")
                except Exception as e:
                    _log(f"❌ Restart error: {e}")

        except Exception as e:
            _log(f"⚠ Health monitor error: {e}")

if not globals().get("_WS_HEALTH_STARTED", False):
    globals()["_WS_HEALTH_STARTED"] = True
    t = threading.Thread(target=_ws_health_monitor, daemon=True, name="WSHealth")
    t.start()
    _log("✅ Health monitor started")

# ----------------------------
# MARKET CLOSE HANDLER
# ----------------------------
def _market_close_handler():
    while not block_stop.is_set():
        time.sleep(60)
        nowt = _get_current_ist().time()
        if nowt > WS_ALLOW_END:
            _log("ℹ Market closed. Stopping WS.")
            try:
                stop_ws()
            except:
                if FAIL_FAST: raise
                pass
            tg_throttled("mkt_close", "ℹ Market closed", 600)
            return

if not globals().get("_MKT_CLOSE_STARTED", False):
    globals()["_MKT_CLOSE_STARTED"] = True
    t = threading.Thread(target=_market_close_handler, daemon=True, name="MktClose")
    t.start()

# ----------------------------
# KEEPALIVE (optional)
# ----------------------------
def _colab_keepalive():
    while not block_stop.is_set():
        time.sleep(300)
        try:
            _ = sum(range(1000))
        except:
            if FAIL_FAST: raise
            pass

if not globals().get("_KEEPALIVE_STARTED", False):
    globals()["_KEEPALIVE_STARTED"] = True
    t = threading.Thread(target=_colab_keepalive, daemon=True, name="KeepAlive")
    t.start()

# ----------------------------
# SMART START
# ----------------------------
def smart_start():
    now_dt = _get_current_ist()
    nowt = now_dt.time()
    _log(f"⏰ Current time: {now_dt.strftime('%Y-%m-%d %H:%M:%S')} IST")

    if nowt < datetime.time(8, 55):
        _log("⏳ Waiting for market start (08:55 IST)...")
        while _get_current_ist().time() < datetime.time(8, 55):
            if block_stop.is_set():
                return False
            time.sleep(10)
        _log("✅ Market opening soon. Starting WS...")

    elif nowt > datetime.time(15, 35):
        globals()["ws_status"] = "MARKET_CLOSED"
        _log("ℹ Market is closed. WS will not start.")
        return False

    return start_ws()

# ----------------------------
# STATUS CHECK
# ----------------------------
def check_status():
    now = _get_current_ist()
    ws_status = globals().get("ws_status", "UNKNOWN")
    last_spot = float(globals().get("last_spot_tick_wall", 0) or 0)
    last_ping = float(globals().get("ws_last_ping", 0) or 0)
    errors = globals().get("errors", 0)
    reconnects = globals().get("reconnects", 0)
    ticks = globals().get("ticks_saved", 0)

    spot_age = f"{time.time() - last_spot:.0f}s" if last_spot > 0 else "N/A"
    ping_age = f"{time.time() - last_ping:.0f}s" if last_ping > 0 else "N/A"

    print("=" * 50)
    print(f"⏰ Time: {now.strftime('%H:%M:%S')} IST")
    print(f"📡 WS Status: {ws_status}")
    print(f"📊 Last Spot: {spot_age} ago")
    print(f"🔔 Last Ping: {ping_age} ago")
    print(f"❌ Errors: {errors}")
    print(f"🔄 Reconnects: {reconnects}")
    print(f"📝 Ticks Saved: {ticks}")
    print(f"💰 Spot LTP: {spot_state.get('ltp', 'N/A')}")
    print("=" * 50)

globals()["check_status"] = check_status

# ----------------------------
# START EVERYTHING
# ----------------------------
_log("=" * 50)
_log("🚀 BLOCK 5 - UPDATED (N19 candle+ATR compatible)")
_log("=" * 50)

result = smart_start()

if result:
    _log("✅ Block-5 ready! WS connected.")
else:
    ws_status = str(globals().get("ws_status", ""))
    if "MARKET_CLOSED" in ws_status:
        _log("ℹ Block-5 ready (market closed mode)")
    else:
        _log("⚠ Block-5 ready but WS not connected. Will auto-retry.")

_log("💡 TIP: Run check_status() anytime to see current state")
_log("💡 TIP: Run start_ws() to manually reconnect")
_log("=" * 50)

# ============================================================
# BLOCK 6/6 — DASHBOARD ✅ (SHIV — GL/GB/N19 ONLY — COMPAT)
# ============================================================
#
# ✅ Optimized for Google Colab
# ✅ Live Spot + CE/PE tiles with staleness dots
# ✅ Shows GL / GB / N19 state only (NO N1 / NO N5)
# ✅ PnL tracking (from global_summary)
# ✅ Shows Option ATR7 points: global_state["CE/PE"]["atr"]
# ✅ Includes shutdown(), check_status()
# ✅ Safe to re-run (won’t start multiple dashboard loops)
# ============================================================

from IPython.display import display, HTML, update_display
import time, datetime, threading, atexit, html as _html, traceback

IST = globals().get("IST") or datetime.timezone(datetime.timedelta(hours=5, minutes=30))

DASH_REFRESH_SEC = float(globals().get("DASH_REFRESH_SEC", 0.8))
DASH_DISPLAY_ID = "SHIV_N19_DASH"

def _G(name, default=None):
    return globals().get(name, default)

def _get_current_ist():
    return datetime.datetime.now(IST)

# ----------------------------
# STOP EVENT
# ----------------------------
def _ensure_event(name="block_stop"):
    ev = _G(name)
    if ev is None or not isinstance(ev, threading.Event):
        ev = threading.Event()
        globals()[name] = ev
    return ev

STOP_EVENT = _ensure_event("block_stop")
globals()["STOP_EVENT"] = STOP_EVENT

# ----------------------------
# SAFE FORMATTERS
# ----------------------------
def _to_float(x):
    if x is None:
        return None
    try:
        return float(x)
    except:
        return None

def fmt(x, nd=2, dash="—"):
    v = _to_float(x)
    if v is None:
        return dash
    return f"{v:.{nd}f}"

def fmt2(x, dash="—"):
    return fmt(x, 2, dash)

def fmt0(x, dash="—"):
    v = _to_float(x)
    if v is None:
        return dash
    return f"{v:.0f}"

def fmt_int(x, dash="—"):
    if x is None:
        return dash
    try:
        return f"{int(x):}"
    except:
        return dash

def fmt_money(x, dash="—"):
    v = _to_float(x)
    if v is None:
        return dash
    sign = "+" if v >= 0 else ""
    return f"₹{sign}{v:,.0f}"

def fmt_time(dt, dash="—"):
    if dt is None:
        return dash
    try:
        if isinstance(dt, datetime.datetime):
            return dt.strftime("%H:%M:%S")
        return str(dt)
    except:
        return dash

def esc(s):
    try:
        return _html.escape("" if s is None else str(s))
    except:
        return ""

def _split_tsym(tsym):
    s = "" if tsym is None else str(tsym)
    return s.split(":")[-1] if ":" in s else s

def _age_str(wall_time):
    if wall_time is None or wall_time <= 0:
        return "N/A"
    age = time.time() - wall_time
    if age < 0:
        age = 0
    if age < 60:
        return f"{age:.0f}s"
    return f"{age/60:.1f}m"

# ----------------------------
# LIVE INDICATOR DOT
# ----------------------------
def live_dot(age_sec):
    if age_sec is None:
        age_sec = 999
    if age_sec < 3:
        return '<span style="color:#0f0;font-size:14px;">●</span>'
    elif age_sec < 10:
        return '<span style="color:#ff0;font-size:14px;">●</span>'
    else:
        return '<span style="color:#f44;font-size:14px;">●</span>'

def status_badge(is_active, active_text="LIVE", idle_text="IDLE"):
    if is_active:
        return f'<span style="background:#0a3;color:#fff;padding:2px 8px;border-radius:3px;font-size:11px;">{active_text}</span>'
    return f'<span style="background:#444;color:#888;padding:2px 8px;border-radius:3px;font-size:11px;">{idle_text}</span>'

# ============================================================
# DASHBOARD HTML GENERATOR
# ============================================================
def dashboard_html():
    """Generate dashboard HTML (GL/GB/N19 only)"""
    try:
        now = _get_current_ist()
        now_wall = time.time()

        # ----------------------------
        # SYSTEM STATE
        # ----------------------------
        ws_status = _G("ws_status", "UNKNOWN")
        reconnects = _G("reconnects", 0)
        errors = _G("errors", 0)
        ticks_saved = _G("ticks_saved", 0)
        tickq_drops = _G("tickq_drops", 0)

        ws_connected = "CONNECTED" in str(ws_status).upper()
        ws_color = "#0f0" if ws_connected else "#f44"

        # ----------------------------
        # MARKET DATA
        # ----------------------------
        spot_state = _G("spot_state", {}) or {}
        global_state = _G("global_state", {}) or {}

        spot_ltp = spot_state.get("ltp")
        spot_vwap = spot_state.get("vwap")
        spot_atr = spot_state.get("atr14_1m") or spot_state.get("atr_last")
        spot_vold = spot_state.get('vol_delta')

        # Tick ages (wall clock)
        last_spot_tick = float(_G("last_spot_tick_wall", 0) or 0)
        last_ce_tick = float(_G("last_ce_tick_wall", 0) or 0)
        last_pe_tick = float(_G("last_pe_tick_wall", 0) or 0)

        spot_age = (now_wall - last_spot_tick) if last_spot_tick > 0 else 999
        ce_age = (now_wall - last_ce_tick) if last_ce_tick > 0 else 999
        pe_age = (now_wall - last_pe_tick) if last_pe_tick > 0 else 999

        # ----------------------------
        # OPTION DATA
        # ----------------------------
        ce_state = global_state.get("CE", {}) or {}
        pe_state = global_state.get("PE", {}) or {}

        ce_ltp = ce_state.get("ltp")
        pe_ltp = pe_state.get("ltp")
        ce_open = ce_state.get("open")
        pe_open = pe_state.get("open")
        ce_bid = ce_state.get("best_bid")
        ce_ask = ce_state.get("best_ask")
        pe_bid = pe_state.get("best_bid")
        pe_ask = pe_state.get("best_ask")
        ce_spread = ce_state.get("spread")
        pe_spread = pe_state.get("spread")

        # Option ATR7 (points) from Block-2
        ce_atr7 = ce_state.get("atr")
        pe_atr7 = pe_state.get("atr")

        ce_vwap1m = ce_state.get('vwap_15m')
        pe_vwap1m = pe_state.get('vwap_15m')
        if ce_vwap1m is None: ce_vwap1m = ce_state.get('vwap_1m')
        if pe_vwap1m is None: pe_vwap1m = pe_state.get('vwap_1m')
        ce_rsi3 = ce_state.get('rsi3_15m')
        pe_rsi3 = pe_state.get('rsi3_15m')
        ce_imb = ce_state.get('imb_ratio')
        pe_imb = pe_state.get('imb_ratio')
        ce_imb_avg20 = ce_state.get('imb_avg20_15m')
        pe_imb_avg20 = pe_state.get('imb_avg20_15m')
        ce_imb15 = ce_state.get('imb15_cur')
        pe_imb15 = pe_state.get('imb15_cur')
        ce_vold = ce_state.get('vol_delta')
        pe_vold = pe_state.get('vol_delta')

        def pct_chg(ltp, opn):
            l = _to_float(ltp)
            o = _to_float(opn)
            if l is None or o is None or o == 0:
                return None
            return ((l - o) / o) * 100.0

        ce_chg = pct_chg(ce_ltp, ce_open)
        pe_chg = pct_chg(pe_ltp, pe_open)

        def chg_color(chg):
            if chg is None:
                return "#888"
            return "#0f0" if chg >= 0 else "#f44"

        # ----------------------------
        # STRATEGY STATES
        # ----------------------------
        gls = _G("gl_state")
        gl_in = bool(getattr(gls, "in_trade", False)) if gls else False
        gl_leg = getattr(gls, "leg", None) if gls else None
        gl_entry = getattr(gls, "entry", None) if gls else None
        gl_sl = getattr(gls, "sl", None) if gls else None
        gl_pref = getattr(gls, "execution_preference", None) if gls else None

        gbs = _G("gb_state")
        gb_in = bool(getattr(gbs, "in_trade", False)) if gbs else False
        gb_leg = getattr(gbs, "leg", None) if gbs else None
        gb_entry = getattr(gbs, "entry", None) if gbs else None
        gb_sl = getattr(gbs, "sl", None) if gbs else None

        n19s = _G("n19_state")
        n19_in = bool(getattr(n19s, "in_trade", False)) if n19s else False
        n19_leg = getattr(n19s, "leg", None) if n19s else None
        n19_entry = getattr(n19s, "entry", None) if n19s else None
        n19_sl = getattr(n19s, "sl", None) if n19s else None

        # N19 phases (if supported)
        n19_ce_phase = "IDLE"
        n19_pe_phase = "IDLE"
        if n19s and hasattr(n19s, "leg_state"):
            ls = n19s.leg_state or {}
            try:
                if "CE" in ls and hasattr(ls["CE"], "phase"):
                    n19_ce_phase = ls["CE"].phase or "IDLE"
                if "PE" in ls and hasattr(ls["PE"], "phase"):
                    n19_pe_phase = ls["PE"].phase or "IDLE"
            except:
                if FAIL_FAST: raise
                pass

        # ----------------------------
        # PNL
        # ----------------------------
        summary = _G("global_summary", {}) or {}
        gl_pnl  = _to_float(summary.get("GL", 0)) or 0
        gb_pnl  = _to_float(summary.get("GB", 0)) or 0
        n19_pnl = _to_float(summary.get("N19", 0)) or 0
        total_pnl = gl_pnl + gb_pnl + n19_pnl

        # ----------------------------
        # SIGNALS
        # ----------------------------
        get_sig = _G("get_last_signal_text")
        gl_sig = get_sig("GL") if callable(get_sig) else "—"
        gb_sig = get_sig("GB") if callable(get_sig) else "—"
        n19_sig = get_sig("N19") if callable(get_sig) else "—"

        # ----------------------------
        # INSTRUMENT INFO
        # ----------------------------
        UNDERLYING = esc(_G("UNDERLYING", ""))
        CE_STRIKE = _G("CE_STRIKE", "")
        PE_STRIKE = _G("PE_STRIKE", "")
        CE_TSYM = esc(_split_tsym(_G("CE_TSYM", "")))
        PE_TSYM = esc(_split_tsym(_G("PE_TSYM", "")))
        EXPIRY_DATE = _G("EXPIRY_DATE", "")

        # ----------------------------
        # FULL HTML
        # ----------------------------
        return f"""
<div style="font-family:'Courier New',monospace;background:#071b14;color:#0f0;padding:15px;border-radius:8px;max-width:1400px;font-size:12px;">

  <!-- HEADER -->
  <div style="display:flex;justify-content:space-between;align-items:center;margin-bottom:15px;border-bottom:1px solid #0f0;padding-bottom:10px;">
    <div>
      <h1 style="color:#0ff;margin:0;font-size:22px;">🚀 SHIV DASHBOARD</h1>
      <div style="font-size:11px;color:#888;margin-top:4px;">{UNDERLYING} | {EXPIRY_DATE} | {CE_TSYM} / {PE_TSYM}</div>
    </div>
    <div style="text-align:right;">
      <div style="font-size:11px;color:#888;">{now.strftime('%Y-%m-%d')}</div>
      <div style="font-size:18px;color:#0f0;font-weight:bold;">{now.strftime('%H:%M:%S')} IST</div>
    </div>
  </div>

  <!-- LIVE MARKET -->
  <div style="background:#0a2118;padding:12px;border-radius:6px;margin-bottom:10px;border:1px solid #0f0;">
    <h3 style="margin:0 0 10px 0;color:#0ff;font-size:13px;">💹 LIVE MARKET</h3>

    <div style="display:grid;grid-template-columns:1fr 2fr 1fr 1fr;gap:10px;align-items:center;background:#0d2d1f;padding:10px;border-radius:4px;margin-bottom:8px;">
      <div>
        <div style="font-size:11px;color:#888;">SPOT</div>
        <div style="font-size:10px;color:#666;">{UNDERLYING}</div>
      </div>
      <div style="text-align:center;">
        <span style="font-size:22px;color:#0f0;font-weight:bold;">{fmt2(spot_ltp)}</span>
      </div>
      <div style="text-align:center;font-size:10px;">
        <div style="color:#888;">VWAP</div>
        <div style="color:#0ff;">{fmt2(spot_vwap)}</div>
        <div style="margin-top:2px;color:#888;">ΔV</div>
        <div style="color:#0ff;">{fmt2(spot_vold)}</div>
      </div>
      <div style="text-align:right;">
        {live_dot(spot_age)} <span style="font-size:10px;color:#666;">{_age_str(last_spot_tick)}</span>
      </div>
    </div>

    <div style="display:grid;grid-template-columns:1fr 1fr;gap:10px;">
      <div style="background:#0d2d1f;padding:10px;border-radius:4px;border-left:3px solid {'#0f0' if (ce_chg or 0) >= 0 else '#f44'};">
        <div style="display:flex;justify-content:space-between;align-items:center;">
          <div>
            <div style="font-size:11px;color:#888;">CE {CE_STRIKE}</div>
            <div style="font-size:10px;color:#666;">{CE_TSYM}</div>
          </div>
          <div style="text-align:right;">{live_dot(ce_age)}</div>
        </div>
        <div style="text-align:center;margin:8px 0;">
          <span style="font-size:20px;color:#0f0;font-weight:bold;">{fmt2(ce_ltp)}</span>
          <span style="font-size:12px;color:{chg_color(ce_chg)};margin-left:8px;">{f'{ce_chg:+.1f}%' if ce_chg is not None else '—'}</span>
        </div>
        <div style="display:flex;justify-content:space-between;font-size:10px;color:#888;">
          <span>Bid: <span style="color:#0f0;">{fmt2(ce_bid)}</span></span>
          <span>Ask: <span style="color:#f44;">{fmt2(ce_ask)}</span></span>
          <span>Spr: {fmt2(ce_spread)}</span>
        </div>
        <div style="margin-top:6px;font-size:10px;color:#888;">ATR7(opt): <span style="color:#0ff;">{fmt2(ce_atr7)}</span></div>
        <div style="margin-top:4px;display:flex;justify-content:space-between;font-size:10px;color:#888;">VWAP:<span style="color:#0ff;">{fmt2(ce_vwap1m)}</span> IMB15/AVG20:<span style="color:#0ff;">{fmt2(ce_imb15)}/{fmt2(ce_imb_avg20)}</span> ΔV:<span style="color:#0ff;">{fmt2(ce_vold)}</span></div>

      </div>

      <div style="background:#0d2d1f;padding:10px;border-radius:4px;border-left:3px solid {'#0f0' if (pe_chg or 0) >= 0 else '#f44'};">
        <div style="display:flex;justify-content:space-between;align-items:center;">
          <div>
            <div style="font-size:11px;color:#888;">PE {PE_STRIKE}</div>
            <div style="font-size:10px;color:#666;">{PE_TSYM}</div>
          </div>
          <div style="text-align:right;">{live_dot(pe_age)}</div>
        </div>
        <div style="text-align:center;margin:8px 0;">
          <span style="font-size:20px;color:#0f0;font-weight:bold;">{fmt2(pe_ltp)}</span>
          <span style="font-size:12px;color:{chg_color(pe_chg)};margin-left:8px;">{f'{pe_chg:+.1f}%' if pe_chg is not None else '—'}</span>
        </div>
        <div style="display:flex;justify-content:space-between;font-size:10px;color:#888;">
          <span>Bid: <span style="color:#0f0;">{fmt2(pe_bid)}</span></span>
          <span>Ask: <span style="color:#f44;">{fmt2(pe_ask)}</span></span>
          <span>Spr: {fmt2(pe_spread)}</span>
        </div>
        <div style="margin-top:6px;font-size:10px;color:#888;">ATR7(opt): <span style="color:#0ff;">{fmt2(pe_atr7)}</span></div>
        <div style="margin-top:4px;display:flex;justify-content:space-between;font-size:10px;color:#888;">VWAP:<span style="color:#0ff;">{fmt2(pe_vwap1m)}</span> IMB15/AVG20:<span style="color:#0ff;">{fmt2(pe_imb15)}/{fmt2(pe_imb_avg20)}</span> ΔV:<span style="color:#0ff;">{fmt2(pe_vold)}</span></div>

      </div>
    </div>
  </div>

  <!-- STRATEGIES -->
  <div style="background:#0a2118;padding:12px;border-radius:6px;margin-bottom:10px;border:1px solid #0f0;">
    <h3 style="margin:0 0 10px 0;color:#0ff;font-size:13px;">🎯 STRATEGIES</h3>

    <table style="width:100%;border-collapse:collapse;font-size:11px;">
      <tr style="border-bottom:1px solid #333;color:#888;">
        <th style="text-align:left;padding:6px;">Strategy</th>
        <th style="text-align:center;padding:6px;">Status</th>
        <th style="text-align:center;padding:6px;">Leg</th>
        <th style="text-align:right;padding:6px;">Entry</th>
        <th style="text-align:right;padding:6px;">SL</th>
        <th style="text-align:right;padding:6px;">Info</th>
        <th style="text-align:right;padding:6px;">PnL</th>
      </tr>

      <tr style="background:{'#0d3d2f' if gl_in else 'transparent'};">
        <td style="padding:6px;font-weight:bold;color:#0ff;">GL (VWAP+IMB)</td>
        <td style="text-align:center;padding:6px;">{status_badge(gl_in)}</td>
        <td style="text-align:center;padding:6px;">{gl_leg or "—"}</td>
        <td style="text-align:right;padding:6px;">{fmt2(gl_entry) if gl_in else "—"}</td>
        <td style="text-align:right;padding:6px;color:#f44;">{fmt2(gl_sl) if gl_in else "—"}</td>
        <td style="text-align:right;padding:6px;font-size:10px;">pref:{esc(gl_pref) if gl_pref else "—"} | {esc(gl_sig)}</td>
        <td style="text-align:right;padding:6px;color:{'#0f0' if gl_pnl >= 0 else '#f44'};">{fmt_money(gl_pnl)}</td>
      </tr>

      <tr style="background:{'#0d3d2f' if gb_in else 'transparent'};">
        <td style="padding:6px;font-weight:bold;color:#0ff;">GB (VWAP)</td>
        <td style="text-align:center;padding:6px;">{status_badge(gb_in)}</td>
        <td style="text-align:center;padding:6px;">{gb_leg or "—"}</td>
        <td style="text-align:right;padding:6px;">{fmt2(gb_entry) if gb_in else "—"}</td>
        <td style="text-align:right;padding:6px;color:#f44;">{fmt2(gb_sl) if gb_in else "—"}</td>
        <td style="text-align:right;padding:6px;font-size:10px;">{esc(gb_sig)}</td>
        <td style="text-align:right;padding:6px;color:{'#0f0' if gb_pnl >= 0 else '#f44'};">{fmt_money(gb_pnl)}</td>
      </tr>

      <tr style="background:{'#0d3d2f' if n19_in else 'transparent'};">
        <td style="padding:6px;font-weight:bold;color:#0ff;">N19 (IMB)</td>
        <td style="text-align:center;padding:6px;">{status_badge(n19_in)}</td>
        <td style="text-align:center;padding:6px;">{n19_leg or "—"}</td>
        <td style="text-align:right;padding:6px;">{fmt2(n19_entry) if n19_in else "—"}</td>
        <td style="text-align:right;padding:6px;color:#f44;">{fmt2(n19_sl) if n19_in else "—"}</td>
        <td style="text-align:right;padding:6px;font-size:10px;">CE:{esc(n19_ce_phase)[:6]} PE:{esc(n19_pe_phase)[:6]} | {esc(n19_sig)}</td>
        <td style="text-align:right;padding:6px;color:{'#0f0' if n19_pnl >= 0 else '#f44'};">{fmt_money(n19_pnl)}</td>
      </tr>

      <tr style="border-top:2px solid #0ff;background:#0d3d2f;">
        <td colspan="6" style="padding:8px;text-align:right;font-weight:bold;font-size:13px;">TOTAL NET PnL:</td>
        <td style="text-align:right;padding:8px;font-size:15px;font-weight:bold;color:{'#0f0' if total_pnl >= 0 else '#f44'};">{fmt_money(total_pnl)}</td>
      </tr>
    </table>
  </div>

  <!-- SYSTEM -->
  <div style="display:grid;grid-template-columns:2fr 1fr;gap:10px;margin-bottom:10px;">
    <div style="background:#0a2118;padding:12px;border-radius:6px;border:1px solid #333;">
      <h3 style="margin:0 0 8px 0;color:#0ff;font-size:12px;">⚙️ SYSTEM</h3>
      <div style="display:grid;grid-template-columns:repeat(4,1fr);gap:8px;font-size:11px;">
        <div><span style="color:#888;">WS:</span> <span style="color:{ws_color};">{esc(str(ws_status)[:18])}</span></div>
        <div><span style="color:#888;">Reconn:</span> {fmt_int(reconnects)}</div>
        <div><span style="color:#888;">Errors:</span> {fmt_int(errors)}</div>
        <div><span style="color:#888;">Ticks:</span> {fmt_int(ticks_saved)}</div>
      </div>
      <div style="display:grid;grid-template-columns:repeat(3,1fr);gap:8px;font-size:10px;margin-top:6px;color:#666;">
        <div>ATR(spot): {fmt2(spot_atr)}</div>
        <div>Q-Drops: {fmt_int(tickq_drops)}</div>
        <div>Run: {esc(_G('RUN_ID', ''))}</div>
      </div>
    </div>

    <div style="background:#0a2118;padding:12px;border-radius:6px;border:1px solid #333;">
      <h3 style="margin:0 0 8px 0;color:#0ff;font-size:12px;">📊 INDICATORS</h3>
      <div style="font-size:11px;">
        <div style="margin-bottom:4px;"><span style="color:#888;">Spot VWAP:</span> {fmt2(spot_vwap)}</div>
        <div style="margin-bottom:4px;"><span style="color:#888;">Opt VWAP(1m):</span> CE {fmt2(ce_vwap1m)} | PE {fmt2(pe_vwap1m)}</div>
        <div style="margin-bottom:4px;"><span style="color:#888;">RSI(3,15m live):</span> CE {fmt2(ce_rsi3)} | PE {fmt2(pe_rsi3)}</div>
        <div><span style="color:#888;">ATR7(opt):</span> CE {fmt2(ce_atr7)} | PE {fmt2(pe_atr7)}</div>
      </div>
    </div>
  </div>

  <!-- LAST SIGNALS -->
  <div style="background:#0a2118;padding:12px;border-radius:6px;border:1px solid #333;">
    <h3 style="margin:0 0 8px 0;color:#0ff;font-size:12px;">📡 LAST SIGNALS</h3>
    <div style="font-size:10px;line-height:1.6;font-family:monospace;">
      <div style="padding:4px;background:#0d2d1f;border-radius:3px;margin-bottom:4px;"><strong style="color:#0ff;">GL:</strong> {esc(gl_sig)[:120]}</div>
      <div style="padding:4px;background:#0d2d1f;border-radius:3px;margin-bottom:4px;"><strong style="color:#0ff;">GB:</strong> {esc(gb_sig)[:120]}</div>
      <div style="padding:4px;background:#0d2d1f;border-radius:3px;"><strong style="color:#0ff;">N19:</strong> {esc(n19_sig)[:120]}</div>
    </div>
  </div>

</div>
"""
    except Exception as e:
        return f"""
<div style="font-family:monospace;background:#200;color:#f44;padding:20px;border-radius:8px;">
  <h2>⚠️ Dashboard Error</h2>
  <p><strong>Error:</strong> {esc(str(e))}</p>
  <pre style="font-size:10px;color:#888;overflow:auto;max-height:220px;">{esc(traceback.format_exc())}</pre>
</div>
"""

globals()["dashboard_html"] = dashboard_html

# ----------------------------
# DASHBOARD LOOP
# ----------------------------
_last_render_hash = None

def _safe_dashboard_html():
    try:
        return dashboard_html()
    except Exception as e:
        return f"<div style='color:red;'>Dashboard error: {esc(e)}</div>"

def _inline_dash_update(h: str):
    try:
        update_display(HTML(h), display_id=DASH_DISPLAY_ID)
    except:
        if FAIL_FAST: raise
        pass

def _inline_dash_loop():
    global _last_render_hash
    while not STOP_EVENT.is_set():
        try:
            h = _safe_dashboard_html()
            hh = hash(h)
            if hh != _last_render_hash:
                _inline_dash_update(h)
                _last_render_hash = hh
        except:
            if FAIL_FAST: raise
            pass
        time.sleep(DASH_REFRESH_SEC)

def start_inline_dashboard():
    if globals().get("_INLINE_DASH_STARTED", False):
        return
    globals()["_INLINE_DASH_STARTED"] = True
    try:
        display(HTML("<div style='color:#0f0;font-family:monospace;'>🚀 Starting dashboard...</div>"), display_id=DASH_DISPLAY_ID)
    except:
        if FAIL_FAST: raise
        pass
    t = threading.Thread(target=_inline_dash_loop, daemon=True, name="Dashboard")
    t.start()
    print("✅ Dashboard started")

# ----------------------------
# WATCHDOG (stale ticks warning)
# ----------------------------
def _watchdog_loop():
    while not STOP_EVENT.is_set():
        time.sleep(10)
        try:
            ws_status = str(_G("ws_status", ""))
            noww = time.time()
            ce_last = float(_G("last_ce_tick_wall", 0) or 0)
            pe_last = float(_G("last_pe_tick_wall", 0) or 0)
            ce_age = (noww - ce_last) if ce_last > 0 else None
            pe_age = (noww - pe_last) if pe_last > 0 else None

            if "CONNECTED" in ws_status.upper():
                if (ce_age is not None and ce_age > 30) or (pe_age is not None and pe_age > 30):
                    tg_fn = _G("tg_throttled")
                    if callable(tg_fn):
                        tg_fn("stale", f"⚠ Stale ticks CE={ce_age:.0f}s PE={pe_age:.0f}s", 120)
        except:
            if FAIL_FAST: raise
            pass

if not globals().get("_WATCHDOG_STARTED", False):
    globals()["_WATCHDOG_STARTED"] = True
    threading.Thread(target=_watchdog_loop, daemon=True, name="Watchdog").start()

# ----------------------------
# SHUTDOWN
# ----------------------------
def shutdown(reason: str = "MANUAL"):
    print(f"🛑 Shutting down: {reason}")
    try:
        tg_fn = _G("tg_throttled")
        if callable(tg_fn):
            tg_fn("shutdown", f"🛑 Shutdown: {reason}", 5)
    except:
        if FAIL_FAST: raise
        pass
    try:
        STOP_EVENT.set()
    except:
        if FAIL_FAST: raise
        pass
    try:
        fn = _G("stop_ws")
        if callable(fn):
            fn()
    except:
        if FAIL_FAST: raise
        pass

globals()["shutdown"] = shutdown

def _on_exit():
    try:
        STOP_EVENT.set()
    except:
        if FAIL_FAST: raise
        pass

atexit.register(_on_exit)

# ----------------------------
# HELPER: QUICK STATUS
# ----------------------------
def check_status():
    """Quick console status check."""
    ws_status = _G("ws_status", "UNKNOWN")
    reconnects = _G("reconnects", 0)
    errors = _G("errors", 0)
    tickq_drops = _G("tickq_drops", 0)

    noww = time.time()
    ce_last = float(_G("last_ce_tick_wall", 0) or 0)
    pe_last = float(_G("last_pe_tick_wall", 0) or 0)
    sp_last = float(_G("last_spot_tick_wall", 0) or 0)

    def _age(x):
        return (noww - x) if x > 0 else None

    print("==== STATUS ====")
    print("WS:", ws_status, "| Reconnects:", reconnects, "| Errors:", errors, "| QDrops:", tickq_drops)
    print("TickAge(s) SPOT:", _age(sp_last), "CE:", _age(ce_last), "PE:", _age(pe_last))
    print("SafeStop:", _G("SAFE_STOP_REASON", ""))
    print("BlockStop:", STOP_EVENT.is_set())
    print("===============")

globals()["check_status"] = check_status

# ----------------------------
# START DASHBOARD
# ----------------------------
start_inline_dashboard()

print("ℹ Dashboard running in background.")
print("💡 Useful commands:")
print("   check_status()  - Check WS and system status")
print("   shutdown()      - Stop everything")
# ----------------------------
# KEEP NOTEBOOK ALIVE (COLAB)
# ----------------------------
KEEP_NOTEBOOK_ALIVE = True  # set False if you don't want blocking mode

if KEEP_NOTEBOOK_ALIVE:
    print("🟢 LIVE MODE: Dashboard will keep running until you stop it manually.")
    print("✅ To stop: click the ⏹ Stop button (Interrupt) OR Runtime > Interrupt execution")
    try:
        while not STOP_EVENT.is_set():
            time.sleep(30)
    except KeyboardInterrupt:
        # When you press stop, we shutdown gracefully
        shutdown("MANUAL_INTERRUPT")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.5/771.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 692.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 2.0 MB/s eta 0:00:00
Mounted at /content/drive
📁 REPORT FOLDER: /content/drive/MyDrive/NANDINI19_REPORTS/2026-02-27/
🆔 RUN_ID: 080345

🔐 KITE LOGIN URL:
 https://kite.zerodha.com/connect/login?api_key=4gd0bgmg4f57ghxi&v=3 

✅ ACCESS_TOKEN generated & saved for today.
✅ PAPER MODE LOCKOUT enabled: live order endpoints blocked.
[PARAM_SET] BANKNIFTY_MONTHLY | imb=1.44 spread=8.0 sl_mult=2.0 min_sl=28.0 trail=T3 lot=30 GB_SL=ATR
📌 PremiumBand=160-330 | MaxSpread=8.00
📍 SPOT: NSE:NIFTY BANK | token=260105
📥 Loading NFO instruments (options)…


✅ Dashboard started
ℹ Dashboard running in background.
💡 Useful commands:
   check_status()  - Check WS and system status
   shutdown()      - Stop everything
🟢 LIVE MODE: Dashboard will keep running until you stop it manually.
✅ To stop: click the ⏹ Stop button (Interrupt) OR Runtime > Interrupt execution
🔥 TRUE OPEN PE=700.00 | src=WS_OHLC_OPEN_DAY | pri=95 | ts=09:15:00
🔥 TRUE OPEN CE=1029.00 | src=WS_OHLC_OPEN_DAY | pri=95 | ts=09:15:01
✅ TRUE OPEN LOCKED | CE=1029.0 (WS_OHLC_OPEN_DAY) | PE=700.0 (WS_OHLC_OPEN_DAY) | lock_time=2026-02-27 09:15:01 IST
[15:15:00] ✅ 15:15 square-off done
✅ 15:15 square-off done
[15:35:14] ℹ Market closed. Stopping WS.
ℹ Market closed. Stopping WS.
[15:35:15] 🔌 WS stopped
🔌 WS stopped


ERROR:kiteconnect.ticker:Connection closed: None - None


[15:35:15] ⚠ WS closed: None - None
⚠ WS closed: None - None
